# 1. Introduction

The purpose of this notebook is to perform cleaning of the Dublin Bus historical data, merge it with weather data, and train a machine learning model.

# 2. Setup & Data Load

Import required modules and packages.

In [ ]:
# import time so that various tasks can be timed
import time

# import math for mathematical functions
import math

# import csv so that results can be output to csv
import csv

# import pandas and numpy for data analysis
import pandas as pd
import numpy as np

# import transform_data function for transforming data into segment format
from transform_data import transform_data

# import convert_timestamp for various timestamp conversion functions
import convert_timestamp

# import matplotlib & seaborn for plotting and visualisaion
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

# import from sklearn for model training
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

# import pickle so that models can be saved to file
import pickle

Set the max number of columns & rows to display.

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

Load the bus data from file.

In [ ]:
df = pd.read_csv('/data_analytics/data/december_bus_data.csv', sep=";", na_values=['\\N'])

Perform a check to see how many rows and columns are in the file.

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("Before any data cleaning, the dataframe contains", rows, "rows and", cols, "columns.")
print()

# 3. Initial Checks on the Bus Data

## 3.1 Check for Duplicate Rows & Columns

In [ ]:
print()
print('Duplicate rows:', df.duplicated()[df.duplicated() == True].shape[0])
print('Duplicate columns:',df.columns.size - df.columns.unique().size)

There are no duplicate rows or columns in the bus data.

## 3.2 Assign Features as Continuous or Categorical

Assign categorical and continous features, and update the type of all categorical features to 'category'.

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'progrnumber', 'stoppointid']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

## 3.3 Check for Constant Categorical Features

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

There are no constant features to be droppped.

## 3.4 Check for Constant Continuous Features

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no constant features to be droppped.

# 4. Checks for Missing Data

## 4.1 Categorical Features

Print details for the categorical columns:

In [ ]:
df[categorical_columns].describe().T

There is a full count for all categorical features.

## 4.2 Continuous Features

Print details for the continuous columns:

In [ ]:
df[continuous_columns].describe().T

There is a full count for all continuous features.

# 5. Drop Legacy Bus Stops

We will remove data for any bus stops that don't appear in the latest GTFS data:

In [ ]:
legacy_stops = (124,128,132,139,140,347,349,401,403,430,431,432,433,434,462,464,465,466,467,468,486,661,662,663,674,765,788,805,806,815,816,863,864,865,866,867,869,870,873,874,935,936,951,952,953,963,964,965,966,967,968,973,974,975,976,979,988,989,990,991,997,998,999,1000,1001,1002,1003,1004,1006,1007,1008,1009,1010,1012,1039,1040,1041,1050,1131,1132,1133,1134,1138,1139,1252,1364,1391,1392,1394,1395,1400,1421,1513,1530,1545,1601,1602,1603,1605,1644,1645,1646,1650,1651,1652,1766,1824,1825,1830,1831,1856,1857,1880,1881,2027,2028,2029,2030,2037,2038,2047,2048,2049,2050,2052,2079,2110,2111,2112,2113,2116,2117,2130,2199,2212,2226,2228,2237,2290,2294,2345,2363,2364,2449,2451,2464,2520,2521,2533,2534,2544,2545,2576,2618,2619,2620,2762,2763,2780,2781,2782,2783,2784,2785,2786,2787,2788,2789,2790,2791,2793,2794,2796,2797,2798,2799,2800,2801,2806,2807,2808,2872,2873,2874,2877,2878,2879,2880,2882,2883,2885,2886,2887,2960,3046,3058,3078,3083,3087,3094,3095,3096,3097,3098,3099,3100,3101,3102,3103,3105,3106,3107,3108,3111,3165,3166,3167,3168,3169,3171,3173,3174,3175,3176,3177,3178,3179,3182,3183,3187,3188,3189,3190,3191,3192,3193,3194,3195,3198,3199,3200,3201,3241,3242,3248,3249,3254,3255,3256,3260,3261,3262,3265,3266,3267,3268,3269,3270,3271,3272,3273,3274,3275,3276,3277,3278,3279,3285,3289,3290,3291,3292,3293,3294,3295,3296,3297,3298,3306,3307,3309,3312,3343,3346,3353,3354,3357,3358,3360,3361,3362,3442,3534,3545,3546,3547,3548,3551,3560,3561,3586,3739,4095,4097,4103,4104,4105,4106,4108,4109,4110,4111,4112,4113,4137,4138,4139,4140,4142,4143,4144,4145,4146,4147,4167,4169,4176,4183,4184,4185,4186,4187,4188,4189,4190,4192,4194,4196,4199,4232,4234,4235,4236,4237,4238,4239,4240,4241,4242,4243,4244,4289,4290,4291,4292,4293,4294,4313,4326,4327,4332,4337,4341,4357,4359,4360,4361,4365,4366,4367,4368,4375,4381,4389,4390,4391,4393,4395,4396,4414,4417,4419,4420,4421,4422,4457,4473,4474,4503,4527,4529,4582,4584,4586,4587,4597,4637,4650,4651,4652,4653,4654,4655,4656,4657,4658,4659,4660,4661,4678,4686,4687,4771,4772,4773,4781,4784,4785,4790,4791,4792,4820,4826,4849,4983,5006,5007,5009,5011,5034,5055,5088,5089,5105,5106,5138,5141,5167,5168,5190,6000,6009,6019,6039,6058,6073,6082,6103,6126,6127,6128,6133,6134,6138,6139,6179,6180,6196,6197,6198,6199,6200,6201,6207,6333,7026,7129,7155,7164,7165,7205,7220,7269,7279,7295,7297,7298,7323,7324,7351,7360,7369,7372,7373,7393,7394,7404,7405,7407,7447,7448,7457,7478,7479,7480,7481,7483,7484,7485,7486,7487,7490,7492,7495,7497,7500,7501,7502,7503,7504,7522,7529,7537,7538,7539,7540,7541,7542,7544,7546,7547,7553,7554,7558,7566,7570,7575,7576,7580,7589,7593,7594,7599,7604,7607,7617,7620,7621,7626,7627,7634,7635,7636,7637,7638,7647,7648,7651,7652,7653,7655,7658,7659,7662,7666,7668,7669,7682,7685,7688,7689)

In [ ]:
df = df.loc[~df['stoppointid'].isin(legacy_stops)]

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After removing legacy stops, the dataframe contains", rows, "rows and", cols, "columns.")
print()

# 6. Transform the Bus Data

Bus data must be transformed so that each row of data holds information on one journey segment. This is done by calling the *transform_data* function:

In [ ]:
df = transform_data(df)

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After transforming the data, the dataframe contains", rows, "rows and", cols, "columns.")
print()

## 6.1 Check for Missing Data

First re-assign the transformed data as continuous or categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid',  \
                         'progrnumber_first', 'stoppointid_first', \
                          'progrnumber_next', 'stoppointid_next']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

Then check for missing data:

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no rows with missing data.

# 7. Import Weather Data

The cleaned weather data is loaded from a csv:

In [ ]:
df_weather = pd.read_csv('/data_analytics/data/weather_cleaned.csv', sep=",", na_values=['\\N'])

Assign features as continuous or categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns_weather = df_weather[['record_date']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns_weather:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns_weather = df_weather.select_dtypes(['float64', 'int64']).columns

# 8. Combine Bus and Weather Data

## 8.1 Split Timestamp for Weather Data

To merge the data, timestamps must be split into month, day and hour.

New features are added:

In [ ]:
df_weather['month'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_month_weather(x))

In [ ]:
df_weather['day'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_day_weather(x))

In [ ]:
df_weather['hour'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_hour_weather(x))

New features are updated to be categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns_weather = df_weather[['record_date', 'month', 'day', 'hour']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns_weather:
    df_weather[column] = df_weather[column].astype('category')

## 8.2 Split Timestamp for Bus Data

### 8.2.1 Add a Month Feature

First check that there are no rows of data where the month is not correct:

In [ ]:
df.loc[df['dayofservice'].str[5:7] != '12']

Then check if there is any data on the last day of the month that runs into the next month:

In [ ]:
df.loc[(df['dayofservice'].str[8:10] == '31') & (df['actualtime_arr_stop_first'] >= 86400)]

In [ ]:
df['month'] = 12

### 8.2.2 Add Day & Hour Features

In [ ]:
df.loc[df.actualtime_arr_stop_first < 86400, 'day'] = pd.to_numeric(df['dayofservice'].str[8:10])
df.loc[df.actualtime_arr_stop_first >= 86400, 'day'] = pd.to_numeric(df['dayofservice'].str[8:10]) + 1

In [ ]:
df['hour'] = df['actualtime_arr_stop_first'].map(lambda x: convert_timestamp.timestamp_to_hour_bus(x))

## 8.3 Merge the Dataframes

In [ ]:
df = pd.merge(df, df_weather,  how='left', left_on=['month','day', 'hour'],\
                     right_on = ['month','day', 'hour'])

Check that there are no rows missing weather data:

In [ ]:
df[df.rain.isnull()]

Update the data types for the new dataframe:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

# 9. Create New Features

## 9.1 Weekday Binary Feature

The new feature is added:

In [ ]:
df['weekday'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_weekday_weekend(x))

Bank Holidays are flagged with 0:

In [ ]:
# Make list of bank holidays for 2018 (based on https://www.officeholidays.com/countries/ireland/2018)
holidays = ['2018-01-01 00:00:00', '2018-03-19 00:00:00', '2018-04-02 00:00:00', '2018-05-07 00:00:00', '2018-06-04 00:00:00',\
           '2018-08-06 00:00:00', '2018-10-29 00:00:00', '2018-12-25 00:00:00', '2018-12-26 00:00:00']

In [ ]:
# update the feature to 0 for bank holidays
df.loc[df.dayofservice.isin(holidays), 'weekday'] = 0

## 9.2 Time Difference Feature

The new feature is added:

In [ ]:
df['time_diff'] = df['actualtime_arr_stop_next'] - df['actualtime_arr_stop_first']

Check for rows where the time difference is less than or equal to zero:

In [ ]:
df.loc[df['time_diff'] <= 0]

Drop the rows that fail this test:

In [ ]:
df = df.loc[df['time_diff'] > 0]

## 9.3 Segment Feature

The new feature is added:

In [ ]:
# based on https://stackoverflow.com/questions/11858472/string-concatenation-of-two-pandas-columns
df = df.assign(segment=(df['stoppointid_first'].astype(int)).astype(str).str.cat((df['stoppointid_next'].astype(int)).astype(str), sep='_'))

Drop any segments that don't appear in the GTFS data:

In [ ]:
# make a list of all segments from the GTFS data
gtfs_segments = ['10_12','100_1111','1005_1788','1013_1015','1014_1015','1015_1016','1015_1074','1015_2812','1016_1017','1017_1018','1018_1019','1019_1020','102_6239','1020_1076','1021_6042','1022_1023','1023_1024','1024_895','1025_1026','1026_1027','1027_1028','1028_2818','1035_1314','1036_4709','1037_1038','1038_2998','104_6241','1042_1043','1043_1044','1044_1045','1045_4710','1046_1047','1047_1048','1048_1049','1049_4333','105_6253','1051_1052','1052_1053','1053_1054','1054_1036','1055_2868','1062_1063','1063_1064','1064_1065','1065_1066','1066_1067','1067_1068','1068_1069','1069_1070','107_6125','1070_1071','1071_4528','1072_1352','1072_7577','1073_3641','1074_6074','1076_1021','1076_1077','1077_1078','1077_2479','1078_1079','1079_1080','1080_1081','1081_1082','1081_2914','1082_1083','1083_1085','1085_1086','1085_1121','1086_1087','1087_1088','1088_1089','1089_1090','1090_1091','1090_2413','1091_1092','1092_1093','1093_1094','1094_1095','1095_1096','1096_1097','1096_1101','1097_1099','1099_6132','11_14','110_112','1100_100','1101_1102','1102_1103','1103_1104','1105_1107','1107_1108','1108_1109','1109_1110','1110_1111','1111_1112','1111_2323','1111_2404','1112_1113','1113_1114','1114_1115','1115_2437','1117_1118','1118_1119','1119_1120','112_113','1120_1164','1121_1122','1122_1123','1123_1124','1124_1125','1125_1127','1127_1130','113_114','1130_1140','1130_2550','1135_7064','1137_5131','114_115','1140_1141','1140_2515','1141_1142','1142_1143','1143_1144','1144_1145','1145_4869','115_37','1150_1151','1151_1152','1152_1153','1153_1154','1154_946','1155_1157','1157_1158','1158_1159','1159_1160','116_117','1160_1161','1161_1162','1162_1163','1163_1164','1163_1337','1164_1165','1165_1166','1166_1167','1167_1168','1168_1169','1169_1170','117_118','1170_1069','1171_1172','1172_1173','1172_286','1172_4509','1172_7671','1173_1174','1174_1175','1174_4725','1175_15','1176_1177','1177_1178','1178_1179','1179_5060','1179_7112','118_119','1181_7350','1182_1550','1184_497','1185_1186','1186_1187','1187_1188','1188_216','119_44','1190_1191','1191_1192','1192_1193','1193_1194','1194_1195','1195_664','1196_1197','1197_1198','1198_1199','1198_1222','1198_4782','1199_1200','12_14','1200_4385','1201_1202','1201_4695','1202_1203','1203_6115','1205_1206','1206_1207','1207_1208','1208_1209','1209_1210','1210_1211','1210_3575','1211_3575','1212_1213','1213_1214','1214_1215','1215_1216','1216_1217','1217_4563','1218_1258','1218_1270','1219_1220','1220_1221','1221_664','1222_1223','1223_1224','1224_1225','1225_1226','1226_1227','1227_1228','1227_1253','1228_1244','1229_1231','1231_1232','1231_1235','1232_1233','1233_1234','1234_7358','1235_1251','1237_1238','1238_1239','1239_247','1240_1241','1241_1242','1242_1277','1243_4696','1244_1246','1244_254','1245_4693','1246_1229','1247_1248','1248_1273','1249_1247','1250_4698','1251_253','1251_7130','1253_969','1254_1255','1255_1256','1256_4694','1258_1259','1259_1260','126_6182','1260_1262','1260_6130','1262_4692','1265_6115','1266_1267','1267_1268','1268_1269','1269_1265','127_112','1270_1272','1272_1273','1273_1274','1274_1275','1275_1276','1276_1277','1277_1219','1278_1282','1278_1934','1279_1282','1279_1283','1279_1934','1280_6348','1280_6366','1281_1329','1282_4456','1283_4456','1283_7579','1284_1285','1285_1016','1285_1286','1285_1287','1286_1288','1287_1288','1288_1289','1289_1290','1289_1360','129_130','1290_1291','1291_1292','1291_2389','1292_1293','1293_1294','1294_1295','1295_1296','1296_1297','1297_1298','1298_1121','1298_1299','1299_1300','130_131','1300_1301','1301_1302','1302_1303','1303_1304','1303_5129','1304_1305','1305_1306','1305_1320','1306_1307','1307_1308','1308_1309','1309_1310','131_164','1310_1311','1311_1312','1312_1313','1313_1055','1314_1316','1315_1317','1316_1315','1317_1318','1318_1319','1319_1281','1320_1321','1321_1322','1322_1323','1323_1324','1324_7068','1325_1326','1326_1327','1327_1328','1328_1329','1329_1330','133_134','1330_1331','1330_2919','1331_1332','1332_1333','1333_1334','1334_1335','1334_2985','1334_7293','1335_1336','1336_1337','1337_1338','1338_1339','1339_1340','134_4890','1340_1341','1341_1342','1342_1343','1343_1344','1344_1345','1345_1347','1347_1348','1347_2634','1348_1349','1349_1350','1350_1352','1352_1353','1353_1354','1353_1355','1354_7578','1355_1357','1357_1358','1357_1359','1358_298','1358_319','1358_326','1358_340','1358_342','1358_4494','1358_5192','1358_7564','1358_7665','1359_274','1359_279','1359_319','1359_320','1359_4724','136_137','1360_1362','1362_1363','1363_1365','1365_1365','1365_1366','1366_1367','1367_1369','1367_7455','1369_1370','137_193','1370_1371','1371_1372','1371_1454','1372_1373','1373_1374','1374_1375','1374_1946','1375_1376','1376_1377','1377_1378','1378_1379','1378_4514','1379_1380','138_133','138_167','1380_1381','1381_1382','1382_1383','1383_1384','1384_1385','1385_1348','1386_1387','1387_1388','1388_1389','1389_3355','1396_2317','1397_1398','1398_1399','1399_5148','14_15','14_53','1401_1402','1402_1403','1403_1404','1404_1405','1405_4856','1406_1407','1407_1409','1409_2095','141_142','1410_1411','1411_1412','1412_1413','1413_1414','1414_1415','1415_1417','1417_1418','1418_5149','1419_6094','142_143','1423_1424','1424_1425','1424_2186','1424_2190','1425_1426','1426_1427','1426_1463','1427_1428','1428_1429','1429_1430','143_115','1430_1432','1432_1433','1433_1434','1434_1435','1435_7439','1436_3952','1438_4855','1439_1440','1440_1386','1441_1442','1442_7414','1443_1444','1443_1616','1443_1647','1444_1445','1445_1447','1445_1647','1445_4319','1445_4407','1445_7078','1447_2637','1449_1450','1450_1451','1451_1452','1451_2191','1452_2640','1453_1456','1454_1455','1455_1453','1456_1457','1457_1458','1458_1459','1459_1460','146_147','1460_1462','1462_5149','1463_4399','1464_1469','1465_1466','1466_1467','1467_1468','1468_1377','1469_1465','147_184','1471_1472','1472_4408','1473_1474','1474_1476','1476_7453','1478_1479','1478_315','1479_315','1479_7622','148_149','1480_1358','1480_2912','1481_7320','1482_1494','1484_1481','1485_4518','1486_1487','1487_1488','1488_1489','1489_1490','149_150','1491_1492','1492_1493','1493_1482','1494_1495','1495_1496','1496_1515','1499_1485','15_17','15_834','150_27','1500_302','1505_4509','1506_1507','1507_1508','1508_1509','1509_1510','151_153','1510_1511','1511_1512','1511_1592','1512_1538','1514_980','1515_1499','1517_1678','1518_1519','1519_1520','1520_7661','1521_4546','1523_1524','1524_1525','1525_1526','1526_6080','1528_1710','1529_4542','153_154','1531_1532','1532_1533','1533_1534','1534_1535','1535_1536','1536_1537','1537_186','1538_4543','1538_4548','1539_1540','154_155','1540_7426','1546_7072','1546_7560','1547_1548','1548_1280','1549_1523','155_156','1550_1551','1551_1552','1552_1553','1552_332','1553_7023','1554_1555','1555_1556','1556_1557','1557_1181','1558_1549','1558_2957','1558_4542','1559_1560','1559_987','156_157','156_7430','1560_1280','1566_1567','1567_173','1567_4372','157_158','1573_1574','1574_1575','1575_4583','1576_1577','1577_1578','1578_4364','1579_1580','1579_7673','158_159','1580_7237','1581_1582','1582_4793','1584_1585','1584_983','1585_1586','1586_1587','1586_1600','1587_1591','1587_4539','159_1539','1591_4542','1592_1593','1593_1594','1594_1595','1595_1596','1596_4363','1597_1599','1598_1597','1599_1573','16_73','1600_7701','1604_1573','1606_1607','1607_1608','1608_1609','1609_1610','1610_7471','1614_1615','1615_1478','1616_1617','1617_1618','1618_1619','1619_195','1620_220','1622_1623','1623_1624','1624_1625','1625_1626','1626_1627','1627_1628','1628_1629','1629_1630','1629_6234','1630_3670','1630_7347','1630_7348','1631_1632','1632_5053','1633_1634','1634_1635','1635_1636','1636_1637','1637_1638','1638_1639','1638_992','1639_1640','164_165','1640_231','1641_1642','1642_213','1643_7571','1647_1648','1647_1649','1648_1649','1649_1911','165_166','1654_1655','1655_1656','1656_1657','1657_4458','1658_1659','1659_1660','166_7458','1660_1661','1661_1662','1661_4523','1662_1664','1662_1666','1664_1665','1665_1666','1666_1667','1666_1807','1666_1808','1667_1668','1668_1669','1669_1670','167_168','1670_1671','1671_1672','1672_1673','1673_1674','1674_1675','1675_1849','1675_7380','1676_1517','1678_1679','1679_4520','168_169','1680_1681','1681_1682','1682_1683','1683_1684','1684_1685','1685_1686','1686_1687','1687_1688','1688_1689','1689_1690','1689_4899','169_141','1690_1691','1691_1692','1692_1693','1693_1694','1694_1695','1695_1696','1696_1697','1696_1700','1697_1698','1698_1699','1699_1700','17_18','1700_1701','1700_1906','1701_1702','1702_16','1702_1703','1703_1704','1703_1905','1704_1705','1705_1706','1706_1707','1707_1708','1708_1709','1709_1528','1710_1711','1711_1712','1712_1713','1713_1714','1713_1715','1714_1715','1715_1478','1715_7453','1719_1828','1724_1725','1725_1726','1726_1727','1727_1728','1728_1729','1729_1730','1729_1731','173_174','1730_1731','1731_1732','1731_1738','1732_1733','1733_1734','1734_1735','1735_1736','1736_1737','1737_1738','1738_1739','1738_614','1739_614','174_175','1740_1741','1741_1742','1741_1747','1742_1743','1743_1744','1744_1745','1745_1746','1746_1747','1747_1748','1747_1749','1748_1749','1749_1751','1749_544','175_176','1750_1769','1751_1752','1752_1753','1753_1754','1754_1759','1755_1756','1756_1757','1757_1794','1758_1750','1759_1758','176_181','1760_1761','1761_1762','1762_1763','1763_1764','1764_1765','1765_1767','1769_1770','177_178','1770_1771','1771_1760','1772_1773','1773_1774','1774_1775','1775_1776','1776_1777','1777_1778','1778_1755','178_1539','1782_1783','1783_1784','1784_1785','1785_1786','1786_624','1788_1789','1789_7199','179_7431','1791_1792','1792_1793','1793_962','1794_7469','1795_1806','1796_1797','1797_1798','1798_1799','1799_1800','18_19','1800_1801','1801_1277','1802_1795','1803_77','1804_1803','1805_1662','1805_1806','1806_1660','1807_7167','1808_1813','1808_1816','1808_2961','1808_4744','1808_7389','1808_7475','181_182','1813_1814','1814_1816','1814_1852','1816_1817','1816_6171','1817_1818','1818_1819','1819_1822','182_183','1820_2468','1822_4994','1826_1827','1827_7382','1828_1829','1829_4997','183_184','1833_1834','1834_1820','1834_1835','1835_1836','1836_1837','1837_2338','1838_2338','184_185','1840_1841','1841_1842','1841_4314','1842_7374','1844_6172','1845_1696','1845_7166','1847_1696','1849_1851','185_186','1850_1867','1851_1813','1852_1853','1853_1854','1854_1844','1858_1859','1859_1860','186_187','186_838','1860_4492','1861_1862','1862_1863','1863_1864','1864_1865','1865_1866','1866_1850','1867_4489','1869_1870','187_188','1870_1871','1871_1872','1872_1873','1873_1874','1874_1875','1875_1876','1876_1877','1877_1878','1878_1879','1879_1899','188_189','1882_7379','1883_1884','1884_1902','1887_1888','1888_1889','1889_1890','189_190','1890_7240','1891_7156','1893_1894','1894_1912','1895_1896','1896_1897','1897_6088','1898_1858','1899_1887','1899_6107','19_21','190_1614','190_191','190_192','1902_1903','1903_1904','1904_1840','1905_1804','1905_1906','1906_1907','1906_1909','1907_1908','1908_1909','1909_1713','191_819','1911_1805','1911_1913','1911_802','1912_1895','1913_1914','1914_1805','192_795','192_819','193_138','1934_1935','1934_2310','1934_2385','1935_1445','1935_1937','1937_1938','1938_1939','1939_1940','194_195','1940_1941','1941_1942','1941_4516','1942_1943','1943_1944','1944_1945','1945_1946','1946_1947','1947_1948','1947_2642','1948_1949','1949_1950','195_196','1950_1951','1951_1952','1952_1953','1953_1954','1954_1955','1955_1956','1956_1957','1956_2109','1957_1958','1958_1959','1959_1960','196_197','1960_4379','1963_4549','1964_4674','1965_1966','1966_1975','1968_1969','1969_1970','197_198','1970_1971','1971_2114','1971_2160','1972_1973','1973_1974','1974_1975','1975_1977','1975_1982','1977_1978','1978_1980','198_199','1980_1981','1981_1982','1981_4406','1982_2642','1982_4406','1982_7435','1984_1985','1985_1986','1986_1987','1987_1988','1987_1989','1988_1989','1988_2642','1989_1375','1989_1472','1989_1990','1989_1995','199_1506','199_200','1990_1992','1992_1993','1993_1994','1994_1995','1995_1996','1995_1998','1996_1997','1997_1998','1998_1999','1998_2001','1998_2002','1999_2001','2_192','2_4726','2_4962','200_201','2001_2002','2002_1358','2002_1359','2003_1358','2007_2008','2007_4571','2008_2009','2008_2071','2009_2010','201_148','201_202','2010_435','2011_2012','2012_3322','2013_2014','2014_2016','2014_4636','2015_2017','2016_2015','2016_2017','2017_2018','2017_3259','2017_7362','2018_2019','2019_2020','202_151','2020_2021','2021_2022','2022_4565','2023_2024','2024_2025','2025_6086','2026_7226','203_204','2031_2032','2032_2033','2033_2034','2034_2035','2035_2036','2036_2039','2036_2040','2039_2040','204_205','2040_2041','2040_3047','2041_2042','2041_3071','2042_2043','2043_2044','2044_2045','2045_2046','2046_4568','205_1620','205_206','205_215','2051_6086','2053_2054','2054_2055','2055_4565','2055_4732','2056_2057','2057_2058','2058_2059','2059_2060','206_207','2060_2061','2060_2062','2061_2062','2062_2063','2062_2064','2063_2064','2064_2065','2064_4727','2065_4727','2066_2067','2067_4728','2068_2069','2069_2070','207_208','2070_2084','2071_2072','2072_2073','2073_2074','2074_2075','2075_2076','2076_460','208_209','2080_2081','2081_2082','2082_2083','2083_2084','2084_768','2085_2086','2086_4455','2086_479','2087_7053','2088_2089','2089_763','2089_770','209_210','2094_1406','2095_2096','2096_2097','2097_2099','2099_2101','21_7602','21_85','210_1643','210_211','2101_2102','2102_2103','2102_2726','2103_2332','2103_7154','2105_2728','2105_6140','2109_3427','211_212','2114_2115','2115_6011','2118_2119','2119_2120','212_213','2120_2121','2121_2123','2123_2685','2124_2125','2125_2126','2126_2127','2127_2128','2128_2129','2129_4797','213_214','2133_6244','2134_2135','2134_3378','2135_2164','2136_2137','2137_2138','2137_7201','2138_2139','2139_2140','214_4432','2140_2141','2141_2142','2142_2143','2143_2145','2145_4670','2146_2147','2147_2148','2148_2149','2149_2150','215_216','2150_2151','2151_2152','2152_2153','2153_2154','2154_6245','2156_6014','2158_2159','2159_4409','216_217','2160_2161','2160_4665','2161_4665','2162_2163','2163_2168','2164_2165','2165_2166','2166_2167','2166_4545','2166_4554','2166_7229','2167_3383','2168_4536','2169_2170','217_218','217_242','2170_4672','2171_7160','2172_2159','2172_4409','2173_2174','2174_3425','2176_1982','2176_2177','2177_2178','2178_7354','218_219','2181_2182','2182_2778','2183_2184','2184_2185','2185_1423','2185_7043','2186_2187','2187_2188','2188_2189','2189_1436','219_220','2190_4434','2191_2192','2192_2193','2193_2194','2194_2195','2195_2196','2196_2197','2197_2198','2198_2200','22_23','220_1622','220_221','2200_2201','2201_4401','2201_7239','2202_2203','2203_2204','2204_4888','2205_2687','2206_4798','2207_2208','2208_2210','221_222','2210_2211','2211_2241','2213_2214','2213_4688','2214_2215','2215_2216','2215_7184','2216_4599','2217_2218','2218_2219','2219_2220','2219_3365','222_223','2220_2221','2221_2223','2223_2224','2224_3886','2225_4620','2227_7049','2229_2230','223_224','2230_2231','2231_2233','2232_2234','2233_2232','2234_2235','2235_2236','2235_2239','2236_2238','2236_2239','2238_5056','2239_2241','224_225','2241_2242','2242_2243','2242_7435','2243_2244','2244_2245','2245_2246','2246_2247','2247_2248','2248_2249','2249_2250','225_226','2250_2251','2251_1472','2255_2256','2256_2717','2257_2258','2258_2259','2259_2260','226_228','2260_2261','2261_4469','2262_2264','2264_2265','2265_2269','2269_2270','227_230','2270_2274','2274_7601','2278_2279','2279_2720','2279_6251','228_229','2280_2281','2281_2282','2282_4468','2283_616','2289_1822','229_227','23_24','230_231','231_1641','231_232','2310_2311','2311_2312','2311_5025','2312_2313','2313_2314','2314_2315','2315_2094','2317_2318','2318_2319','2319_2320','232_233','2320_2321','2321_2322','2322_6163','2323_2324','2324_2325','2325_2326','2326_5150','2327_2328','2328_2329','2329_2330','233_234','2330_2331','2331_1397','2331_1424','2332_2333','2333_2334','2334_4662','2335_2336','2336_2337','2337_2339','2338_1840','2338_1845','2339_2340','2339_2420','234_235','2340_2601','2346_4435','2349_2351','235_236','2351_2558','2351_2621','2351_4445','2352_2573','2353_2357','2353_4643','2357_2358','2357_2359','2357_2566','2358_2359','2359_2360','2359_7122','236_1190','236_237','2360_4340','2360_4348','2369_2370','237_213','2370_2371','2371_2372','2372_2373','2373_1424','2373_2377','2374_2146','2377_2378','2377_2431','2378_2183','2379_2380','2380_2382','2382_5099','2383_1479','2383_2002','2383_2003','2385_2386','2386_2387','2387_2388','2388_1290','2389_2390','2390_2391','2391_2392','2392_2393','2393_2394','2394_1090','2394_2395','2395_2396','2396_2397','2397_2398','2398_2399','2399_2400','24_25','2400_2401','2401_2402','2402_2403','2403_1096','2404_2405','2405_2406','2406_2408','2408_2409','2409_2410','2410_2411','2411_2412','2412_2438','2413_2414','2414_2415','2415_2416','2416_2417','2417_2418','2418_2419','2419_2452','242_243','2420_2421','2421_2423','2423_2424','2424_2447','2425_2426','2426_2427','2427_2428','2428_2429','2429_2430','243_244','243_254','2430_2371','2431_2432','2432_2433','2433_2327','2433_2434','2434_2435','2435_2436','2436_2437','2437_1117','2437_2438','2438_2439','2439_2440','244_245','2440_2441','2441_2442','2442_2443','2443_2444','2444_1344','2445_2457','2446_2459','2447_5133','2448_5026','245_246','2450_2490','2452_2453','2453_2454','2454_2455','2455_2456','2456_2445','2457_4392','2458_2446','2459_2460','246_248','2460_2461','2461_2462','2462_2463','2463_2431','2465_2466','2466_2467','2467_2494','2468_7319','247_1240','2471_2472','2472_2473','2473_2474','2474_2475','2475_2476','2476_2477','2477_2478','2478_1170','2479_2480','248_249','2480_2481','2481_2482','2482_2483','2483_2484','2484_2485','2485_2486','2486_2487','2487_2488','2488_2489','2489_2448','249_250','2490_2491','2491_2492','2492_2493','2493_2465','2494_2495','2495_2496','2496_2497','2497_2471','2498_2499','2499_2500','2499_7216','25_27','250_251','2500_2501','2501_7623','2508_2278','251_252','2510_7242','2515_2516','2516_2517','2517_2518','2518_4749','252_255','2522_2524','2523_2532','2524_2523','253_5033','2532_4877','2535_2536','2536_2616','2538_2606','2539_2540','254_1246','254_244','2540_2542','2540_4882','2542_3004','2547_2548','2548_2600','2548_946','255_234','2550_2516','2550_2551','2551_2553','2553_2554','2554_2425','2554_2555','2555_2556','2556_2557','2557_4436','2558_2559','2558_6170','2559_2560','2560_2561','2561_2562','2562_2564','2564_4927','2566_2567','2567_2568','2568_2569','2569_2570','2569_7550','2570_2571','2570_7222','2571_2359','2573_2574','2574_2575','2574_2577','2575_4929','2577_2578','2578_2579','2579_7395','2583_2584','2584_2585','2585_2586','2586_2587','2587_2588','2588_2589','2589_2590','2590_2591','2591_2592','2592_2357','2594_2595','2595_2596','2596_2597','2596_2611','2597_2599','2599_2600','2599_4577','2600_1155','2601_4446','2602_2613','2603_2557','2604_2605','2605_2538','2606_2607','2607_2608','2608_2609','2609_2610','261_270','261_271','2610_2554','2611_2602','2613_2614','2614_2615','2615_2535','2616_2617','2617_2557','262_270','2621_2622','2622_2623','2622_2750','2623_2624','2624_4441','2626_2669','2628_2629','2629_6095','263_271','263_277','263_4508','263_4725','263_6071','2631_2359','2632_2633','2633_2369','2634_2635','2635_2636','2636_2383','2636_5097','2637_1956','2637_2638','2637_2721','2638_2640','2638_2721','264_6059','2640_2641','2641_1948','2641_2643','2642_2643','2642_7435','2643_2644','2644_2645','2644_2709','2645_2646','2646_2647','2647_2648','2648_2649','2649_2650','265_271','265_6059','2650_2652','2652_2653','2653_2654','2654_2655','2655_2656','2656_2657','2656_2668','2657_2658','2658_2659','2659_2660','2660_2661','2661_2662','2662_2909','2663_2664','2664_2665','2665_2666','2666_2667','2667_2696','2668_2672','2669_4641','2670_2910','2672_2673','2673_2206','2674_2675','2675_2124','2676_2118','2678_2681','2679_4691','2680_2676','2680_4677','2681_2680','2682_4796','2685_2686','2686_7510','2687_2688','2688_2689','2689_2696','2696_2697','2697_2700','2697_2713','27_28','270_335','270_336','2700_2701','2701_2702','2702_2703','2703_2704','2704_2706','2706_6070','2708_5134','2709_2710','271_1278','271_340','271_404','271_4521','271_496','2710_2711','2711_2712','2712_2655','2713_2714','2714_2715','2715_2716','2716_2718','2717_2257','2718_2719','2718_7435','2719_1989','2719_7435','272_335','2720_2280','2721_7012','2722_1471','2722_4413','2725_2737','2726_2727','2727_2105','2728_2729','2729_2730','273_405','2730_6083','2733_2734','2734_5169','2735_2736','2736_2725','2737_2738','2738_2739','2739_7177','274_1505','274_2','274_7','2740_2741','2741_2742','2742_2743','2743_2745','2745_2746','2746_2747','2747_2748','2748_7062','2750_2751','2751_2752','2752_2753','2753_2754','2754_2755','2755_2756','2756_2764','2764_7178','2765_2766','2766_2767','2767_2772','2768_2769','2769_2770','277_288','277_292','277_336','2770_2771','2771_2773','2772_2768','2773_2774','2774_2777','2775_2181','2777_2775','2778_2185','278_10','278_2','278_7615','278_8','279_3','279_7','2795_756','28_29','2804_376','2809_2810','2809_408','2809_7612','281_261','281_4','281_6','281_7','2810_2811','2810_7385','2810_7387','2810_749','2810_750','2810_842','2810_843','2811_749','2811_842','2811_843','2812_2813','2813_2814','2814_852','2815_2816','2816_2817','2817_2818','2818_2819','2819_2820','2819_3012','2820_2822','2822_2823','2823_5032','2824_2825','2825_2826','2826_2827','2826_2842','2827_2828','2828_2829','2829_2830','2830_2831','2831_2832','2832_2833','2832_4349','2833_6006','2833_7375','2835_7176','2836_2837','2837_2838','2838_2839','2839_2840','284_4504','2840_2841','2840_2845','2841_2865','2842_2843','2843_2844','2844_2845','2845_2846','2846_2847','2847_4397','2848_2849','2849_2850','2850_2851','2851_2852','2852_4707','2852_4987','2853_2854','2854_2855','2855_2856','2856_2857','2857_2858','2858_2859','2859_2860','286_271','2860_2861','2861_2829','2861_2862','2862_2863','2863_2864','2864_4486','2865_2866','2866_2825','2866_2892','2866_6044','2867_1035','2868_2892','288_1171','289_496','2892_2893','2893_2894','2894_2895','2895_2896','2896_2897','2897_1062','2897_2898','2898_3562','2899_884','29_4680','2900_2901','2901_2902','2902_2903','2903_1074','2906_2907','2907_2908','2909_2906','2909_6020','2910_2911','2911_7507','2912_1358','2914_2915','2914_2989','2915_2916','2916_2917','2917_2918','2918_1300','2919_2920','292_1171','2920_2921','2921_2922','2922_2923','2923_2924','2924_2925','2925_2948','2925_7442','2927_2929','2929_2930','2930_7441','2934_2935','2935_2936','2936_2937','2937_2938','2938_7440','2940_2948','2941_2942','2942_2943','2942_7065','2943_2944','2944_2945','2945_2946','2946_2947','2947_1305','2948_2949','2949_2950','2950_2954','2954_2955','2954_7067','2955_7067','2956_2965','2957_2958','2958_4938','2961_4901','2962_2963','2963_2964','2964_4175','2965_2967','2965_2983','2966_2967','2967_2968','2968_2969','2969_2970','297_407','297_496','2970_2971','2970_4988','2971_5051','2976_2977','2977_2978','2978_2979','2979_2980','298_496','2980_2981','2981_2954','2981_2991','2983_2984','2984_2927','2984_2941','2984_7449','2985_2986','2986_2988','2988_3001','2989_2990','299_497','2990_1042','2991_2992','2992_1325','2993_2994','2994_4180','2995_3154','2996_3258','2997_4153','2998_2999','2999_3000','3_12','3_192','30_69','300_496','300_497','3000_3001','3001_3002','3002_1166','3003_2522','3004_4756','3005_4757','3006_4871','3007_6283','3008_6089','301_336','301_4495','3011_2896','3012_877','3014_3015','3015_3016','3016_3017','3017_3018','3018_3019','3019_3020','302_333','302_340','302_4495','3020_3021','3021_3022','3022_441','3023_3024','3024_3025','3025_3026','3026_3027','3027_3028','3028_3029','3029_3030','3030_3031','3031_875','3032_3033','3033_3034','3034_3036','3034_3114','3036_3037','3037_3038','3038_3039','3039_3040','3040_3041','3041_2035','3041_3042','3042_2035','3045_4150','3047_3202','3048_3049','3049_3050','3050_3051','3051_3052','3052_3053','3053_3054','3054_3055','3055_3056','3056_3057','3057_3059','3057_4830','3059_3060','3060_3061','3061_3062','3062_3063','3063_3064','3064_3065','3065_3066','3066_3067','3067_3202','3068_3069','3069_3070','307_308','3070_2041','3071_3072','3072_3073','3073_3074','3074_3075','3075_3076','3076_3077','3077_3079','3077_3084','3079_3080','308_309','3080_3081','3080_3082','3081_3084','3081_7188','3082_3084','3082_3114','3082_7660','3084_469','3085_3034','3088_3704','3089_3090','309_2383','309_7412','3090_3706','3091_3092','3092_3093','3093_3535','31_32','3112_3924','3114_3115','3114_4353','3115_3116','3116_3117','3116_6085','3117_3118','3118_3119','3119_3120','312_1444','3120_3122','3122_3123','3123_3124','3124_3125','3125_3126','3126_3127','3127_3128','3128_3130','3129_3130','3130_3131','3131_3132','3132_3133','3133_3134','3133_5046','3134_3135','3135_3136','3135_7363','3136_3138','3138_3139','3139_4124','3140_3141','3141_3142','3142_3143','3142_3224','3143_3144','3144_3145','3144_5047','3144_7639','3145_3146','3145_5128','3146_3147','3147_3148','3148_3149','3148_5127','3149_3150','315_313','315_335','315_336','315_404','315_406','315_4495','3150_3151','3151_3153','3152_2995','3153_3152','3154_3155','3155_3156','3156_3158','3158_3159','3159_3160','3160_3161','3161_3162','3162_3163','3163_3164','3164_3079','3164_3084','3164_7188','317_312','318_312','3181_451','3184_449','319_1443','319_274','319_278','319_279','319_281','319_7491','3196_3345','32_33','320_278','320_281','3202_3203','3203_3204','3204_3205','3204_4568','3204_7641','3205_3206','3206_3207','3207_3208','3208_3209','3209_3210','3210_4981','3211_3212','3212_3213','3213_3214','3214_3215','3215_3216','3216_3217','3217_3218','3218_3219','3219_3220','3219_3537','322_323','322_7697','3220_3221','3220_5046','3221_4982','3222_3223','3223_3224','3224_3225','3225_3226','3226_3227','3227_3228','3228_3229','3229_7472','3229_7644','323_332','323_6234','3230_3231','3230_3232','3231_3246','3232_3233','3233_3234','3234_4462','3235_3236','3236_3237','3237_3238','3238_3239','3239_3240','324_1555','324_1634','324_327','324_6235','3240_3048','3240_3068','3243_3244','3244_3245','3245_3212','3246_3247','3247_7056','325_1443','3258_2060','3259_7361','326_1443','327_6016','327_7113','328_1443','3280_3282','3282_3479','3283_3284','3284_3310','329_1443','33_7270','3310_3481','3317_3318','3318_3320','3320_2064','3321_2066','3321_4727','3322_3563','3328_3329','3329_3330','333_747','3330_3331','3331_3351','3332_3333','3333_3334','3334_3352','3335_3336','3336_3337','3337_3338','3338_3339','3339_3340','334_406','334_7581','3340_4324','3344_7421','3345_4727','335_1278','335_404','335_4521','3351_3332','3352_3335','3355_3356','3356_1441','336_1278','336_1279','336_7581','3365_3366','3366_3367','3367_3886','3368_4626','3368_7137','3370_7168','3370_7231','3372_3373','3373_3374','3374_2233','3376_3377','3377_3378','3378_3379','3379_4669','3383_3384','3384_3385','3385_3386','3386_3387','3387_4559','3389_3390','3390_6119','3392_3393','3393_3394','3394_3396','3396_3397','3397_3398','3398_3399','3399_6187','340_350','3400_3401','3401_3402','3402_3403','3403_3404','3404_3405','3405_3406','3406_3410','3407_3408','3408_3409','3409_4558','3410_3407','3411_3412','3412_3413','3413_3414','3414_3415','3415_3416','3416_3417','3417_3418','3418_3419','3419_2374','3419_3420','3419_4762','342_1279','342_4495','342_4521','3420_3421','3421_3422','3422_3423','3422_4671','3423_2156','3423_4409','3425_2176','3426_3434','3427_3428','3428_3429','3429_3432','3432_3426','3432_3434','3434_3435','3434_3453','3435_3453','3436_7432','3437_3439','3437_3440','3438_5022','3439_3440','3440_3441','3440_4557','3441_4560','3443_3444','3444_3445','3444_3447','3445_3446','3446_3447','3447_3448','3447_3454','3448_3449','3449_3436','345_318','345_320','3453_3437','3454_3455','3455_3456','3455_5022','3456_3457','3457_3438','346_317','3461_4532','3461_4537','3465_4673','3466_3467','3467_3469','3469_3470','3470_3471','3471_3472','3472_3473','3473_3474','3473_5015','3474_3475','3475_3476','3476_3477','3477_3478','3478_3280','3479_4086','3481_3482','3482_3483','3483_3484','3484_3485','3485_3486','3486_3487','3487_3488','3487_7415','3488_3489','3489_3492','3491_6007','3492_2835','3493_3494','3494_3495','3495_3496','3496_3497','3497_3498','3498_3499','3499_3500','35_36','350_2809','350_351','3500_3501','3501_3502','3502_3503','3503_3504','3504_3505','3505_3506','3506_3507','3507_3508','3508_3510','351_352','3510_3512','3512_3514','3514_3515','3515_3516','3516_3517','3517_3518','3518_3519','3519_3521','352_353','3521_3523','3523_3524','3524_3525','3525_3527','3527_3528','3528_3529','3529_3532','353_354','3532_3533','3533_3491','3535_3536','3536_3537','3537_3538','3538_7556','3539_3540','354_355','3540_3541','3541_3542','3542_3543','3542_3544','3543_3141','3544_3552','355_356','3552_3553','3553_3554','3554_3555','3555_3556','3556_3557','3557_7317','356_357','3562_2899','3563_444','3565_3825','3566_3567','3567_7292','3569_3571','357_390','3571_3801','3572_3701','3573_3662','3574_3641','3575_3576','3576_3577','3576_3648','3577_3578','3578_3579','3579_3580','3580_3640','3583_3584','3584_3585','3585_3587','3585_3625','3587_3588','3588_3589','3588_3590','3589_3591','3589_3605','3590_3589','3591_3592','3592_3593','3593_3594','3594_3595','3595_3596','3596_3597','3597_3598','3598_4465','3599_3600','3599_3610','36_37','3600_3601','3601_3602','3602_3604','3604_3616','3605_3606','3606_3607','3607_3608','3607_6051','3608_3609','3609_3598','3610_3611','3611_6135','3612_3613','3613_3615','3615_3616','3616_3618','3618_3619','3619_3620','3619_6087','3620_3621','3620_3622','3621_3622','3622_3623','3623_4387','3624_3587','3625_3626','3626_3627','3627_3628','3628_6052','3629_3630','3630_3631','3631_3632','3632_3633','3633_3634','3634_3635','3635_3636','3636_905','3639_3644','3640_3574','3641_3642','3642_3643','3643_3583','3644_3645','3645_3646','3646_3647','3647_3661','3648_3649','3649_3650','3650_7328','3651_3652','3652_3653','3653_3654','3653_6055','3654_5084','3655_5086','3656_3657','3656_7194','3657_3658','3658_3659','3658_7329','3659_3660','3660_3647','3661_3573','3662_1212','3663_7347','3664_3813','3665_7400','3669_3671','3669_7349','3669_7399','3669_7611','3670_3671','3671_3672','3672_3674','3674_3675','3675_3676','3675_5030','3676_3677','3676_5073','3677_3678','3678_3679','3679_3711','3679_5075','3682_3864','3686_5077','3689_3690','3689_3691','3690_5079','3690_6117','3691_3572','3691_3692','3692_3694','3694_3695','3695_3696','3695_7115','3696_3697','3697_3698','3698_3699','3699_3885','37_38','3701_3702','3702_3088','3704_3705','3705_733','3706_3707','3707_3708','3708_3709','3709_3710','371_391','3710_4925','3711_3712','3711_7189','3712_3713','3712_4916','3712_6359','3713_3714','3714_3715','3714_3719','3715_3755','3715_5175','3718_3720','3719_3718','372_373','3720_7691','3721_3722','3722_5069','3723_3724','3724_3725','3725_3726','3725_3738','3726_7039','3728_3729','3729_3730','373_374','3730_3732','3732_3733','3733_3734','3734_3735','3735_7039','3736_3738','3738_7170','374_375','3740_3741','3741_5165','3743_3780','3744_3715','3744_3745','3745_3746','3746_3749','3748_6050','3749_3750','375_2804','3750_3752','3750_6252','3751_3689','3751_7114','3752_3751','3755_3756','3756_3757','3757_3758','3758_3759','3759_3760','376_377','3760_3761','3761_3762','3762_3763','3763_3764','3764_3765','3765_3766','3766_3767','3767_3768','3768_3770','3769_3771','377_378','3770_3769','3771_3772','3772_3773','3773_3774','3773_5176','3774_7172','3775_3777','3777_3778','3778_7690','378_380','3780_3744','3781_3782','3782_3783','3783_3784','3784_3785','3785_3786','3786_3787','3787_3788','3788_3789','3789_3790','3790_3791','3791_3792','3792_3793','3793_3794','3794_3795','3795_3797','3797_3798','3798_3799','3798_6076','3799_6076','38_39','380_2087','3801_3802','3802_3805','3805_7175','3806_3807','3807_3808','3808_3809','3809_7174','381_382','3811_3812','3812_3664','3813_3814','3814_3815','3815_3816','3816_3566','3818_3819','3819_3820','382_4451','3820_3822','3822_3823','3823_3824','3824_3565','3825_3826','3826_3827','3827_3828','3828_3829','3829_3831','383_384','3831_3832','3832_3833','3833_3834','3834_3835','3835_3836','3836_3837','3837_3838','3838_3840','384_385','3840_3842','3842_3843','3843_3844','3844_3845','3845_3846','3846_3848','3848_3849','3849_3850','385_387','3850_3851','3851_3852','3852_3853','3853_3854','3854_3855','3855_3856','3856_3857','3857_3858','3858_3859','3859_3860','3860_3861','3861_3862','3862_3863','3863_3748','3864_3865','3865_4910','3865_7133','3867_7135','387_388','388_389','3881_5096','3884_3686','3885_1631','3885_3663','3886_3887','3886_3888','3887_3888','3888_3890','3888_7186','389_393','3890_3891','3891_3949','3891_7187','3892_3893','3893_3895','3895_3896','3896_3897','3897_3898','3898_3899','3899_3900','39_116','39_40','390_372','3900_3901','3901_3902','3902_3904','3904_3905','3905_3906','3905_7085','3906_3907','3907_3908','3907_3909','3908_3909','3909_5118','391_392','3910_3911','3911_3912','3912_3913','3913_3914','3914_3925','3914_7075','3917_3918','3918_3926','3918_5114','3919_3920','392_395','3920_3921','3921_3922','3922_3923','3923_3112','3923_7079','3924_3927','3925_3008','3926_7074','3927_3928','3928_3929','3929_7108','393_371','3935_3936','3936_3938','3936_7110','3938_3939','3939_3940','3940_3941','3941_3943','3943_3944','3944_3945','3945_3946','3946_4001','3947_3892','3949_3950','395_396','3950_2229','3950_3372','3952_2190','3954_3955','3955_3956','3956_3957','3957_3958','3957_3993','3958_3959','3959_3960','3959_7234','396_397','3960_3961','3960_6156','3961_3962','3962_3963','3963_3964','3964_3965','3965_3968','3966_5113','3968_3966','3968_7771','397_398','398_399','3980_6089','3981_3982','3982_3983','3982_7704','3983_3985','3985_3986','3986_3987','3986_7313','3987_3989','3989_3990','399_400','3990_3991','3991_3998','3993_3994','3994_3995','3995_3996','3996_3997','3996_3998','3997_3998','3998_3999','3999_4000','4_194','4_4962','4_7555','40_146','40_41','400_279','400_281','400_303','400_319','400_346','400_4522','400_7392','400_7564','400_7581','4000_4001','4001_3949','4001_7193','4002_5112','4003_4404','4004_4005','4005_4006','4006_4007','4007_4008','4008_4010','4009_4011','4010_4009','4011_5110','4012_4013','4013_4014','4014_4098','4015_4017','4017_4018','4018_4019','4019_4020','4020_4100','4021_4022','4022_4023','4023_4024','4024_4027','4027_7250','404_747','405_408','4051_4052','4052_4054','4054_4055','4055_4057','4056_4133','4057_4101','4058_4059','4059_4060','406_2810','406_4530','406_747','4060_4061','4061_4062','4062_4063','4063_4064','4064_4056','4065_4066','4066_4067','4067_4068','4068_4069','4069_4070','407_2498','4070_4071','4071_4072','4072_4073','4073_2583','408_409','4086_4087','4087_4088','4088_4089','4089_4090','409_410','409_411','4090_4091','4091_6097','4093_4094','4094_4096','4096_4114','4098_4015','41_42','410_411','4100_4021','4101_4102','4102_4058','411_412','411_414','4114_4115','4115_4116','4116_4117','4117_4118','4118_4119','4119_4120','412_413','4120_4121','4121_4122','4122_4123','4123_3283','4124_4125','4125_4126','4126_4127','4127_4128','4128_4129','4129_4130','413_414','4130_4131','4131_7294','4132_4134','4132_4171','4133_4065','4134_4135','4135_4136','4136_4207','414_415','414_416','4148_4149','4149_3045','415_416','4150_4151','4151_2997','4151_4156','4153_4154','4154_4416','4156_4168','416_417','4168_4170','417_418','417_485','4170_4132','4170_4153','4171_4172','4172_4173','4173_2962','4175_5091','4177_4178','4178_4179','4179_2993','418_419','4180_4181','4181_4182','4182_4151','419_420','42_43','420_2088','420_421','4201_4202','4202_4203','4203_4204','4203_5090','4204_4205','4205_4206','4206_3140','4207_4208','4208_4209','4209_4210','421_422','4210_4211','4211_4212','4211_4215','4212_4213','4213_5139','4214_4215','4215_4216','4216_4217','4216_4222','4217_4218','4218_4219','4219_4220','422_423','4220_4221','4221_4222','4222_4224','4224_4225','4225_4226','4225_7462','4226_4227','4227_4228','4228_4229','4229_4230','423_424','4230_4231','4230_4248','4231_7352','424_425','4246_4248','4248_4249','4248_4252','4249_4250','425_426','4250_4251','4251_4252','4252_4253','4252_4305','4252_6137','4253_4254','4253_4268','4253_4305','4254_6137','4256_4257','4257_4258','4258_4259','4259_7272','426_427','4260_4261','4260_7296','4261_7271','4262_4263','4263_4264','4264_7296','4265_4266','4266_4253','4266_4267','4266_4307','4267_4268','4268_4269','4268_4273','4269_4270','427_428','4270_4271','4271_4273','4273_4274','4273_4277','4274_4275','4275_4276','4276_4277','4277_4278','4277_7462','4278_4279','4279_4280','428_429','4280_4281','4281_4282','4282_4283','4283_4284','4284_4285','4284_4295','4285_4286','4286_4287','4287_4288','4288_4295','429_3032','429_6334','4295_4296','4296_4297','4296_4301','4297_4299','4299_4300','43_44','4300_4301','4301_4302','4302_4591','4302_5135','4303_4304','4304_4148','4305_4306','4306_4307','4306_4309','4307_4308','4308_4253','4308_4267','4309_4254','4314_1687','4315_4316','4316_4317','4317_7013','4319_1476','4320_1476','4321_7342','4323_1845','4324_4769','4325_3328','4330_3679','4331_4339','4333_4334','4334_4712','4336_1051','4339_7616','4339_7692','4342_4348','4344_2349','4347_2349','4347_7181','4348_4646','4348_7122','4349_3184','435_2011','435_7353','4350_5143','4353_4355','4354_3196','4355_4354','4362_2959','4362_7025','4363_1598','4364_1579','4369_6219','437_459','4371_832','4372_179','4376_4377','4377_4890','4379_1963','4379_1968','4380_1172','4380_496','4382_1185','4382_1196','4383_4699','4384_518','4384_519','4385_1201','4386_653','4387_3634','4388_4700','4392_2458','4397_2848','4399_4400','44_7603','4400_1464','4401_2202','4401_2213','4404_4405','4405_3991','4406_1984','4407_2637','4407_4320','4408_1473','4409_1972','4409_2173','4409_4550','441_442','4410_2678','4411_2239','4411_4715','4413_1476','4413_1480','4413_7453','4415_301','4415_302','4416_4201','442_443','443_444','4432_119','4433_3752','4434_2379','4435_2594','4435_2604','4435_2632','4436_5008','4438_3432','444_445','4440_2628','4441_2626','4445_2352','4446_2603','445_4779','4451_383','4455_381','4456_1284','4458_1658','4460_453','4462_3235','4464_1869','4465_944','4466_633','4468_2283','4469_2262','447_5045','4472_4386','4475_3023','4477_3321','4479_4376','448_3181','4481_4485','4485_822','4486_2865','4489_4747','449_450','4492_1861','4493_4362','4495_1278','4495_1279','4495_2809','4495_4522','4495_4720','4495_746','4495_7581','4498_1660','4499_4934','45_46','450_3181','4500_4481','4501_4935','4502_4936','4504_7028','4508_7402','4509_499','4509_7708','451_447','4513_272','4513_286','4514_4515','4515_1995','4516_1369','4518_509','4520_4825','4521_1283','4521_1934','4521_1935','4522_1934','4523_4524','4524_4322','4525_4526','4526_1700','4528_1072','453_4477','453_454','4530_748','4532_4534','4533_4844','4534_4535','4535_2169','4536_6003','4537_2176','4538_7396','4539_1606','454_4475','4542_1531','4543_1560','4545_4552','4546_1559','4548_1514','4548_980','4549_1964','4550_4551','4551_1965','4552_4554','4554_7016','4554_7229','4554_7230','4555_3418','4555_4576','4557_3441','4558_3411','4559_3389','4560_5126','4561_4629','4562_7386','4563_1218','4564_3443','4564_3444','4565_4566','4566_4567','4567_2031','4567_3238','4568_4569','4569_2023','4569_4570','4570_2056','4571_2013','4571_2014','4573_4893','4574_4575','4575_1680','4576_3418','4577_4578','4578_1100','4583_1518','4583_1576','4588_4589','4589_7364','459_2080','4590_7366','4591_4590','4592_980','4593_4594','4593_4595','4593_7516','4594_4595','4595_4596','4596_4563','4599_2217','4599_4601','4599_7184','46_47','460_2011','4601_5058','4602_4603','4603_4604','4604_4605','4605_4562','4606_7142','4607_4608','4607_7140','4608_4609','4609_4610','461_2068','4610_4611','4611_4612','4612_4613','4613_3370','4616_2227','4617_4555','4619_4561','4620_7563','4623_4619','4624_4625','4625_4628','4626_4627','4627_4624','4628_4623','4629_4630','4630_4631','4631_4632','4632_5164','4633_2234','4636_2016','4640_4347','4640_7214','4641_4642','4642_2355','4643_4644','4644_4645','4645_4440','4646_4647','4647_2346','4647_4435','4662_2335','4663_4664','4664_2160','4665_4666','4666_4667','4667_2162','4667_4668','4668_3376','4668_4677','4669_2146','4669_2165','4670_2165','4670_3420','4671_4672','4672_3465','4673_2172','4674_4663','4677_2136','4680_31','4680_90','4683_107','4688_4689','4689_4410','469_470','4690_4411','4691_2682','4691_4796','4692_1245','4693_1254','4694_1249','4695_1243','4696_4697','4697_1250','4698_4383','4699_4388','47_48','470_471','4700_1266','4705_475','4707_4708','4708_4336','4709_1037','471_472','4710_1046','4712_2853','4713_4714','4714_4960','4717_2499','472_473','4720_1443','4724_263','4725_273','4725_334','4725_4495','4725_6059','4725_6071','4726_7555','4727_4728','4727_768','4728_437','4728_461','473_4705','4731_2023','4732_3243','4742_4743','4743_4744','4743_4745','4744_4745','4745_4989','4746_1845','4747_1845','4747_1882','4749_4750','475_476','4750_3006','4750_4751','4751_4754','4754_4755','4755_3003','4756_3005','4757_4758','4758_6343','4759_4760','476_477','4760_2547','4762_4763','4763_6245','4765_4766','4766_4767','4767_4768','4768_1845','4768_4325','4769_4770','477_478','4770_4765','4779_7057','478_479','478_7053','4782_1782','4788_6273','479_480','4793_6068','4795_2686','4795_4688','4796_4690','4797_7473','4798_2674','4798_4715','4799_2205','48_49','480_481','4800_5157','4804_738','4806_5156','481_482','482_483','4825_7233','483_485','4830_7667','4843_4905','4844_4177','4847_4848','4848_4350','485_487','485_488','4854_1438','4855_1439','4856_4857','4857_307','4861_4862','4863_4864','4864_4865','4865_4577','4869_4870','487_488','4870_3007','4871_4872','4872_4873','4873_4874','4874_2532','4877_4878','4878_7180','488_489','488_491','4880_2539','4880_2606','4882_4883','4883_4884','4884_4885','4885_6343','4886_4887','4887_1150','4888_4798','489_490','4890_4369','4891_7458','4893_4894','4894_4895','4895_7217','4897_4898','4898_4574','4899_4900','49_51','490_491','4900_4746','4901_4902','4902_1672','4903_1883','4903_7219','4905_4906','4906_4907','4907_4908','4908_4909','4909_3884','4909_4921','491_492','491_493','4910_4911','4911_4912','4912_4913','4913_4914','4914_4915','4915_4957','4915_7654','4916_4917','4917_4918','4918_4919','4919_4920','492_493','4920_4910','4921_4922','4922_4923','4923_4924','4924_4958','4925_3679','4925_4926','4926_3678','4927_4928','4928_2352','4929_7460','493_494','4930_4931','4931_7459','4932_4933','4933_2590','4934_4500','4935_4502','4936_79','4937_1547','4938_1549','494_495','495_400','495_496','495_7587','495_7588','4952_768','4953_768','4957_7133','4958_3752','496_497','496_515','496_522','496_523','4960_4932','4962_264','4962_4725','4962_6059','497_515','4974_4976','4976_7126','4978_4979','4979_4986','4980_2289','4981_3211','4984_448','4985_6219','4986_7107','4987_2978','4988_2853','4989_1845','4989_4990','499_500','4990_4991','4991_4992','4992_1719','4994_4995','4995_4996','4996_1826','4997_4998','4998_4999','4999_1833','500_501','5008_4640','501_1484','5013_5014','5014_7291','5015_7290','5022_3461','5022_6002','5023_5024','5024_5161','5025_4854','5026_2450','5028_3696','5028_7115','5029_5028','5030_5031','5031_6033','5032_2824','5032_2867','5033_7130','5044_3751','5044_4433','5045_4460','5046_7639','5047_3146','5047_3224','5051_5052','5052_2975','5053_1633','5056_2239','5058_4602','5060_5061','5061_1231','5062_691','5064_7173','5069_3723','5071_5072','5072_4321','5072_7342','5073_3679','5073_5074','5074_6054','5075_5076','5076_3682','5077_3694','5077_5078','5078_3689','5079_3694','5081_5082','5082_3639','5084_6054','5086_3656','509_510','5090_4204','5091_5092','5092_7574','5096_7145','5097_2383','5099_2383','51_52','510_511','511_513','5110_4012','5111_4051','5112_4003','5113_3918','5118_3910','5119_5120','512_513','5120_5121','5121_5122','5122_3461','5123_5124','5124_5125','5125_6001','5127_5128','5128_2060','5128_2996','5129_5130','513_4513','513_7402','5130_1135','5131_5132','5132_1331','5133_2596','5134_2718','5135_4303','5136_4262','5139_4214','5140_1500','5140_620','5142_7567','5143_2837','5144_7027','5144_7702','5145_276','5147_68','5148_1401','5149_1419','515_516','5150_2327','5154_4804','5155_4806','5156_7552','5157_5154','516_4384','5160_5155','5161_5162','5162_2571','5162_4880','5164_4633','5165_3743','5165_7169','5169_2735','5171_2976','5174_6078','5175_3755','5176_3774','518_2255','519_521','5192_342','5192_350','52_261','52_262','52_263','52_265','52_277','52_4725','521_522','522_1741','522_523','523_1740','523_4382','523_525','523_669','525_526','526_527','527_528','528_529','529_530','53_54','530_531','531_709','532_533','532_649','533_534','534_535','535_536','536_537','537_538','538_539','539_540','54_55','540_541','541_542','542_543','543_544','544_545','544_546','545_546','546_547','546_910','546_914','547_548','548_549','549_550','549_677','55_56','550_551','551_552','552_553','553_554','554_555','555_557','557_558','558_559','559_560','56_58','560_562','562_563','563_564','564_565','565_566','566_567','566_707','567_568','568_569','569_570','57_58','570_571','571_572','572_573','573_574','574_575','575_576','576_577','577_579','579_580','58_4499','58_59','580_581','581_582','582_583','583_584','584_585','585_586','586_587','587_588','588_589','588_590','589_590','59_60','590_1729','590_591','591_592','592_593','593_594','594_595','595_596','596_597','597_598','598_599','599_600','6_194','60_1802','60_820','600_601','600_654','6001_4438','6002_5119','6003_5123','6004_3','6004_7','6005_1546','6005_7072','6006_3493','6007_2836','601_602','6010_3656','6011_2133','6013_6247','6014_2158','6014_6242','6015_5029','6016_6017','6017_7192','602_603','6020_6023','6021_7507','6023_6026','6024_6021','6025_6024','6026_6027','6027_6029','6028_6025','603_604','6030_6028','6033_3089','604_605','6041_2826','6042_1022','6048_5064','605_606','6050_3749','6050_7398','6051_6053','6052_3613','6053_7327','6054_4330','6055_1073','6057_2183','6059_334','6059_404','606_607','6068_1584','607_608','6070_2708','6074_748','6074_793','6075_2631','6076_6077','6077_3569','6078_7112','6079_6315','608_609','6080_1585','6080_983','6083_2733','6085_2026','6085_2053','6086_3161','6086_7419','6087_3620','6088_1898','6089_3981','609_610','6094_2102','6095_7062','6097_4093','610_611','6107_6108','6107_6109','6108_6109','6109_6110','611_612','6110_7020','6110_7029','6111_6112','6112_6113','6113_1898','6113_4323','6115_1205','6115_4593','6117_3655','6117_5079','6119_3392','612_613','6122_35','6124_4004','6125_7092','613_614','6130_1262','6132_2551','6132_4861','6135_3612','6137_4256','6137_4956','614_4415','614_615','614_675','6140_6144','6141_6142','6142_2181','6143_6011','6144_6145','6145_6146','6146_6147','6147_6149','6149_6150','615_616','6150_6143','6152_6153','6153_6243','6154_6155','6155_1987','6155_6141','6156_6165','6157_6158','6158_3964','6158_7314','6159_6160','616_617','6160_6161','6161_3991','6162_7022','6163_1095','6164_6159','6165_6157','617_618','6170_5023','6170_7183','6171_4974','6172_4493','618_619','6182_322','6185_6186','6186_6216','6187_6188','6188_6215','619_675','620_1172','620_621','620_7568','620_7569','621_289','621_291','621_293','621_4495','6215_6184','6216_3400','6219_6229','6228_4788','6228_7111','6229_6348','6229_6364','6229_7131','6230_6228','6234_332','6235_327','6238_6182','6239_104','624_625','6241_105','6242_6152','6243_6154','6244_2134','6245_2156','6245_4671','6245_6013','6247_2679','625_626','6251_5140','6251_7706','6252_301','6252_302','6253_7191','626_627','627_628','6273_136','6273_4891','6276_6362','6277_6278','6278_6279','6279_6280','628_629','6282_6335','6283_6282','6285_6286','6286_6287','6287_6288','6288_6342','629_630','6290_7120','6298_6336','63_64','630_4466','6310_6311','6311_4495','6311_7399','6312_7197','6315_6316','6316_7236','6318_6319','6319_7246','6320_4594','6324_6325','6325_6341','6326_4886','633_634','6334_7660','6335_6326','6336_7069','6339_6324','634_1005','6341_6277','6342_6298','6343_4759','6348_1182','635_636','6359_7189','636_637','6361_6367','6362_7276','6363_6361','6364_1558','6365_6276','6366_6230','6367_4985','637_638','638_639','639_640','64_65','640_641','641_642','642_643','643_644','644_645','645_1796','649_650','65_66','650_651','651_4472','653_722','654_655','655_656','656_657','657_658','658_659','659_604','66_4498','664_614','664_665','665_666','666_667','667_668','668_614','669_670','670_671','671_672','672_4382','675_1184','675_1500','675_303','675_4380','675_4415','675_620','675_621','677_678','678_679','679_680','68_30','680_681','681_682','682_683','683_684','684_685','685_686','686_688','688_689','689_690','69_7686','69_828','690_5062','691_692','692_693','692_694','692_695','693_585','694_695','695_696','696_697','697_698','698_699','699_700','7_11','7_194','700_701','7009_1604','701_702','7010_7703','7011_2171','7011_7160','7012_2201','7012_2644','7013_6162','7014_4315','7015_4617','7016_4619','7017_7018','7018_7030','702_703','7020_7029','7021_6111','7021_7163','7022_1554','7023_7014','7025_4464','7027_4538','7028_5147','7029_7038','703_704','7030_6111','7030_7021','7031_4897','7038_7011','7039_7204','704_705','7043_1424','7044_993','7047_7161','7048_7699','7049_2236','705_706','7050_7185','7053_2088','7056_4731','7057_4847','7057_5142','706_567','706_707','7062_6075','7063_1137','7064_7066','7065_7063','7066_2924','7067_2956','7067_2984','7068_2966','7069_7443','707_708','7070_7445','7073_7097','7074_3919','7075_3925','7078_1449','7079_7080','708_731','7080_7081','7081_7565','7082_7083','7083_7084','7084_7108','7085_7086','7086_7087','7087_7088','7088_7089','7089_7091','709_532','7091_3910','7092_110','7097_7098','7098_7099','7099_7100','71_72','710_711','7100_7101','7101_7505','7102_7103','7103_7104','7104_7105','7105_7106','7106_6005','7107_7102','7108_3935','711_712','7110_3938','7111_6365','7112_972','7113_127','7114_7116','7115_6310','7116_3694','7117_7118','7118_6289','7119_7127','712_713','7120_7119','7121_7183','7122_7121','7123_4347','7124_7343','7125_7344','7126_7124','7127_7143','7128_7117','713_714','7130_1237','7132_1558','7132_6230','7133_7134','7134_3867','7135_7136','7136_3874','7137_7139','7138_3370','7139_4623','714_715','7140_7141','7141_7138','7142_7015','7143_3368','7143_7168','7144_3881','7145_3884','7145_4843','7149_7277','715_716','7153_6056','7154_7153','7156_7020','7158_7048','7159_7017','7159_7388','716_717','7160_7700','7161_7162','7163_1893','7166_1847','7167_1808','7168_4616','7169_3743','717_718','7170_3740','7172_3775','7173_3811','7175_3806','7176_2836','7177_2740','7178_2765','718_719','7180_7123','7180_7331','7183_4880','7184_2225','7184_7050','7185_7186','7186_7187','7187_3947','7187_3954','7187_4001','7188_3082','719_720','7191_6238','7192_4683','7193_7049','7194_7196','7195_3653','7196_6310','7197_7195','72_5144','720_680','720_721','7201_2138','7204_3728','7204_3736','7207_7288','7208_7249','721_680','721_693','7214_4347','7216_2501','7217_7227','7218_7031','7219_7218','722_723','7222_6170','7224_1581','7227_6172','7229_3418','7229_4617','723_724','7230_4617','7231_7128','7233_4573','7234_4002','7236_7245','7237_7241','7238_7224','7239_4401','724_725','7240_1891','7241_7247','7242_7238','7243_7126','7245_6317','7246_6320','7247_7243','7248_7207','7248_7254','7249_7253','725_726','7250_7248','7250_7252','7251_4051','7252_7248','7253_7251','7254_7257','7255_7249','7256_7255','7257_7258','7258_7260','7259_7256','726_727','7260_7263','7261_7259','7262_7261','7263_7264','7264_7268','7265_7262','7266_7265','7268_7270','727_728','7270_6122','7270_7266','7271_5136','7272_4956','7276_7149','7277_6363','7278_7289','728_729','7280_7281','7281_7284','7282_7278','7283_7282','7284_7287','7286_7283','7287_7208','7288_7286','7289_7280','729_730','7290_5016','7291_3485','7292_3818','7293_1335','7294_4132','7294_4156','7296_4265','7296_4266','73_74','730_5160','731_710','7313_6164','7314_3964','7317_3224','7318_4978','7318_4980','7319_7342','7320_1486','7325_3317','7325_4984','7326_7418','7327_3629','7328_3651','7329_3659','733_6015','7330_2051','7331_2559','7333_783','7334_1838','7340_783','7342_7125','7343_4978','7343_4980','7343_7318','7344_2510','7344_7334','7347_3669','7348_3669','7348_3671','7349_1631','7350_1558','7350_6230','7352_4246','7353_4571','7353_7584','7354_1975','7358_1231','7358_1235','7358_5174','7361_7362','7362_3129','7362_3134','7363_4588','7364_7365','7365_4210','7366_7367','7367_7368','7368_3143','7370_7339','7374_1845','7375_3466','7379_4903','738_739','7380_1676','7381_4742','7382_7381','7386_4607','7388_7017','7389_4324','7389_7025','739_740','7391_493','7392_1443','7392_312','7395_6124','7396_5145','7397_5140','7397_6252','7398_6252','7398_7397','7399_7398','7399_7611','74_4371','740_741','7400_3669','7401_3665','7402_4380','7402_4717','741_742','7412_7413','7413_2383','7414_1396','7415_7416','7416_7325','7417_3471','7418_7417','7419_7420','742_743','7420_3345','7421_7422','7422_6085','7423_1566','7424_7423','7425_7424','7426_7427','7427_7428','7428_138','743_744','7430_177','7431_175','7432_3454','7433_4677','7435_2722','7435_4413','7438_1410','7439_7454','744_745','7440_2940','7441_2933','7442_7446','7443_7444','7444_2941','7445_6339','7446_7070','7449_2941','745_600','7453_1478','7453_1480','7453_2001','7454_1380','7455_7438','7458_4479','7458_7425','746_844','7460_4930','7461_4230','7461_4281','7462_7461','7469_1724','747_748','747_842','747_843','7471_1533','7472_3230','7473_2679','7475_6107','748_7390','748_749','749_494','749_750','749_751','75_4501','750_751','7505_5071','7505_7678','7507_2663','751_752','7510_2207','7510_4799','7513_807','7514_2279','7516_6079','752_7339','752_753','753_7339','753_7370','753_754','754_755','755_756','7550_2570','7551_4800','7552_924','7555_4962','7556_3539','7559_7073','7559_7107','756_757','7560_7679','7563_3368','7563_7231','7564_340','7564_4521','7565_7082','7567_7326','757_758','7571_207','7574_4177','7577_1353','7578_7582','7579_1285','758_759','758_760','7581_1283','7582_297','7582_299','7582_319','7582_340','7582_407','7584_3344','7585_7587','7586_7587','7587_303','7587_328','7587_345','7587_407','7587_496','7587_7588','7588_281','7588_325','7588_328','7588_329','7588_4717','759_760','7591_496','7592_3815','760_2007','760_761','760_762','7601_2508','7602_22','7602_85','7603_45','761_762','7611_7398','7612_2811','7616_5081','762_763','7622_297','7622_334','7623_2508','7623_3676','7623_3715','7623_3719','7623_5030','7623_6312','7623_7401','763_2007','763_4952','763_4953','763_767','7639_7640','7640_3134','7640_5047','7641_7642','7642_7643','7643_3214','7644_7645','7645_7646','7646_3238','7654_4843','7660_3114','7661_1521','7667_3091','767_768','7671_513','7672_826','7673_7674','7674_7675','7675_7676','7676_7677','7677_7559','7678_7560','7679_7680','768_2085','768_769','768_770','768_772','7680_7681','7681_7738','7683_1582','769_770','7690_3781','7691_3721','7692_6010','7698_7113','7699_7159','77_78','770_771','770_773','7700_7047','7701_7010','7702_7009','7703_5147','7704_3983','7705_6251','7706_7707','7707_512','7708_7709','7709_7710','771_772','7710_7711','7711_7705','772_417','772_773','772_775','773_774','7738_7683','774_775','774_906','775_776','775_906','776_777','777_779','777_906','7771_3966','779_780','78_79','780_781','781_782','782_783','783_784','783_785','784_785','785_786','786_791','786_792','786_793','79_80','790_7585','791_7585','792_320','792_7586','793_274','793_320','793_495','793_7586','795_796','796_56','796_57','796_797','797_798','798_799','799_800','8_12','80_82','800_801','801_802','802_1654','802_803','803_804','804_7513','807_808','808_809','809_810','81_817','810_811','811_812','812_813','813_814','814_81','817_818','818_819','819_261','819_263','819_264','819_265','819_4725','82_817','82_83','820_821','821_63','822_823','823_824','824_825','825_826','826_827','827_71','828_829','829_830','83_84','830_831','831_75','832_833','833_77','834_835','835_836','836_837','837_1506','838_839','839_840','84_51','840_841','841_49','842_1014','842_845','842_846','843_1013','843_845','844_846','845_846','846_7340','846_847','846_848','847_848','848_2795','848_758','848_759','848_849','849_850','85_203','85_86','850_851','851_852','852_853','853_854','854_855','855_2815','855_856','856_857','857_858','858_859','859_860','86_87','860_861','861_862','862_3014','87_88','875_876','876_3011','876_878','877_876','878_879','879_880','88_89','880_881','881_882','882_883','883_884','884_885','885_886','886_887','887_2900','887_903','89_27','893_894','894_1067','895_1025','896_897','897_893','90_91','903_904','904_906','905_4331','906_907','906_909','907_908','908_909','909_786','909_790','909_792','91_129','91_92','910_911','911_912','912_913','913_914','914_938','915_916','916_917','917_918','918_919','919_7551','92_93','924_925','925_927','927_938','928_588','928_929','929_930','93_94','930_931','931_587','938_940','94_102','94_126','940_941','941_942','942_943','943_3599','944_945','945_947','946_1155','947_948','948_949','949_928','950_915','956_957','957_958','958_959','959_960','960_961','961_962','962_635','969_1176','972_1625','980_981','981_1518','981_1576','983_984','984_985','985_1529','987_4937','992_7044','993_5060']

In [ ]:
df = df.loc[df['segment'].isin(gtfs_segments)]

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After removing legacy segments, the dataframe contains", rows, "rows and", cols, "columns.")
print()

## 9.4 Peak Time Feature

Create a feature that has value 1 during peak traffic times, and value 0 otherwise (based on https://www.irishexaminer.com/breakingnews/ireland/rush-hour-now-lasts-for-six-hours-each-day-842746.html):

In [ ]:
df.loc[(df.hour.isin([7,8,9,16,17,18])) & (df.weekday == 1), 'peak'] = 1
df.loc[~((df.hour.isin([7,8,9,16,17,18])) & (df.weekday == 1)), 'peak'] = 0

## 9.5 Mean Feature

We will use mean/target encoding for the **time_diff** feature as it has such high cardinality.

In [ ]:
means = df.groupby('segment')['time_diff'].mean().round()

In [ ]:
df['segment_mean'] = df['segment'].map(means)

## 9.6 Standard Deviation Feature

We will create a feature that has the standard deviation of time_diff for each segment:

In [ ]:
stds = df.groupby('segment')['time_diff'].std().round()

In [ ]:
df['segment_std'] = df['segment'].map(stds)

Check that all rows have a value for standard deviation:

In [ ]:
df.loc[np.isnan(df['segment_std'])]

Drop rows where segment_std is NaN:

In [ ]:
df = df.loc[~np.isnan(df['segment_std'])]

## 9.7 School Holiday Feature

In [ ]:
# Make list of school holidays for 2018 (based on https://www.education.ie/en/Circulars-and-Forms/Active-Circulars/cl0009_2017.pdf)
school_holidays = ['2018-01-01 00:00:00', '2018-01-02 00:00:00', '2018-01-03 00:00:00', '2018-01-04 00:00:00', '2018-01-05 00:00:00', \
           '2018-02-12 00:00:00', '2018-02-13 00:00:00', '2018-02-14 00:00:00', '2018-02-15 00:00:00', '2018-02-16 00:00:00', \
           '2018-03-26 00:00:00', '2018-03-27 00:00:00', '2018-03-28 00:00:00', '2018-03-29 00:00:00', '2018-03-30 00:00:00', \
           '2018-04-02 00:00:00', '2018-04-03 00:00:00', '2018-04-04 00:00:00', '2018-04-05 00:00:00', '2018-04-06 00:00:00', \
           '2018-10-29 00:00:00', '2018-10-30 00:00:00', '2018-10-31 00:00:00', '2018-11-01 00:00:00', '2018-11-02 00:00:00', \
           '2018-12-24 00:00:00', '2018-12-27 00:00:00', '2018-12-28 00:00:00', '2018-12-31 00:00:00']

In [ ]:
# add the feature
df.loc[df.dayofservice.isin(school_holidays), 'school_hol'] = 1
df.loc[~df.dayofservice.isin(school_holidays), 'school_hol'] = 0

## 9.8 Update Data Types for the New Features

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date', 'weekday', 'segment', 'peak', 'school_hol']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

# 10. Data Integrity Checks

## 10.1 Deal with Outliers

Make a list of all segments:

In [ ]:
# make a list of all segments
segments = ['10_12','100_1111','1005_1788','1013_1015','1014_1015','1015_1016','1015_1074','1015_2812','1016_1017','1017_1018','1018_1019','1019_1020','102_6239','1020_1076','1021_6042','1022_1023','1023_1024','1024_895','1025_1026','1026_1027','1027_1028','1028_2818','1035_1314','1036_4709','1037_1038','1038_2998','104_6241','1042_1043','1043_1044','1044_1045','1045_4710','1046_1047','1047_1048','1048_1049','1049_4333','105_6253','1051_1052','1052_1053','1053_1054','1054_1036','1055_2868','1062_1063','1063_1064','1064_1065','1065_1066','1066_1067','1067_1068','1068_1069','1069_1070','107_6125','1070_1071','1071_4528','1072_1352','1072_7577','1073_3641','1074_6074','1076_1021','1076_1077','1077_1078','1077_2479','1078_1079','1079_1080','1080_1081','1081_1082','1081_2914','1082_1083','1083_1085','1085_1086','1085_1121','1086_1087','1087_1088','1088_1089','1089_1090','1090_1091','1090_2413','1091_1092','1092_1093','1093_1094','1094_1095','1095_1096','1096_1097','1096_1101','1097_1099','1099_6132','11_14','110_112','1100_100','1101_1102','1102_1103','1103_1104','1105_1107','1107_1108','1108_1109','1109_1110','1110_1111','1111_1112','1111_2323','1111_2404','1112_1113','1113_1114','1114_1115','1115_2437','1117_1118','1118_1119','1119_1120','112_113','1120_1164','1121_1122','1122_1123','1123_1124','1124_1125','1125_1127','1127_1130','113_114','1130_1140','1130_2550','1135_7064','1137_5131','114_115','1140_1141','1140_2515','1141_1142','1142_1143','1143_1144','1144_1145','1145_4869','115_37','1150_1151','1151_1152','1152_1153','1153_1154','1154_946','1155_1157','1157_1158','1158_1159','1159_1160','116_117','1160_1161','1161_1162','1162_1163','1163_1164','1163_1337','1164_1165','1165_1166','1166_1167','1167_1168','1168_1169','1169_1170','117_118','1170_1069','1171_1172','1172_1173','1172_286','1172_4509','1172_7671','1173_1174','1174_1175','1174_4725','1175_15','1176_1177','1177_1178','1178_1179','1179_5060','1179_7112','118_119','1181_7350','1182_1550','1184_497','1185_1186','1186_1187','1187_1188','1188_216','119_44','1190_1191','1191_1192','1192_1193','1193_1194','1194_1195','1195_664','1196_1197','1197_1198','1198_1199','1198_1222','1198_4782','1199_1200','12_14','1200_4385','1201_1202','1201_4695','1202_1203','1203_6115','1205_1206','1206_1207','1207_1208','1208_1209','1209_1210','1210_1211','1210_3575','1211_3575','1212_1213','1213_1214','1214_1215','1215_1216','1216_1217','1217_4563','1218_1258','1218_1270','1219_1220','1220_1221','1221_664','1222_1223','1223_1224','1224_1225','1225_1226','1226_1227','1227_1228','1227_1253','1228_1244','1229_1231','1231_1232','1231_1235','1232_1233','1233_1234','1234_7358','1235_1251','1237_1238','1238_1239','1239_247','1240_1241','1241_1242','1242_1277','1243_4696','1244_1246','1244_254','1245_4693','1246_1229','1247_1248','1248_1273','1249_1247','1250_4698','1251_253','1251_7130','1253_969','1254_1255','1255_1256','1256_4694','1258_1259','1259_1260','126_6182','1260_1262','1260_6130','1262_4692','1265_6115','1266_1267','1267_1268','1268_1269','1269_1265','127_112','1270_1272','1272_1273','1273_1274','1274_1275','1275_1276','1276_1277','1277_1219','1278_1282','1278_1934','1279_1282','1279_1283','1279_1934','1280_6348','1280_6366','1281_1329','1282_4456','1283_4456','1283_7579','1284_1285','1285_1016','1285_1286','1285_1287','1286_1288','1287_1288','1288_1289','1289_1290','1289_1360','129_130','1290_1291','1291_1292','1291_2389','1292_1293','1293_1294','1294_1295','1295_1296','1296_1297','1297_1298','1298_1121','1298_1299','1299_1300','130_131','1300_1301','1301_1302','1302_1303','1303_1304','1303_5129','1304_1305','1305_1306','1305_1320','1306_1307','1307_1308','1308_1309','1309_1310','131_164','1310_1311','1311_1312','1312_1313','1313_1055','1314_1316','1315_1317','1316_1315','1317_1318','1318_1319','1319_1281','1320_1321','1321_1322','1322_1323','1323_1324','1324_7068','1325_1326','1326_1327','1327_1328','1328_1329','1329_1330','133_134','1330_1331','1330_2919','1331_1332','1332_1333','1333_1334','1334_1335','1334_2985','1334_7293','1335_1336','1336_1337','1337_1338','1338_1339','1339_1340','134_4890','1340_1341','1341_1342','1342_1343','1343_1344','1344_1345','1345_1347','1347_1348','1347_2634','1348_1349','1349_1350','1350_1352','1352_1353','1353_1354','1353_1355','1354_7578','1355_1357','1357_1358','1357_1359','1358_298','1358_319','1358_326','1358_340','1358_342','1358_4494','1358_5192','1358_7564','1358_7665','1359_274','1359_279','1359_319','1359_320','1359_4724','136_137','1360_1362','1362_1363','1363_1365','1365_1365','1365_1366','1366_1367','1367_1369','1367_7455','1369_1370','137_193','1370_1371','1371_1372','1371_1454','1372_1373','1373_1374','1374_1375','1374_1946','1375_1376','1376_1377','1377_1378','1378_1379','1378_4514','1379_1380','138_133','138_167','1380_1381','1381_1382','1382_1383','1383_1384','1384_1385','1385_1348','1386_1387','1387_1388','1388_1389','1389_3355','1396_2317','1397_1398','1398_1399','1399_5148','14_15','14_53','1401_1402','1402_1403','1403_1404','1404_1405','1405_4856','1406_1407','1407_1409','1409_2095','141_142','1410_1411','1411_1412','1412_1413','1413_1414','1414_1415','1415_1417','1417_1418','1418_5149','1419_6094','142_143','1423_1424','1424_1425','1424_2186','1424_2190','1425_1426','1426_1427','1426_1463','1427_1428','1428_1429','1429_1430','143_115','1430_1432','1432_1433','1433_1434','1434_1435','1435_7439','1436_3952','1438_4855','1439_1440','1440_1386','1441_1442','1442_7414','1443_1444','1443_1616','1443_1647','1444_1445','1445_1447','1445_1647','1445_4319','1445_4407','1445_7078','1447_2637','1449_1450','1450_1451','1451_1452','1451_2191','1452_2640','1453_1456','1454_1455','1455_1453','1456_1457','1457_1458','1458_1459','1459_1460','146_147','1460_1462','1462_5149','1463_4399','1464_1469','1465_1466','1466_1467','1467_1468','1468_1377','1469_1465','147_184','1471_1472','1472_4408','1473_1474','1474_1476','1476_7453','1478_1479','1478_315','1479_315','1479_7622','148_149','1480_1358','1480_2912','1481_7320','1482_1494','1484_1481','1485_4518','1486_1487','1487_1488','1488_1489','1489_1490','149_150','1491_1492','1492_1493','1493_1482','1494_1495','1495_1496','1496_1515','1499_1485','15_17','15_834','150_27','1500_302','1505_4509','1506_1507','1507_1508','1508_1509','1509_1510','151_153','1510_1511','1511_1512','1511_1592','1512_1538','1514_980','1515_1499','1517_1678','1518_1519','1519_1520','1520_7661','1521_4546','1523_1524','1524_1525','1525_1526','1526_6080','1528_1710','1529_4542','153_154','1531_1532','1532_1533','1533_1534','1534_1535','1535_1536','1536_1537','1537_186','1538_4543','1538_4548','1539_1540','154_155','1540_7426','1546_7072','1546_7560','1547_1548','1548_1280','1549_1523','155_156','1550_1551','1551_1552','1552_1553','1552_332','1553_7023','1554_1555','1555_1556','1556_1557','1557_1181','1558_1549','1558_2957','1558_4542','1559_1560','1559_987','156_157','156_7430','1560_1280','1566_1567','1567_173','1567_4372','157_158','1573_1574','1574_1575','1575_4583','1576_1577','1577_1578','1578_4364','1579_1580','1579_7673','158_159','1580_7237','1581_1582','1582_4793','1584_1585','1584_983','1585_1586','1586_1587','1586_1600','1587_1591','1587_4539','159_1539','1591_4542','1592_1593','1593_1594','1594_1595','1595_1596','1596_4363','1597_1599','1598_1597','1599_1573','16_73','1600_7701','1604_1573','1606_1607','1607_1608','1608_1609','1609_1610','1610_7471','1614_1615','1615_1478','1616_1617','1617_1618','1618_1619','1619_195','1620_220','1622_1623','1623_1624','1624_1625','1625_1626','1626_1627','1627_1628','1628_1629','1629_1630','1629_6234','1630_3670','1630_7347','1630_7348','1631_1632','1632_5053','1633_1634','1634_1635','1635_1636','1636_1637','1637_1638','1638_1639','1638_992','1639_1640','164_165','1640_231','1641_1642','1642_213','1643_7571','1647_1648','1647_1649','1648_1649','1649_1911','165_166','1654_1655','1655_1656','1656_1657','1657_4458','1658_1659','1659_1660','166_7458','1660_1661','1661_1662','1661_4523','1662_1664','1662_1666','1664_1665','1665_1666','1666_1667','1666_1807','1666_1808','1667_1668','1668_1669','1669_1670','167_168','1670_1671','1671_1672','1672_1673','1673_1674','1674_1675','1675_1849','1675_7380','1676_1517','1678_1679','1679_4520','168_169','1680_1681','1681_1682','1682_1683','1683_1684','1684_1685','1685_1686','1686_1687','1687_1688','1688_1689','1689_1690','1689_4899','169_141','1690_1691','1691_1692','1692_1693','1693_1694','1694_1695','1695_1696','1696_1697','1696_1700','1697_1698','1698_1699','1699_1700','17_18','1700_1701','1700_1906','1701_1702','1702_16','1702_1703','1703_1704','1703_1905','1704_1705','1705_1706','1706_1707','1707_1708','1708_1709','1709_1528','1710_1711','1711_1712','1712_1713','1713_1714','1713_1715','1714_1715','1715_1478','1715_7453','1719_1828','1724_1725','1725_1726','1726_1727','1727_1728','1728_1729','1729_1730','1729_1731','173_174','1730_1731','1731_1732','1731_1738','1732_1733','1733_1734','1734_1735','1735_1736','1736_1737','1737_1738','1738_1739','1738_614','1739_614','174_175','1740_1741','1741_1742','1741_1747','1742_1743','1743_1744','1744_1745','1745_1746','1746_1747','1747_1748','1747_1749','1748_1749','1749_1751','1749_544','175_176','1750_1769','1751_1752','1752_1753','1753_1754','1754_1759','1755_1756','1756_1757','1757_1794','1758_1750','1759_1758','176_181','1760_1761','1761_1762','1762_1763','1763_1764','1764_1765','1765_1767','1769_1770','177_178','1770_1771','1771_1760','1772_1773','1773_1774','1774_1775','1775_1776','1776_1777','1777_1778','1778_1755','178_1539','1782_1783','1783_1784','1784_1785','1785_1786','1786_624','1788_1789','1789_7199','179_7431','1791_1792','1792_1793','1793_962','1794_7469','1795_1806','1796_1797','1797_1798','1798_1799','1799_1800','18_19','1800_1801','1801_1277','1802_1795','1803_77','1804_1803','1805_1662','1805_1806','1806_1660','1807_7167','1808_1813','1808_1816','1808_2961','1808_4744','1808_7389','1808_7475','181_182','1813_1814','1814_1816','1814_1852','1816_1817','1816_6171','1817_1818','1818_1819','1819_1822','182_183','1820_2468','1822_4994','1826_1827','1827_7382','1828_1829','1829_4997','183_184','1833_1834','1834_1820','1834_1835','1835_1836','1836_1837','1837_2338','1838_2338','184_185','1840_1841','1841_1842','1841_4314','1842_7374','1844_6172','1845_1696','1845_7166','1847_1696','1849_1851','185_186','1850_1867','1851_1813','1852_1853','1853_1854','1854_1844','1858_1859','1859_1860','186_187','186_838','1860_4492','1861_1862','1862_1863','1863_1864','1864_1865','1865_1866','1866_1850','1867_4489','1869_1870','187_188','1870_1871','1871_1872','1872_1873','1873_1874','1874_1875','1875_1876','1876_1877','1877_1878','1878_1879','1879_1899','188_189','1882_7379','1883_1884','1884_1902','1887_1888','1888_1889','1889_1890','189_190','1890_7240','1891_7156','1893_1894','1894_1912','1895_1896','1896_1897','1897_6088','1898_1858','1899_1887','1899_6107','19_21','190_1614','190_191','190_192','1902_1903','1903_1904','1904_1840','1905_1804','1905_1906','1906_1907','1906_1909','1907_1908','1908_1909','1909_1713','191_819','1911_1805','1911_1913','1911_802','1912_1895','1913_1914','1914_1805','192_795','192_819','193_138','1934_1935','1934_2310','1934_2385','1935_1445','1935_1937','1937_1938','1938_1939','1939_1940','194_195','1940_1941','1941_1942','1941_4516','1942_1943','1943_1944','1944_1945','1945_1946','1946_1947','1947_1948','1947_2642','1948_1949','1949_1950','195_196','1950_1951','1951_1952','1952_1953','1953_1954','1954_1955','1955_1956','1956_1957','1956_2109','1957_1958','1958_1959','1959_1960','196_197','1960_4379','1963_4549','1964_4674','1965_1966','1966_1975','1968_1969','1969_1970','197_198','1970_1971','1971_2114','1971_2160','1972_1973','1973_1974','1974_1975','1975_1977','1975_1982','1977_1978','1978_1980','198_199','1980_1981','1981_1982','1981_4406','1982_2642','1982_4406','1982_7435','1984_1985','1985_1986','1986_1987','1987_1988','1987_1989','1988_1989','1988_2642','1989_1375','1989_1472','1989_1990','1989_1995','199_1506','199_200','1990_1992','1992_1993','1993_1994','1994_1995','1995_1996','1995_1998','1996_1997','1997_1998','1998_1999','1998_2001','1998_2002','1999_2001','2_192','2_4726','2_4962','200_201','2001_2002','2002_1358','2002_1359','2003_1358','2007_2008','2007_4571','2008_2009','2008_2071','2009_2010','201_148','201_202','2010_435','2011_2012','2012_3322','2013_2014','2014_2016','2014_4636','2015_2017','2016_2015','2016_2017','2017_2018','2017_3259','2017_7362','2018_2019','2019_2020','202_151','2020_2021','2021_2022','2022_4565','2023_2024','2024_2025','2025_6086','2026_7226','203_204','2031_2032','2032_2033','2033_2034','2034_2035','2035_2036','2036_2039','2036_2040','2039_2040','204_205','2040_2041','2040_3047','2041_2042','2041_3071','2042_2043','2043_2044','2044_2045','2045_2046','2046_4568','205_1620','205_206','205_215','2051_6086','2053_2054','2054_2055','2055_4565','2055_4732','2056_2057','2057_2058','2058_2059','2059_2060','206_207','2060_2061','2060_2062','2061_2062','2062_2063','2062_2064','2063_2064','2064_2065','2064_4727','2065_4727','2066_2067','2067_4728','2068_2069','2069_2070','207_208','2070_2084','2071_2072','2072_2073','2073_2074','2074_2075','2075_2076','2076_460','208_209','2080_2081','2081_2082','2082_2083','2083_2084','2084_768','2085_2086','2086_4455','2086_479','2087_7053','2088_2089','2089_763','2089_770','209_210','2094_1406','2095_2096','2096_2097','2097_2099','2099_2101','21_7602','21_85','210_1643','210_211','2101_2102','2102_2103','2102_2726','2103_2332','2103_7154','2105_2728','2105_6140','2109_3427','211_212','2114_2115','2115_6011','2118_2119','2119_2120','212_213','2120_2121','2121_2123','2123_2685','2124_2125','2125_2126','2126_2127','2127_2128','2128_2129','2129_4797','213_214','2133_6244','2134_2135','2134_3378','2135_2164','2136_2137','2137_2138','2137_7201','2138_2139','2139_2140','214_4432','2140_2141','2141_2142','2142_2143','2143_2145','2145_4670','2146_2147','2147_2148','2148_2149','2149_2150','215_216','2150_2151','2151_2152','2152_2153','2153_2154','2154_6245','2156_6014','2158_2159','2159_4409','216_217','2160_2161','2160_4665','2161_4665','2162_2163','2163_2168','2164_2165','2165_2166','2166_2167','2166_4545','2166_4554','2166_7229','2167_3383','2168_4536','2169_2170','217_218','217_242','2170_4672','2171_7160','2172_2159','2172_4409','2173_2174','2174_3425','2176_1982','2176_2177','2177_2178','2178_7354','218_219','2181_2182','2182_2778','2183_2184','2184_2185','2185_1423','2185_7043','2186_2187','2187_2188','2188_2189','2189_1436','219_220','2190_4434','2191_2192','2192_2193','2193_2194','2194_2195','2195_2196','2196_2197','2197_2198','2198_2200','22_23','220_1622','220_221','2200_2201','2201_4401','2201_7239','2202_2203','2203_2204','2204_4888','2205_2687','2206_4798','2207_2208','2208_2210','221_222','2210_2211','2211_2241','2213_2214','2213_4688','2214_2215','2215_2216','2215_7184','2216_4599','2217_2218','2218_2219','2219_2220','2219_3365','222_223','2220_2221','2221_2223','2223_2224','2224_3886','2225_4620','2227_7049','2229_2230','223_224','2230_2231','2231_2233','2232_2234','2233_2232','2234_2235','2235_2236','2235_2239','2236_2238','2236_2239','2238_5056','2239_2241','224_225','2241_2242','2242_2243','2242_7435','2243_2244','2244_2245','2245_2246','2246_2247','2247_2248','2248_2249','2249_2250','225_226','2250_2251','2251_1472','2255_2256','2256_2717','2257_2258','2258_2259','2259_2260','226_228','2260_2261','2261_4469','2262_2264','2264_2265','2265_2269','2269_2270','227_230','2270_2274','2274_7601','2278_2279','2279_2720','2279_6251','228_229','2280_2281','2281_2282','2282_4468','2283_616','2289_1822','229_227','23_24','230_231','231_1641','231_232','2310_2311','2311_2312','2311_5025','2312_2313','2313_2314','2314_2315','2315_2094','2317_2318','2318_2319','2319_2320','232_233','2320_2321','2321_2322','2322_6163','2323_2324','2324_2325','2325_2326','2326_5150','2327_2328','2328_2329','2329_2330','233_234','2330_2331','2331_1397','2331_1424','2332_2333','2333_2334','2334_4662','2335_2336','2336_2337','2337_2339','2338_1840','2338_1845','2339_2340','2339_2420','234_235','2340_2601','2346_4435','2349_2351','235_236','2351_2558','2351_2621','2351_4445','2352_2573','2353_2357','2353_4643','2357_2358','2357_2359','2357_2566','2358_2359','2359_2360','2359_7122','236_1190','236_237','2360_4340','2360_4348','2369_2370','237_213','2370_2371','2371_2372','2372_2373','2373_1424','2373_2377','2374_2146','2377_2378','2377_2431','2378_2183','2379_2380','2380_2382','2382_5099','2383_1479','2383_2002','2383_2003','2385_2386','2386_2387','2387_2388','2388_1290','2389_2390','2390_2391','2391_2392','2392_2393','2393_2394','2394_1090','2394_2395','2395_2396','2396_2397','2397_2398','2398_2399','2399_2400','24_25','2400_2401','2401_2402','2402_2403','2403_1096','2404_2405','2405_2406','2406_2408','2408_2409','2409_2410','2410_2411','2411_2412','2412_2438','2413_2414','2414_2415','2415_2416','2416_2417','2417_2418','2418_2419','2419_2452','242_243','2420_2421','2421_2423','2423_2424','2424_2447','2425_2426','2426_2427','2427_2428','2428_2429','2429_2430','243_244','243_254','2430_2371','2431_2432','2432_2433','2433_2327','2433_2434','2434_2435','2435_2436','2436_2437','2437_1117','2437_2438','2438_2439','2439_2440','244_245','2440_2441','2441_2442','2442_2443','2443_2444','2444_1344','2445_2457','2446_2459','2447_5133','2448_5026','245_246','2450_2490','2452_2453','2453_2454','2454_2455','2455_2456','2456_2445','2457_4392','2458_2446','2459_2460','246_248','2460_2461','2461_2462','2462_2463','2463_2431','2465_2466','2466_2467','2467_2494','2468_7319','247_1240','2471_2472','2472_2473','2473_2474','2474_2475','2475_2476','2476_2477','2477_2478','2478_1170','2479_2480','248_249','2480_2481','2481_2482','2482_2483','2483_2484','2484_2485','2485_2486','2486_2487','2487_2488','2488_2489','2489_2448','249_250','2490_2491','2491_2492','2492_2493','2493_2465','2494_2495','2495_2496','2496_2497','2497_2471','2498_2499','2499_2500','2499_7216','25_27','250_251','2500_2501','2501_7623','2508_2278','251_252','2510_7242','2515_2516','2516_2517','2517_2518','2518_4749','252_255','2522_2524','2523_2532','2524_2523','253_5033','2532_4877','2535_2536','2536_2616','2538_2606','2539_2540','254_1246','254_244','2540_2542','2540_4882','2542_3004','2547_2548','2548_2600','2548_946','255_234','2550_2516','2550_2551','2551_2553','2553_2554','2554_2425','2554_2555','2555_2556','2556_2557','2557_4436','2558_2559','2558_6170','2559_2560','2560_2561','2561_2562','2562_2564','2564_4927','2566_2567','2567_2568','2568_2569','2569_2570','2569_7550','2570_2571','2570_7222','2571_2359','2573_2574','2574_2575','2574_2577','2575_4929','2577_2578','2578_2579','2579_7395','2583_2584','2584_2585','2585_2586','2586_2587','2587_2588','2588_2589','2589_2590','2590_2591','2591_2592','2592_2357','2594_2595','2595_2596','2596_2597','2596_2611','2597_2599','2599_2600','2599_4577','2600_1155','2601_4446','2602_2613','2603_2557','2604_2605','2605_2538','2606_2607','2607_2608','2608_2609','2609_2610','261_270','261_271','2610_2554','2611_2602','2613_2614','2614_2615','2615_2535','2616_2617','2617_2557','262_270','2621_2622','2622_2623','2622_2750','2623_2624','2624_4441','2626_2669','2628_2629','2629_6095','263_271','263_277','263_4508','263_4725','263_6071','2631_2359','2632_2633','2633_2369','2634_2635','2635_2636','2636_2383','2636_5097','2637_1956','2637_2638','2637_2721','2638_2640','2638_2721','264_6059','2640_2641','2641_1948','2641_2643','2642_2643','2642_7435','2643_2644','2644_2645','2644_2709','2645_2646','2646_2647','2647_2648','2648_2649','2649_2650','265_271','265_6059','2650_2652','2652_2653','2653_2654','2654_2655','2655_2656','2656_2657','2656_2668','2657_2658','2658_2659','2659_2660','2660_2661','2661_2662','2662_2909','2663_2664','2664_2665','2665_2666','2666_2667','2667_2696','2668_2672','2669_4641','2670_2910','2672_2673','2673_2206','2674_2675','2675_2124','2676_2118','2678_2681','2679_4691','2680_2676','2680_4677','2681_2680','2682_4796','2685_2686','2686_7510','2687_2688','2688_2689','2689_2696','2696_2697','2697_2700','2697_2713','27_28','270_335','270_336','2700_2701','2701_2702','2702_2703','2703_2704','2704_2706','2706_6070','2708_5134','2709_2710','271_1278','271_340','271_404','271_4521','271_496','2710_2711','2711_2712','2712_2655','2713_2714','2714_2715','2715_2716','2716_2718','2717_2257','2718_2719','2718_7435','2719_1989','2719_7435','272_335','2720_2280','2721_7012','2722_1471','2722_4413','2725_2737','2726_2727','2727_2105','2728_2729','2729_2730','273_405','2730_6083','2733_2734','2734_5169','2735_2736','2736_2725','2737_2738','2738_2739','2739_7177','274_1505','274_2','274_7','2740_2741','2741_2742','2742_2743','2743_2745','2745_2746','2746_2747','2747_2748','2748_7062','2750_2751','2751_2752','2752_2753','2753_2754','2754_2755','2755_2756','2756_2764','2764_7178','2765_2766','2766_2767','2767_2772','2768_2769','2769_2770','277_288','277_292','277_336','2770_2771','2771_2773','2772_2768','2773_2774','2774_2777','2775_2181','2777_2775','2778_2185','278_10','278_2','278_7615','278_8','279_3','279_7','2795_756','28_29','2804_376','2809_2810','2809_408','2809_7612','281_261','281_4','281_6','281_7','2810_2811','2810_7385','2810_7387','2810_749','2810_750','2810_842','2810_843','2811_749','2811_842','2811_843','2812_2813','2813_2814','2814_852','2815_2816','2816_2817','2817_2818','2818_2819','2819_2820','2819_3012','2820_2822','2822_2823','2823_5032','2824_2825','2825_2826','2826_2827','2826_2842','2827_2828','2828_2829','2829_2830','2830_2831','2831_2832','2832_2833','2832_4349','2833_6006','2833_7375','2835_7176','2836_2837','2837_2838','2838_2839','2839_2840','284_4504','2840_2841','2840_2845','2841_2865','2842_2843','2843_2844','2844_2845','2845_2846','2846_2847','2847_4397','2848_2849','2849_2850','2850_2851','2851_2852','2852_4707','2852_4987','2853_2854','2854_2855','2855_2856','2856_2857','2857_2858','2858_2859','2859_2860','286_271','2860_2861','2861_2829','2861_2862','2862_2863','2863_2864','2864_4486','2865_2866','2866_2825','2866_2892','2866_6044','2867_1035','2868_2892','288_1171','289_496','2892_2893','2893_2894','2894_2895','2895_2896','2896_2897','2897_1062','2897_2898','2898_3562','2899_884','29_4680','2900_2901','2901_2902','2902_2903','2903_1074','2906_2907','2907_2908','2909_2906','2909_6020','2910_2911','2911_7507','2912_1358','2914_2915','2914_2989','2915_2916','2916_2917','2917_2918','2918_1300','2919_2920','292_1171','2920_2921','2921_2922','2922_2923','2923_2924','2924_2925','2925_2948','2925_7442','2927_2929','2929_2930','2930_7441','2934_2935','2935_2936','2936_2937','2937_2938','2938_7440','2940_2948','2941_2942','2942_2943','2942_7065','2943_2944','2944_2945','2945_2946','2946_2947','2947_1305','2948_2949','2949_2950','2950_2954','2954_2955','2954_7067','2955_7067','2956_2965','2957_2958','2958_4938','2961_4901','2962_2963','2963_2964','2964_4175','2965_2967','2965_2983','2966_2967','2967_2968','2968_2969','2969_2970','297_407','297_496','2970_2971','2970_4988','2971_5051','2976_2977','2977_2978','2978_2979','2979_2980','298_496','2980_2981','2981_2954','2981_2991','2983_2984','2984_2927','2984_2941','2984_7449','2985_2986','2986_2988','2988_3001','2989_2990','299_497','2990_1042','2991_2992','2992_1325','2993_2994','2994_4180','2995_3154','2996_3258','2997_4153','2998_2999','2999_3000','3_12','3_192','30_69','300_496','300_497','3000_3001','3001_3002','3002_1166','3003_2522','3004_4756','3005_4757','3006_4871','3007_6283','3008_6089','301_336','301_4495','3011_2896','3012_877','3014_3015','3015_3016','3016_3017','3017_3018','3018_3019','3019_3020','302_333','302_340','302_4495','3020_3021','3021_3022','3022_441','3023_3024','3024_3025','3025_3026','3026_3027','3027_3028','3028_3029','3029_3030','3030_3031','3031_875','3032_3033','3033_3034','3034_3036','3034_3114','3036_3037','3037_3038','3038_3039','3039_3040','3040_3041','3041_2035','3041_3042','3042_2035','3045_4150','3047_3202','3048_3049','3049_3050','3050_3051','3051_3052','3052_3053','3053_3054','3054_3055','3055_3056','3056_3057','3057_3059','3057_4830','3059_3060','3060_3061','3061_3062','3062_3063','3063_3064','3064_3065','3065_3066','3066_3067','3067_3202','3068_3069','3069_3070','307_308','3070_2041','3071_3072','3072_3073','3073_3074','3074_3075','3075_3076','3076_3077','3077_3079','3077_3084','3079_3080','308_309','3080_3081','3080_3082','3081_3084','3081_7188','3082_3084','3082_3114','3082_7660','3084_469','3085_3034','3088_3704','3089_3090','309_2383','309_7412','3090_3706','3091_3092','3092_3093','3093_3535','31_32','3112_3924','3114_3115','3114_4353','3115_3116','3116_3117','3116_6085','3117_3118','3118_3119','3119_3120','312_1444','3120_3122','3122_3123','3123_3124','3124_3125','3125_3126','3126_3127','3127_3128','3128_3130','3129_3130','3130_3131','3131_3132','3132_3133','3133_3134','3133_5046','3134_3135','3135_3136','3135_7363','3136_3138','3138_3139','3139_4124','3140_3141','3141_3142','3142_3143','3142_3224','3143_3144','3144_3145','3144_5047','3144_7639','3145_3146','3145_5128','3146_3147','3147_3148','3148_3149','3148_5127','3149_3150','315_313','315_335','315_336','315_404','315_406','315_4495','3150_3151','3151_3153','3152_2995','3153_3152','3154_3155','3155_3156','3156_3158','3158_3159','3159_3160','3160_3161','3161_3162','3162_3163','3163_3164','3164_3079','3164_3084','3164_7188','317_312','318_312','3181_451','3184_449','319_1443','319_274','319_278','319_279','319_281','319_7491','3196_3345','32_33','320_278','320_281','3202_3203','3203_3204','3204_3205','3204_4568','3204_7641','3205_3206','3206_3207','3207_3208','3208_3209','3209_3210','3210_4981','3211_3212','3212_3213','3213_3214','3214_3215','3215_3216','3216_3217','3217_3218','3218_3219','3219_3220','3219_3537','322_323','322_7697','3220_3221','3220_5046','3221_4982','3222_3223','3223_3224','3224_3225','3225_3226','3226_3227','3227_3228','3228_3229','3229_7472','3229_7644','323_332','323_6234','3230_3231','3230_3232','3231_3246','3232_3233','3233_3234','3234_4462','3235_3236','3236_3237','3237_3238','3238_3239','3239_3240','324_1555','324_1634','324_327','324_6235','3240_3048','3240_3068','3243_3244','3244_3245','3245_3212','3246_3247','3247_7056','325_1443','3258_2060','3259_7361','326_1443','327_6016','327_7113','328_1443','3280_3282','3282_3479','3283_3284','3284_3310','329_1443','33_7270','3310_3481','3317_3318','3318_3320','3320_2064','3321_2066','3321_4727','3322_3563','3328_3329','3329_3330','333_747','3330_3331','3331_3351','3332_3333','3333_3334','3334_3352','3335_3336','3336_3337','3337_3338','3338_3339','3339_3340','334_406','334_7581','3340_4324','3344_7421','3345_4727','335_1278','335_404','335_4521','3351_3332','3352_3335','3355_3356','3356_1441','336_1278','336_1279','336_7581','3365_3366','3366_3367','3367_3886','3368_4626','3368_7137','3370_7168','3370_7231','3372_3373','3373_3374','3374_2233','3376_3377','3377_3378','3378_3379','3379_4669','3383_3384','3384_3385','3385_3386','3386_3387','3387_4559','3389_3390','3390_6119','3392_3393','3393_3394','3394_3396','3396_3397','3397_3398','3398_3399','3399_6187','340_350','3400_3401','3401_3402','3402_3403','3403_3404','3404_3405','3405_3406','3406_3410','3407_3408','3408_3409','3409_4558','3410_3407','3411_3412','3412_3413','3413_3414','3414_3415','3415_3416','3416_3417','3417_3418','3418_3419','3419_2374','3419_3420','3419_4762','342_1279','342_4495','342_4521','3420_3421','3421_3422','3422_3423','3422_4671','3423_2156','3423_4409','3425_2176','3426_3434','3427_3428','3428_3429','3429_3432','3432_3426','3432_3434','3434_3435','3434_3453','3435_3453','3436_7432','3437_3439','3437_3440','3438_5022','3439_3440','3440_3441','3440_4557','3441_4560','3443_3444','3444_3445','3444_3447','3445_3446','3446_3447','3447_3448','3447_3454','3448_3449','3449_3436','345_318','345_320','3453_3437','3454_3455','3455_3456','3455_5022','3456_3457','3457_3438','346_317','3461_4532','3461_4537','3465_4673','3466_3467','3467_3469','3469_3470','3470_3471','3471_3472','3472_3473','3473_3474','3473_5015','3474_3475','3475_3476','3476_3477','3477_3478','3478_3280','3479_4086','3481_3482','3482_3483','3483_3484','3484_3485','3485_3486','3486_3487','3487_3488','3487_7415','3488_3489','3489_3492','3491_6007','3492_2835','3493_3494','3494_3495','3495_3496','3496_3497','3497_3498','3498_3499','3499_3500','35_36','350_2809','350_351','3500_3501','3501_3502','3502_3503','3503_3504','3504_3505','3505_3506','3506_3507','3507_3508','3508_3510','351_352','3510_3512','3512_3514','3514_3515','3515_3516','3516_3517','3517_3518','3518_3519','3519_3521','352_353','3521_3523','3523_3524','3524_3525','3525_3527','3527_3528','3528_3529','3529_3532','353_354','3532_3533','3533_3491','3535_3536','3536_3537','3537_3538','3538_7556','3539_3540','354_355','3540_3541','3541_3542','3542_3543','3542_3544','3543_3141','3544_3552','355_356','3552_3553','3553_3554','3554_3555','3555_3556','3556_3557','3557_7317','356_357','3562_2899','3563_444','3565_3825','3566_3567','3567_7292','3569_3571','357_390','3571_3801','3572_3701','3573_3662','3574_3641','3575_3576','3576_3577','3576_3648','3577_3578','3578_3579','3579_3580','3580_3640','3583_3584','3584_3585','3585_3587','3585_3625','3587_3588','3588_3589','3588_3590','3589_3591','3589_3605','3590_3589','3591_3592','3592_3593','3593_3594','3594_3595','3595_3596','3596_3597','3597_3598','3598_4465','3599_3600','3599_3610','36_37','3600_3601','3601_3602','3602_3604','3604_3616','3605_3606','3606_3607','3607_3608','3607_6051','3608_3609','3609_3598','3610_3611','3611_6135','3612_3613','3613_3615','3615_3616','3616_3618','3618_3619','3619_3620','3619_6087','3620_3621','3620_3622','3621_3622','3622_3623','3623_4387','3624_3587','3625_3626','3626_3627','3627_3628','3628_6052','3629_3630','3630_3631','3631_3632','3632_3633','3633_3634','3634_3635','3635_3636','3636_905','3639_3644','3640_3574','3641_3642','3642_3643','3643_3583','3644_3645','3645_3646','3646_3647','3647_3661','3648_3649','3649_3650','3650_7328','3651_3652','3652_3653','3653_3654','3653_6055','3654_5084','3655_5086','3656_3657','3656_7194','3657_3658','3658_3659','3658_7329','3659_3660','3660_3647','3661_3573','3662_1212','3663_7347','3664_3813','3665_7400','3669_3671','3669_7349','3669_7399','3669_7611','3670_3671','3671_3672','3672_3674','3674_3675','3675_3676','3675_5030','3676_3677','3676_5073','3677_3678','3678_3679','3679_3711','3679_5075','3682_3864','3686_5077','3689_3690','3689_3691','3690_5079','3690_6117','3691_3572','3691_3692','3692_3694','3694_3695','3695_3696','3695_7115','3696_3697','3697_3698','3698_3699','3699_3885','37_38','3701_3702','3702_3088','3704_3705','3705_733','3706_3707','3707_3708','3708_3709','3709_3710','371_391','3710_4925','3711_3712','3711_7189','3712_3713','3712_4916','3712_6359','3713_3714','3714_3715','3714_3719','3715_3755','3715_5175','3718_3720','3719_3718','372_373','3720_7691','3721_3722','3722_5069','3723_3724','3724_3725','3725_3726','3725_3738','3726_7039','3728_3729','3729_3730','373_374','3730_3732','3732_3733','3733_3734','3734_3735','3735_7039','3736_3738','3738_7170','374_375','3740_3741','3741_5165','3743_3780','3744_3715','3744_3745','3745_3746','3746_3749','3748_6050','3749_3750','375_2804','3750_3752','3750_6252','3751_3689','3751_7114','3752_3751','3755_3756','3756_3757','3757_3758','3758_3759','3759_3760','376_377','3760_3761','3761_3762','3762_3763','3763_3764','3764_3765','3765_3766','3766_3767','3767_3768','3768_3770','3769_3771','377_378','3770_3769','3771_3772','3772_3773','3773_3774','3773_5176','3774_7172','3775_3777','3777_3778','3778_7690','378_380','3780_3744','3781_3782','3782_3783','3783_3784','3784_3785','3785_3786','3786_3787','3787_3788','3788_3789','3789_3790','3790_3791','3791_3792','3792_3793','3793_3794','3794_3795','3795_3797','3797_3798','3798_3799','3798_6076','3799_6076','38_39','380_2087','3801_3802','3802_3805','3805_7175','3806_3807','3807_3808','3808_3809','3809_7174','381_382','3811_3812','3812_3664','3813_3814','3814_3815','3815_3816','3816_3566','3818_3819','3819_3820','382_4451','3820_3822','3822_3823','3823_3824','3824_3565','3825_3826','3826_3827','3827_3828','3828_3829','3829_3831','383_384','3831_3832','3832_3833','3833_3834','3834_3835','3835_3836','3836_3837','3837_3838','3838_3840','384_385','3840_3842','3842_3843','3843_3844','3844_3845','3845_3846','3846_3848','3848_3849','3849_3850','385_387','3850_3851','3851_3852','3852_3853','3853_3854','3854_3855','3855_3856','3856_3857','3857_3858','3858_3859','3859_3860','3860_3861','3861_3862','3862_3863','3863_3748','3864_3865','3865_4910','3865_7133','3867_7135','387_388','388_389','3881_5096','3884_3686','3885_1631','3885_3663','3886_3887','3886_3888','3887_3888','3888_3890','3888_7186','389_393','3890_3891','3891_3949','3891_7187','3892_3893','3893_3895','3895_3896','3896_3897','3897_3898','3898_3899','3899_3900','39_116','39_40','390_372','3900_3901','3901_3902','3902_3904','3904_3905','3905_3906','3905_7085','3906_3907','3907_3908','3907_3909','3908_3909','3909_5118','391_392','3910_3911','3911_3912','3912_3913','3913_3914','3914_3925','3914_7075','3917_3918','3918_3926','3918_5114','3919_3920','392_395','3920_3921','3921_3922','3922_3923','3923_3112','3923_7079','3924_3927','3925_3008','3926_7074','3927_3928','3928_3929','3929_7108','393_371','3935_3936','3936_3938','3936_7110','3938_3939','3939_3940','3940_3941','3941_3943','3943_3944','3944_3945','3945_3946','3946_4001','3947_3892','3949_3950','395_396','3950_2229','3950_3372','3952_2190','3954_3955','3955_3956','3956_3957','3957_3958','3957_3993','3958_3959','3959_3960','3959_7234','396_397','3960_3961','3960_6156','3961_3962','3962_3963','3963_3964','3964_3965','3965_3968','3966_5113','3968_3966','3968_7771','397_398','398_399','3980_6089','3981_3982','3982_3983','3982_7704','3983_3985','3985_3986','3986_3987','3986_7313','3987_3989','3989_3990','399_400','3990_3991','3991_3998','3993_3994','3994_3995','3995_3996','3996_3997','3996_3998','3997_3998','3998_3999','3999_4000','4_194','4_4962','4_7555','40_146','40_41','400_279','400_281','400_303','400_319','400_346','400_4522','400_7392','400_7564','400_7581','4000_4001','4001_3949','4001_7193','4002_5112','4003_4404','4004_4005','4005_4006','4006_4007','4007_4008','4008_4010','4009_4011','4010_4009','4011_5110','4012_4013','4013_4014','4014_4098','4015_4017','4017_4018','4018_4019','4019_4020','4020_4100','4021_4022','4022_4023','4023_4024','4024_4027','4027_7250','404_747','405_408','4051_4052','4052_4054','4054_4055','4055_4057','4056_4133','4057_4101','4058_4059','4059_4060','406_2810','406_4530','406_747','4060_4061','4061_4062','4062_4063','4063_4064','4064_4056','4065_4066','4066_4067','4067_4068','4068_4069','4069_4070','407_2498','4070_4071','4071_4072','4072_4073','4073_2583','408_409','4086_4087','4087_4088','4088_4089','4089_4090','409_410','409_411','4090_4091','4091_6097','4093_4094','4094_4096','4096_4114','4098_4015','41_42','410_411','4100_4021','4101_4102','4102_4058','411_412','411_414','4114_4115','4115_4116','4116_4117','4117_4118','4118_4119','4119_4120','412_413','4120_4121','4121_4122','4122_4123','4123_3283','4124_4125','4125_4126','4126_4127','4127_4128','4128_4129','4129_4130','413_414','4130_4131','4131_7294','4132_4134','4132_4171','4133_4065','4134_4135','4135_4136','4136_4207','414_415','414_416','4148_4149','4149_3045','415_416','4150_4151','4151_2997','4151_4156','4153_4154','4154_4416','4156_4168','416_417','4168_4170','417_418','417_485','4170_4132','4170_4153','4171_4172','4172_4173','4173_2962','4175_5091','4177_4178','4178_4179','4179_2993','418_419','4180_4181','4181_4182','4182_4151','419_420','42_43','420_2088','420_421','4201_4202','4202_4203','4203_4204','4203_5090','4204_4205','4205_4206','4206_3140','4207_4208','4208_4209','4209_4210','421_422','4210_4211','4211_4212','4211_4215','4212_4213','4213_5139','4214_4215','4215_4216','4216_4217','4216_4222','4217_4218','4218_4219','4219_4220','422_423','4220_4221','4221_4222','4222_4224','4224_4225','4225_4226','4225_7462','4226_4227','4227_4228','4228_4229','4229_4230','423_424','4230_4231','4230_4248','4231_7352','424_425','4246_4248','4248_4249','4248_4252','4249_4250','425_426','4250_4251','4251_4252','4252_4253','4252_4305','4252_6137','4253_4254','4253_4268','4253_4305','4254_6137','4256_4257','4257_4258','4258_4259','4259_7272','426_427','4260_4261','4260_7296','4261_7271','4262_4263','4263_4264','4264_7296','4265_4266','4266_4253','4266_4267','4266_4307','4267_4268','4268_4269','4268_4273','4269_4270','427_428','4270_4271','4271_4273','4273_4274','4273_4277','4274_4275','4275_4276','4276_4277','4277_4278','4277_7462','4278_4279','4279_4280','428_429','4280_4281','4281_4282','4282_4283','4283_4284','4284_4285','4284_4295','4285_4286','4286_4287','4287_4288','4288_4295','429_3032','429_6334','4295_4296','4296_4297','4296_4301','4297_4299','4299_4300','43_44','4300_4301','4301_4302','4302_4591','4302_5135','4303_4304','4304_4148','4305_4306','4306_4307','4306_4309','4307_4308','4308_4253','4308_4267','4309_4254','4314_1687','4315_4316','4316_4317','4317_7013','4319_1476','4320_1476','4321_7342','4323_1845','4324_4769','4325_3328','4330_3679','4331_4339','4333_4334','4334_4712','4336_1051','4339_7616','4339_7692','4342_4348','4344_2349','4347_2349','4347_7181','4348_4646','4348_7122','4349_3184','435_2011','435_7353','4350_5143','4353_4355','4354_3196','4355_4354','4362_2959','4362_7025','4363_1598','4364_1579','4369_6219','437_459','4371_832','4372_179','4376_4377','4377_4890','4379_1963','4379_1968','4380_1172','4380_496','4382_1185','4382_1196','4383_4699','4384_518','4384_519','4385_1201','4386_653','4387_3634','4388_4700','4392_2458','4397_2848','4399_4400','44_7603','4400_1464','4401_2202','4401_2213','4404_4405','4405_3991','4406_1984','4407_2637','4407_4320','4408_1473','4409_1972','4409_2173','4409_4550','441_442','4410_2678','4411_2239','4411_4715','4413_1476','4413_1480','4413_7453','4415_301','4415_302','4416_4201','442_443','443_444','4432_119','4433_3752','4434_2379','4435_2594','4435_2604','4435_2632','4436_5008','4438_3432','444_445','4440_2628','4441_2626','4445_2352','4446_2603','445_4779','4451_383','4455_381','4456_1284','4458_1658','4460_453','4462_3235','4464_1869','4465_944','4466_633','4468_2283','4469_2262','447_5045','4472_4386','4475_3023','4477_3321','4479_4376','448_3181','4481_4485','4485_822','4486_2865','4489_4747','449_450','4492_1861','4493_4362','4495_1278','4495_1279','4495_2809','4495_4522','4495_4720','4495_746','4495_7581','4498_1660','4499_4934','45_46','450_3181','4500_4481','4501_4935','4502_4936','4504_7028','4508_7402','4509_499','4509_7708','451_447','4513_272','4513_286','4514_4515','4515_1995','4516_1369','4518_509','4520_4825','4521_1283','4521_1934','4521_1935','4522_1934','4523_4524','4524_4322','4525_4526','4526_1700','4528_1072','453_4477','453_454','4530_748','4532_4534','4533_4844','4534_4535','4535_2169','4536_6003','4537_2176','4538_7396','4539_1606','454_4475','4542_1531','4543_1560','4545_4552','4546_1559','4548_1514','4548_980','4549_1964','4550_4551','4551_1965','4552_4554','4554_7016','4554_7229','4554_7230','4555_3418','4555_4576','4557_3441','4558_3411','4559_3389','4560_5126','4561_4629','4562_7386','4563_1218','4564_3443','4564_3444','4565_4566','4566_4567','4567_2031','4567_3238','4568_4569','4569_2023','4569_4570','4570_2056','4571_2013','4571_2014','4573_4893','4574_4575','4575_1680','4576_3418','4577_4578','4578_1100','4583_1518','4583_1576','4588_4589','4589_7364','459_2080','4590_7366','4591_4590','4592_980','4593_4594','4593_4595','4593_7516','4594_4595','4595_4596','4596_4563','4599_2217','4599_4601','4599_7184','46_47','460_2011','4601_5058','4602_4603','4603_4604','4604_4605','4605_4562','4606_7142','4607_4608','4607_7140','4608_4609','4609_4610','461_2068','4610_4611','4611_4612','4612_4613','4613_3370','4616_2227','4617_4555','4619_4561','4620_7563','4623_4619','4624_4625','4625_4628','4626_4627','4627_4624','4628_4623','4629_4630','4630_4631','4631_4632','4632_5164','4633_2234','4636_2016','4640_4347','4640_7214','4641_4642','4642_2355','4643_4644','4644_4645','4645_4440','4646_4647','4647_2346','4647_4435','4662_2335','4663_4664','4664_2160','4665_4666','4666_4667','4667_2162','4667_4668','4668_3376','4668_4677','4669_2146','4669_2165','4670_2165','4670_3420','4671_4672','4672_3465','4673_2172','4674_4663','4677_2136','4680_31','4680_90','4683_107','4688_4689','4689_4410','469_470','4690_4411','4691_2682','4691_4796','4692_1245','4693_1254','4694_1249','4695_1243','4696_4697','4697_1250','4698_4383','4699_4388','47_48','470_471','4700_1266','4705_475','4707_4708','4708_4336','4709_1037','471_472','4710_1046','4712_2853','4713_4714','4714_4960','4717_2499','472_473','4720_1443','4724_263','4725_273','4725_334','4725_4495','4725_6059','4725_6071','4726_7555','4727_4728','4727_768','4728_437','4728_461','473_4705','4731_2023','4732_3243','4742_4743','4743_4744','4743_4745','4744_4745','4745_4989','4746_1845','4747_1845','4747_1882','4749_4750','475_476','4750_3006','4750_4751','4751_4754','4754_4755','4755_3003','4756_3005','4757_4758','4758_6343','4759_4760','476_477','4760_2547','4762_4763','4763_6245','4765_4766','4766_4767','4767_4768','4768_1845','4768_4325','4769_4770','477_478','4770_4765','4779_7057','478_479','478_7053','4782_1782','4788_6273','479_480','4793_6068','4795_2686','4795_4688','4796_4690','4797_7473','4798_2674','4798_4715','4799_2205','48_49','480_481','4800_5157','4804_738','4806_5156','481_482','482_483','4825_7233','483_485','4830_7667','4843_4905','4844_4177','4847_4848','4848_4350','485_487','485_488','4854_1438','4855_1439','4856_4857','4857_307','4861_4862','4863_4864','4864_4865','4865_4577','4869_4870','487_488','4870_3007','4871_4872','4872_4873','4873_4874','4874_2532','4877_4878','4878_7180','488_489','488_491','4880_2539','4880_2606','4882_4883','4883_4884','4884_4885','4885_6343','4886_4887','4887_1150','4888_4798','489_490','4890_4369','4891_7458','4893_4894','4894_4895','4895_7217','4897_4898','4898_4574','4899_4900','49_51','490_491','4900_4746','4901_4902','4902_1672','4903_1883','4903_7219','4905_4906','4906_4907','4907_4908','4908_4909','4909_3884','4909_4921','491_492','491_493','4910_4911','4911_4912','4912_4913','4913_4914','4914_4915','4915_4957','4915_7654','4916_4917','4917_4918','4918_4919','4919_4920','492_493','4920_4910','4921_4922','4922_4923','4923_4924','4924_4958','4925_3679','4925_4926','4926_3678','4927_4928','4928_2352','4929_7460','493_494','4930_4931','4931_7459','4932_4933','4933_2590','4934_4500','4935_4502','4936_79','4937_1547','4938_1549','494_495','495_400','495_496','495_7587','495_7588','4952_768','4953_768','4957_7133','4958_3752','496_497','496_515','496_522','496_523','4960_4932','4962_264','4962_4725','4962_6059','497_515','4974_4976','4976_7126','4978_4979','4979_4986','4980_2289','4981_3211','4984_448','4985_6219','4986_7107','4987_2978','4988_2853','4989_1845','4989_4990','499_500','4990_4991','4991_4992','4992_1719','4994_4995','4995_4996','4996_1826','4997_4998','4998_4999','4999_1833','500_501','5008_4640','501_1484','5013_5014','5014_7291','5015_7290','5022_3461','5022_6002','5023_5024','5024_5161','5025_4854','5026_2450','5028_3696','5028_7115','5029_5028','5030_5031','5031_6033','5032_2824','5032_2867','5033_7130','5044_3751','5044_4433','5045_4460','5046_7639','5047_3146','5047_3224','5051_5052','5052_2975','5053_1633','5056_2239','5058_4602','5060_5061','5061_1231','5062_691','5064_7173','5069_3723','5071_5072','5072_4321','5072_7342','5073_3679','5073_5074','5074_6054','5075_5076','5076_3682','5077_3694','5077_5078','5078_3689','5079_3694','5081_5082','5082_3639','5084_6054','5086_3656','509_510','5090_4204','5091_5092','5092_7574','5096_7145','5097_2383','5099_2383','51_52','510_511','511_513','5110_4012','5111_4051','5112_4003','5113_3918','5118_3910','5119_5120','512_513','5120_5121','5121_5122','5122_3461','5123_5124','5124_5125','5125_6001','5127_5128','5128_2060','5128_2996','5129_5130','513_4513','513_7402','5130_1135','5131_5132','5132_1331','5133_2596','5134_2718','5135_4303','5136_4262','5139_4214','5140_1500','5140_620','5142_7567','5143_2837','5144_7027','5144_7702','5145_276','5147_68','5148_1401','5149_1419','515_516','5150_2327','5154_4804','5155_4806','5156_7552','5157_5154','516_4384','5160_5155','5161_5162','5162_2571','5162_4880','5164_4633','5165_3743','5165_7169','5169_2735','5171_2976','5174_6078','5175_3755','5176_3774','518_2255','519_521','5192_342','5192_350','52_261','52_262','52_263','52_265','52_277','52_4725','521_522','522_1741','522_523','523_1740','523_4382','523_525','523_669','525_526','526_527','527_528','528_529','529_530','53_54','530_531','531_709','532_533','532_649','533_534','534_535','535_536','536_537','537_538','538_539','539_540','54_55','540_541','541_542','542_543','543_544','544_545','544_546','545_546','546_547','546_910','546_914','547_548','548_549','549_550','549_677','55_56','550_551','551_552','552_553','553_554','554_555','555_557','557_558','558_559','559_560','56_58','560_562','562_563','563_564','564_565','565_566','566_567','566_707','567_568','568_569','569_570','57_58','570_571','571_572','572_573','573_574','574_575','575_576','576_577','577_579','579_580','58_4499','58_59','580_581','581_582','582_583','583_584','584_585','585_586','586_587','587_588','588_589','588_590','589_590','59_60','590_1729','590_591','591_592','592_593','593_594','594_595','595_596','596_597','597_598','598_599','599_600','6_194','60_1802','60_820','600_601','600_654','6001_4438','6002_5119','6003_5123','6004_3','6004_7','6005_1546','6005_7072','6006_3493','6007_2836','601_602','6010_3656','6011_2133','6013_6247','6014_2158','6014_6242','6015_5029','6016_6017','6017_7192','602_603','6020_6023','6021_7507','6023_6026','6024_6021','6025_6024','6026_6027','6027_6029','6028_6025','603_604','6030_6028','6033_3089','604_605','6041_2826','6042_1022','6048_5064','605_606','6050_3749','6050_7398','6051_6053','6052_3613','6053_7327','6054_4330','6055_1073','6057_2183','6059_334','6059_404','606_607','6068_1584','607_608','6070_2708','6074_748','6074_793','6075_2631','6076_6077','6077_3569','6078_7112','6079_6315','608_609','6080_1585','6080_983','6083_2733','6085_2026','6085_2053','6086_3161','6086_7419','6087_3620','6088_1898','6089_3981','609_610','6094_2102','6095_7062','6097_4093','610_611','6107_6108','6107_6109','6108_6109','6109_6110','611_612','6110_7020','6110_7029','6111_6112','6112_6113','6113_1898','6113_4323','6115_1205','6115_4593','6117_3655','6117_5079','6119_3392','612_613','6122_35','6124_4004','6125_7092','613_614','6130_1262','6132_2551','6132_4861','6135_3612','6137_4256','6137_4956','614_4415','614_615','614_675','6140_6144','6141_6142','6142_2181','6143_6011','6144_6145','6145_6146','6146_6147','6147_6149','6149_6150','615_616','6150_6143','6152_6153','6153_6243','6154_6155','6155_1987','6155_6141','6156_6165','6157_6158','6158_3964','6158_7314','6159_6160','616_617','6160_6161','6161_3991','6162_7022','6163_1095','6164_6159','6165_6157','617_618','6170_5023','6170_7183','6171_4974','6172_4493','618_619','6182_322','6185_6186','6186_6216','6187_6188','6188_6215','619_675','620_1172','620_621','620_7568','620_7569','621_289','621_291','621_293','621_4495','6215_6184','6216_3400','6219_6229','6228_4788','6228_7111','6229_6348','6229_6364','6229_7131','6230_6228','6234_332','6235_327','6238_6182','6239_104','624_625','6241_105','6242_6152','6243_6154','6244_2134','6245_2156','6245_4671','6245_6013','6247_2679','625_626','6251_5140','6251_7706','6252_301','6252_302','6253_7191','626_627','627_628','6273_136','6273_4891','6276_6362','6277_6278','6278_6279','6279_6280','628_629','6282_6335','6283_6282','6285_6286','6286_6287','6287_6288','6288_6342','629_630','6290_7120','6298_6336','63_64','630_4466','6310_6311','6311_4495','6311_7399','6312_7197','6315_6316','6316_7236','6318_6319','6319_7246','6320_4594','6324_6325','6325_6341','6326_4886','633_634','6334_7660','6335_6326','6336_7069','6339_6324','634_1005','6341_6277','6342_6298','6343_4759','6348_1182','635_636','6359_7189','636_637','6361_6367','6362_7276','6363_6361','6364_1558','6365_6276','6366_6230','6367_4985','637_638','638_639','639_640','64_65','640_641','641_642','642_643','643_644','644_645','645_1796','649_650','65_66','650_651','651_4472','653_722','654_655','655_656','656_657','657_658','658_659','659_604','66_4498','664_614','664_665','665_666','666_667','667_668','668_614','669_670','670_671','671_672','672_4382','675_1184','675_1500','675_303','675_4380','675_4415','675_620','675_621','677_678','678_679','679_680','68_30','680_681','681_682','682_683','683_684','684_685','685_686','686_688','688_689','689_690','69_7686','69_828','690_5062','691_692','692_693','692_694','692_695','693_585','694_695','695_696','696_697','697_698','698_699','699_700','7_11','7_194','700_701','7009_1604','701_702','7010_7703','7011_2171','7011_7160','7012_2201','7012_2644','7013_6162','7014_4315','7015_4617','7016_4619','7017_7018','7018_7030','702_703','7020_7029','7021_6111','7021_7163','7022_1554','7023_7014','7025_4464','7027_4538','7028_5147','7029_7038','703_704','7030_6111','7030_7021','7031_4897','7038_7011','7039_7204','704_705','7043_1424','7044_993','7047_7161','7048_7699','7049_2236','705_706','7050_7185','7053_2088','7056_4731','7057_4847','7057_5142','706_567','706_707','7062_6075','7063_1137','7064_7066','7065_7063','7066_2924','7067_2956','7067_2984','7068_2966','7069_7443','707_708','7070_7445','7073_7097','7074_3919','7075_3925','7078_1449','7079_7080','708_731','7080_7081','7081_7565','7082_7083','7083_7084','7084_7108','7085_7086','7086_7087','7087_7088','7088_7089','7089_7091','709_532','7091_3910','7092_110','7097_7098','7098_7099','7099_7100','71_72','710_711','7100_7101','7101_7505','7102_7103','7103_7104','7104_7105','7105_7106','7106_6005','7107_7102','7108_3935','711_712','7110_3938','7111_6365','7112_972','7113_127','7114_7116','7115_6310','7116_3694','7117_7118','7118_6289','7119_7127','712_713','7120_7119','7121_7183','7122_7121','7123_4347','7124_7343','7125_7344','7126_7124','7127_7143','7128_7117','713_714','7130_1237','7132_1558','7132_6230','7133_7134','7134_3867','7135_7136','7136_3874','7137_7139','7138_3370','7139_4623','714_715','7140_7141','7141_7138','7142_7015','7143_3368','7143_7168','7144_3881','7145_3884','7145_4843','7149_7277','715_716','7153_6056','7154_7153','7156_7020','7158_7048','7159_7017','7159_7388','716_717','7160_7700','7161_7162','7163_1893','7166_1847','7167_1808','7168_4616','7169_3743','717_718','7170_3740','7172_3775','7173_3811','7175_3806','7176_2836','7177_2740','7178_2765','718_719','7180_7123','7180_7331','7183_4880','7184_2225','7184_7050','7185_7186','7186_7187','7187_3947','7187_3954','7187_4001','7188_3082','719_720','7191_6238','7192_4683','7193_7049','7194_7196','7195_3653','7196_6310','7197_7195','72_5144','720_680','720_721','7201_2138','7204_3728','7204_3736','7207_7288','7208_7249','721_680','721_693','7214_4347','7216_2501','7217_7227','7218_7031','7219_7218','722_723','7222_6170','7224_1581','7227_6172','7229_3418','7229_4617','723_724','7230_4617','7231_7128','7233_4573','7234_4002','7236_7245','7237_7241','7238_7224','7239_4401','724_725','7240_1891','7241_7247','7242_7238','7243_7126','7245_6317','7246_6320','7247_7243','7248_7207','7248_7254','7249_7253','725_726','7250_7248','7250_7252','7251_4051','7252_7248','7253_7251','7254_7257','7255_7249','7256_7255','7257_7258','7258_7260','7259_7256','726_727','7260_7263','7261_7259','7262_7261','7263_7264','7264_7268','7265_7262','7266_7265','7268_7270','727_728','7270_6122','7270_7266','7271_5136','7272_4956','7276_7149','7277_6363','7278_7289','728_729','7280_7281','7281_7284','7282_7278','7283_7282','7284_7287','7286_7283','7287_7208','7288_7286','7289_7280','729_730','7290_5016','7291_3485','7292_3818','7293_1335','7294_4132','7294_4156','7296_4265','7296_4266','73_74','730_5160','731_710','7313_6164','7314_3964','7317_3224','7318_4978','7318_4980','7319_7342','7320_1486','7325_3317','7325_4984','7326_7418','7327_3629','7328_3651','7329_3659','733_6015','7330_2051','7331_2559','7333_783','7334_1838','7340_783','7342_7125','7343_4978','7343_4980','7343_7318','7344_2510','7344_7334','7347_3669','7348_3669','7348_3671','7349_1631','7350_1558','7350_6230','7352_4246','7353_4571','7353_7584','7354_1975','7358_1231','7358_1235','7358_5174','7361_7362','7362_3129','7362_3134','7363_4588','7364_7365','7365_4210','7366_7367','7367_7368','7368_3143','7370_7339','7374_1845','7375_3466','7379_4903','738_739','7380_1676','7381_4742','7382_7381','7386_4607','7388_7017','7389_4324','7389_7025','739_740','7391_493','7392_1443','7392_312','7395_6124','7396_5145','7397_5140','7397_6252','7398_6252','7398_7397','7399_7398','7399_7611','74_4371','740_741','7400_3669','7401_3665','7402_4380','7402_4717','741_742','7412_7413','7413_2383','7414_1396','7415_7416','7416_7325','7417_3471','7418_7417','7419_7420','742_743','7420_3345','7421_7422','7422_6085','7423_1566','7424_7423','7425_7424','7426_7427','7427_7428','7428_138','743_744','7430_177','7431_175','7432_3454','7433_4677','7435_2722','7435_4413','7438_1410','7439_7454','744_745','7440_2940','7441_2933','7442_7446','7443_7444','7444_2941','7445_6339','7446_7070','7449_2941','745_600','7453_1478','7453_1480','7453_2001','7454_1380','7455_7438','7458_4479','7458_7425','746_844','7460_4930','7461_4230','7461_4281','7462_7461','7469_1724','747_748','747_842','747_843','7471_1533','7472_3230','7473_2679','7475_6107','748_7390','748_749','749_494','749_750','749_751','75_4501','750_751','7505_5071','7505_7678','7507_2663','751_752','7510_2207','7510_4799','7513_807','7514_2279','7516_6079','752_7339','752_753','753_7339','753_7370','753_754','754_755','755_756','7550_2570','7551_4800','7552_924','7555_4962','7556_3539','7559_7073','7559_7107','756_757','7560_7679','7563_3368','7563_7231','7564_340','7564_4521','7565_7082','7567_7326','757_758','7571_207','7574_4177','7577_1353','7578_7582','7579_1285','758_759','758_760','7581_1283','7582_297','7582_299','7582_319','7582_340','7582_407','7584_3344','7585_7587','7586_7587','7587_303','7587_328','7587_345','7587_407','7587_496','7587_7588','7588_281','7588_325','7588_328','7588_329','7588_4717','759_760','7591_496','7592_3815','760_2007','760_761','760_762','7601_2508','7602_22','7602_85','7603_45','761_762','7611_7398','7612_2811','7616_5081','762_763','7622_297','7622_334','7623_2508','7623_3676','7623_3715','7623_3719','7623_5030','7623_6312','7623_7401','763_2007','763_4952','763_4953','763_767','7639_7640','7640_3134','7640_5047','7641_7642','7642_7643','7643_3214','7644_7645','7645_7646','7646_3238','7654_4843','7660_3114','7661_1521','7667_3091','767_768','7671_513','7672_826','7673_7674','7674_7675','7675_7676','7676_7677','7677_7559','7678_7560','7679_7680','768_2085','768_769','768_770','768_772','7680_7681','7681_7738','7683_1582','769_770','7690_3781','7691_3721','7692_6010','7698_7113','7699_7159','77_78','770_771','770_773','7700_7047','7701_7010','7702_7009','7703_5147','7704_3983','7705_6251','7706_7707','7707_512','7708_7709','7709_7710','771_772','7710_7711','7711_7705','772_417','772_773','772_775','773_774','7738_7683','774_775','774_906','775_776','775_906','776_777','777_779','777_906','7771_3966','779_780','78_79','780_781','781_782','782_783','783_784','783_785','784_785','785_786','786_791','786_792','786_793','79_80','790_7585','791_7585','792_320','792_7586','793_274','793_320','793_495','793_7586','795_796','796_56','796_57','796_797','797_798','798_799','799_800','8_12','80_82','800_801','801_802','802_1654','802_803','803_804','804_7513','807_808','808_809','809_810','81_817','810_811','811_812','812_813','813_814','814_81','817_818','818_819','819_261','819_263','819_264','819_265','819_4725','82_817','82_83','820_821','821_63','822_823','823_824','824_825','825_826','826_827','827_71','828_829','829_830','83_84','830_831','831_75','832_833','833_77','834_835','835_836','836_837','837_1506','838_839','839_840','84_51','840_841','841_49','842_1014','842_845','842_846','843_1013','843_845','844_846','845_846','846_7340','846_847','846_848','847_848','848_2795','848_758','848_759','848_849','849_850','85_203','85_86','850_851','851_852','852_853','853_854','854_855','855_2815','855_856','856_857','857_858','858_859','859_860','86_87','860_861','861_862','862_3014','87_88','875_876','876_3011','876_878','877_876','878_879','879_880','88_89','880_881','881_882','882_883','883_884','884_885','885_886','886_887','887_2900','887_903','89_27','893_894','894_1067','895_1025','896_897','897_893','90_91','903_904','904_906','905_4331','906_907','906_909','907_908','908_909','909_786','909_790','909_792','91_129','91_92','910_911','911_912','912_913','913_914','914_938','915_916','916_917','917_918','918_919','919_7551','92_93','924_925','925_927','927_938','928_588','928_929','929_930','93_94','930_931','931_587','938_940','94_102','94_126','940_941','941_942','942_943','943_3599','944_945','945_947','946_1155','947_948','948_949','949_928','950_915','956_957','957_958','958_959','959_960','960_961','961_962','962_635','969_1176','972_1625','980_981','981_1518','981_1576','983_984','984_985','985_1529','987_4937','992_7044','993_5060']

Get all outliers i.e. rows where the value is more than three times the standard deviation for the segment.

In [ ]:
# based on https://stackoverflow.com/questions/45928046/identifying-statistical-outliers-with-pandas-groupby-and-individual-columns
stds = 3.0
z = df[['segment', 'time_diff']].groupby('segment').transform(lambda group: (group - group.mean()).div(group.std()))
outliers = z.abs() > stds

Print the outliers:

In [ ]:
df[outliers.any(axis=1)]

Remove the outliers:

In [ ]:
df_outliers = df[outliers.any(axis=1)]
df = pd.concat([df,df_outliers]).drop_duplicates(keep=False)

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After removing outliers, we have", rows, "rows and", cols, "columns.")
print()

## 10.2 Segment Checks

We need to check if we are missing data for any segments.

In [ ]:
# loop through each segment and check how many rows we have for it
# if there are no rows for a particular segment, then add it to the missing list
missing = []
for segment in segments:
    df_temp = df.loc[(df['segment'] == segment)]
    if df_temp.shape[0] == 0:
        missing.append(segment)

In [ ]:
len(missing)

In [ ]:
print("Missing Segments")
for seg in missing:
    print(seg)

These segments do not appear in the data, but exist in current Dublin Bus routes. As such we must deal with them in some way as predictions may be requested for them.

# 11. Export the Cleaned Data

## 11.1 Remove Features

Remove features that we no longer need at this point:

- dayofservice
- progrnumber_first
- stoppointid_first
- progrnumber_next
- stoppointid_next
- actualtime_arr_stop_next
- record_date
- month
- rhum
- msl

In [ ]:
df = df.drop(columns=['dayofservice', 'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next', \
                     'actualtime_arr_stop_next', 'record_date', 'month', 'rhum', 'msl'])

## 11.2 Export the Data

The cleaned data is exported at this point so that it can be reloaded easily without having to re-run the whole notebook:

In [ ]:
df.to_hdf('/data_analytics/data/december_bus_data_cleaned.hdf', key='cleaned', format='table')

# 12. Data Quality Plan

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| time_diff | There are many rows (571) where this feature is zero or negative. This doesn't make sense in the context of time difference between stops. | Drop affected rows. |
| segment | There are many rows 66,085) where the value for this feature does not appear in the GTFS data. Our assumption is that these segments are no longer used due to routes changing and stops no longer being used by Dublin Bus. | Drop affected rows. |
| segment_std | NaN values (5 rows) | Drop affected rows. |
| time_diff | Outliers (86,927 rows) | Group by segment, then remove any rows where the time_diff is more than 3 times the standard deviation. |

# 13. Import the Cleaned Data

Import the cleaned data if required:

In [ ]:
df = pd.read_hdf('/data_analytics/data/december_bus_data_cleaned.hdf')

# 14. Encode the Categorical Features

The categorical features we will use to train the model are:

- hour
- weekday
- peak

weekday, peak & school_hol are already binary features. We will encode hour.

In [ ]:
# encode values for hour
df_dummies = pd.get_dummies(df['hour'], prefix='hour')
df = pd.concat([df, df_dummies], axis =1)

# 15. Split Test & Training Data

Split the data between training and test data keeping trips together:

In [ ]:
# based on https://stackoverflow.com/questions/45980621/how-to-split-a-dataset-to-train-test-where-some-rows-are-dependent
unique_trips = df['tripid'].unique()
train_trips, test_trips = np.split(
    np.random.permutation(unique_trips), [int(.7 * len(unique_trips))]
)

df_train = df[df['tripid'].isin(train_trips)]
df_test = df[df['tripid'].isin(test_trips)]

# 16. Prepare Features

In [ ]:
# Prepare the descriptive & target features for the training data
X_train = df_train[['actualtime_arr_stop_first','segment_mean','weekday', 'segment_std','peak', 'school_hol', 'rain', 'temp', 'hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23']]
y_train = df_train.time_diff

In [ ]:
# Prepare the descriptive & target features for the test data
X_test = df_test[['actualtime_arr_stop_first','segment_mean','weekday', 'segment_std','peak', 'school_hol', 'rain', 'temp', 'hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23']]
y_test = df_test.time_diff

In [ ]:
# normalise the features for training
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# 17. Train the Model

In [ ]:
# specify the GTB parameters
gtb = GradientBoostingRegressor(max_depth=5, n_estimators=125)

In [ ]:
# Fit model on the training data
start = time.time()
gtb_model = gtb.fit(X_train, y_train)
end = time.time()
print("The model was trained in " + str((end - start)/60) + " minutes!")

# 18. Pickle the Model & Scaler

Save the model to file:

In [ ]:
filename = '/data_analytics/model_dec.sav' 
pickle.dump(gtb_model, open(filename, 'wb'))

Save the scaler to file:

In [ ]:
filename = '/data_analytics/scaler_dec.sav'
pickle.dump(scaler, open(filename, 'wb'))

Load the model if required:

In [ ]:
filename = '/data_analytics/model_dec.sav'
with open(filename, 'rb') as file:
        gtb_model = pickle.load(file)

Load the scaler if required:

In [ ]:
filename = '/data_analytics/scaler_dec.sav'
with open(filename, 'rb') as file:
        scaler = pickle.load(file)

# 19. Test the Model

## 19.1 Test for All Segments

In [ ]:
# make predictions based on the test data
gtb_predicted = (gtb_model.predict(X_test))

In [ ]:
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_test, gtb_predicted))
print()
print("Root Mean Squared Error: ", math.sqrt(metrics.mean_squared_error(y_test, gtb_predicted)))
print()
print("R Squared:", metrics.r2_score(y_test, gtb_predicted))

## 19.2 Test Each Segment

In [ ]:
# make a list of all segments
segments = ['10_12','100_1111','1005_1788','1013_1015','1014_1015','1015_1016','1015_1074','1015_2812','1016_1017','1017_1018','1018_1019','1019_1020','102_6239','1020_1076','1021_6042','1022_1023','1023_1024','1024_895','1025_1026','1026_1027','1027_1028','1028_2818','1035_1314','1036_4709','1037_1038','1038_2998','104_6241','1042_1043','1043_1044','1044_1045','1045_4710','1046_1047','1047_1048','1048_1049','1049_4333','105_6253','1051_1052','1052_1053','1053_1054','1054_1036','1055_2868','1062_1063','1063_1064','1064_1065','1065_1066','1066_1067','1067_1068','1068_1069','1069_1070','107_6125','1070_1071','1071_4528','1072_1352','1072_7577','1073_3641','1074_6074','1076_1021','1076_1077','1077_1078','1077_2479','1078_1079','1079_1080','1080_1081','1081_1082','1081_2914','1082_1083','1083_1085','1085_1086','1085_1121','1086_1087','1087_1088','1088_1089','1089_1090','1090_1091','1090_2413','1091_1092','1092_1093','1093_1094','1094_1095','1095_1096','1096_1097','1096_1101','1097_1099','1099_6132','11_14','110_112','1100_100','1101_1102','1102_1103','1103_1104','1105_1107','1107_1108','1108_1109','1109_1110','1110_1111','1111_1112','1111_2323','1111_2404','1112_1113','1113_1114','1114_1115','1115_2437','1117_1118','1118_1119','1119_1120','112_113','1120_1164','1121_1122','1122_1123','1123_1124','1124_1125','1125_1127','1127_1130','113_114','1130_1140','1130_2550','1135_7064','1137_5131','114_115','1140_1141','1140_2515','1141_1142','1142_1143','1143_1144','1144_1145','1145_4869','115_37','1150_1151','1151_1152','1152_1153','1153_1154','1154_946','1155_1157','1157_1158','1158_1159','1159_1160','116_117','1160_1161','1161_1162','1162_1163','1163_1164','1163_1337','1164_1165','1165_1166','1166_1167','1167_1168','1168_1169','1169_1170','117_118','1170_1069','1171_1172','1172_1173','1172_286','1172_4509','1172_7671','1173_1174','1174_1175','1174_4725','1175_15','1176_1177','1177_1178','1178_1179','1179_5060','1179_7112','118_119','1181_7350','1182_1550','1184_497','1185_1186','1186_1187','1187_1188','1188_216','119_44','1190_1191','1191_1192','1192_1193','1193_1194','1194_1195','1195_664','1196_1197','1197_1198','1198_1199','1198_1222','1198_4782','1199_1200','12_14','1200_4385','1201_1202','1201_4695','1202_1203','1203_6115','1205_1206','1206_1207','1207_1208','1208_1209','1209_1210','1210_1211','1210_3575','1211_3575','1212_1213','1213_1214','1214_1215','1215_1216','1216_1217','1217_4563','1218_1258','1218_1270','1219_1220','1220_1221','1221_664','1222_1223','1223_1224','1224_1225','1225_1226','1226_1227','1227_1228','1227_1253','1228_1244','1229_1231','1231_1232','1231_1235','1232_1233','1233_1234','1234_7358','1235_1251','1237_1238','1238_1239','1239_247','1240_1241','1241_1242','1242_1277','1243_4696','1244_1246','1244_254','1245_4693','1246_1229','1247_1248','1248_1273','1249_1247','1250_4698','1251_253','1251_7130','1253_969','1254_1255','1255_1256','1256_4694','1258_1259','1259_1260','126_6182','1260_1262','1260_6130','1262_4692','1265_6115','1266_1267','1267_1268','1268_1269','1269_1265','127_112','1270_1272','1272_1273','1273_1274','1274_1275','1275_1276','1276_1277','1277_1219','1278_1282','1278_1934','1279_1282','1279_1283','1279_1934','1280_6348','1280_6366','1281_1329','1282_4456','1283_4456','1283_7579','1284_1285','1285_1016','1285_1286','1285_1287','1286_1288','1287_1288','1288_1289','1289_1290','1289_1360','129_130','1290_1291','1291_1292','1291_2389','1292_1293','1293_1294','1294_1295','1295_1296','1296_1297','1297_1298','1298_1121','1298_1299','1299_1300','130_131','1300_1301','1301_1302','1302_1303','1303_1304','1303_5129','1304_1305','1305_1306','1305_1320','1306_1307','1307_1308','1308_1309','1309_1310','131_164','1310_1311','1311_1312','1312_1313','1313_1055','1314_1316','1315_1317','1316_1315','1317_1318','1318_1319','1319_1281','1320_1321','1321_1322','1322_1323','1323_1324','1324_7068','1325_1326','1326_1327','1327_1328','1328_1329','1329_1330','133_134','1330_1331','1330_2919','1331_1332','1332_1333','1333_1334','1334_1335','1334_2985','1334_7293','1335_1336','1336_1337','1337_1338','1338_1339','1339_1340','134_4890','1340_1341','1341_1342','1342_1343','1343_1344','1344_1345','1345_1347','1347_1348','1347_2634','1348_1349','1349_1350','1350_1352','1352_1353','1353_1354','1353_1355','1354_7578','1355_1357','1357_1358','1357_1359','1358_298','1358_319','1358_326','1358_340','1358_342','1358_4494','1358_5192','1358_7564','1358_7665','1359_274','1359_279','1359_319','1359_320','1359_4724','136_137','1360_1362','1362_1363','1363_1365','1365_1365','1365_1366','1366_1367','1367_1369','1367_7455','1369_1370','137_193','1370_1371','1371_1372','1371_1454','1372_1373','1373_1374','1374_1375','1374_1946','1375_1376','1376_1377','1377_1378','1378_1379','1378_4514','1379_1380','138_133','138_167','1380_1381','1381_1382','1382_1383','1383_1384','1384_1385','1385_1348','1386_1387','1387_1388','1388_1389','1389_3355','1396_2317','1397_1398','1398_1399','1399_5148','14_15','14_53','1401_1402','1402_1403','1403_1404','1404_1405','1405_4856','1406_1407','1407_1409','1409_2095','141_142','1410_1411','1411_1412','1412_1413','1413_1414','1414_1415','1415_1417','1417_1418','1418_5149','1419_6094','142_143','1423_1424','1424_1425','1424_2186','1424_2190','1425_1426','1426_1427','1426_1463','1427_1428','1428_1429','1429_1430','143_115','1430_1432','1432_1433','1433_1434','1434_1435','1435_7439','1436_3952','1438_4855','1439_1440','1440_1386','1441_1442','1442_7414','1443_1444','1443_1616','1443_1647','1444_1445','1445_1447','1445_1647','1445_4319','1445_4407','1445_7078','1447_2637','1449_1450','1450_1451','1451_1452','1451_2191','1452_2640','1453_1456','1454_1455','1455_1453','1456_1457','1457_1458','1458_1459','1459_1460','146_147','1460_1462','1462_5149','1463_4399','1464_1469','1465_1466','1466_1467','1467_1468','1468_1377','1469_1465','147_184','1471_1472','1472_4408','1473_1474','1474_1476','1476_7453','1478_1479','1478_315','1479_315','1479_7622','148_149','1480_1358','1480_2912','1481_7320','1482_1494','1484_1481','1485_4518','1486_1487','1487_1488','1488_1489','1489_1490','149_150','1491_1492','1492_1493','1493_1482','1494_1495','1495_1496','1496_1515','1499_1485','15_17','15_834','150_27','1500_302','1505_4509','1506_1507','1507_1508','1508_1509','1509_1510','151_153','1510_1511','1511_1512','1511_1592','1512_1538','1514_980','1515_1499','1517_1678','1518_1519','1519_1520','1520_7661','1521_4546','1523_1524','1524_1525','1525_1526','1526_6080','1528_1710','1529_4542','153_154','1531_1532','1532_1533','1533_1534','1534_1535','1535_1536','1536_1537','1537_186','1538_4543','1538_4548','1539_1540','154_155','1540_7426','1546_7072','1546_7560','1547_1548','1548_1280','1549_1523','155_156','1550_1551','1551_1552','1552_1553','1552_332','1553_7023','1554_1555','1555_1556','1556_1557','1557_1181','1558_1549','1558_2957','1558_4542','1559_1560','1559_987','156_157','156_7430','1560_1280','1566_1567','1567_173','1567_4372','157_158','1573_1574','1574_1575','1575_4583','1576_1577','1577_1578','1578_4364','1579_1580','1579_7673','158_159','1580_7237','1581_1582','1582_4793','1584_1585','1584_983','1585_1586','1586_1587','1586_1600','1587_1591','1587_4539','159_1539','1591_4542','1592_1593','1593_1594','1594_1595','1595_1596','1596_4363','1597_1599','1598_1597','1599_1573','16_73','1600_7701','1604_1573','1606_1607','1607_1608','1608_1609','1609_1610','1610_7471','1614_1615','1615_1478','1616_1617','1617_1618','1618_1619','1619_195','1620_220','1622_1623','1623_1624','1624_1625','1625_1626','1626_1627','1627_1628','1628_1629','1629_1630','1629_6234','1630_3670','1630_7347','1630_7348','1631_1632','1632_5053','1633_1634','1634_1635','1635_1636','1636_1637','1637_1638','1638_1639','1638_992','1639_1640','164_165','1640_231','1641_1642','1642_213','1643_7571','1647_1648','1647_1649','1648_1649','1649_1911','165_166','1654_1655','1655_1656','1656_1657','1657_4458','1658_1659','1659_1660','166_7458','1660_1661','1661_1662','1661_4523','1662_1664','1662_1666','1664_1665','1665_1666','1666_1667','1666_1807','1666_1808','1667_1668','1668_1669','1669_1670','167_168','1670_1671','1671_1672','1672_1673','1673_1674','1674_1675','1675_1849','1675_7380','1676_1517','1678_1679','1679_4520','168_169','1680_1681','1681_1682','1682_1683','1683_1684','1684_1685','1685_1686','1686_1687','1687_1688','1688_1689','1689_1690','1689_4899','169_141','1690_1691','1691_1692','1692_1693','1693_1694','1694_1695','1695_1696','1696_1697','1696_1700','1697_1698','1698_1699','1699_1700','17_18','1700_1701','1700_1906','1701_1702','1702_16','1702_1703','1703_1704','1703_1905','1704_1705','1705_1706','1706_1707','1707_1708','1708_1709','1709_1528','1710_1711','1711_1712','1712_1713','1713_1714','1713_1715','1714_1715','1715_1478','1715_7453','1719_1828','1724_1725','1725_1726','1726_1727','1727_1728','1728_1729','1729_1730','1729_1731','173_174','1730_1731','1731_1732','1731_1738','1732_1733','1733_1734','1734_1735','1735_1736','1736_1737','1737_1738','1738_1739','1738_614','1739_614','174_175','1740_1741','1741_1742','1741_1747','1742_1743','1743_1744','1744_1745','1745_1746','1746_1747','1747_1748','1747_1749','1748_1749','1749_1751','1749_544','175_176','1750_1769','1751_1752','1752_1753','1753_1754','1754_1759','1755_1756','1756_1757','1757_1794','1758_1750','1759_1758','176_181','1760_1761','1761_1762','1762_1763','1763_1764','1764_1765','1765_1767','1769_1770','177_178','1770_1771','1771_1760','1772_1773','1773_1774','1774_1775','1775_1776','1776_1777','1777_1778','1778_1755','178_1539','1782_1783','1783_1784','1784_1785','1785_1786','1786_624','1788_1789','1789_7199','179_7431','1791_1792','1792_1793','1793_962','1794_7469','1795_1806','1796_1797','1797_1798','1798_1799','1799_1800','18_19','1800_1801','1801_1277','1802_1795','1803_77','1804_1803','1805_1662','1805_1806','1806_1660','1807_7167','1808_1813','1808_1816','1808_2961','1808_4744','1808_7389','1808_7475','181_182','1813_1814','1814_1816','1814_1852','1816_1817','1816_6171','1817_1818','1818_1819','1819_1822','182_183','1820_2468','1822_4994','1826_1827','1827_7382','1828_1829','1829_4997','183_184','1833_1834','1834_1820','1834_1835','1835_1836','1836_1837','1837_2338','1838_2338','184_185','1840_1841','1841_1842','1841_4314','1842_7374','1844_6172','1845_1696','1845_7166','1847_1696','1849_1851','185_186','1850_1867','1851_1813','1852_1853','1853_1854','1854_1844','1858_1859','1859_1860','186_187','186_838','1860_4492','1861_1862','1862_1863','1863_1864','1864_1865','1865_1866','1866_1850','1867_4489','1869_1870','187_188','1870_1871','1871_1872','1872_1873','1873_1874','1874_1875','1875_1876','1876_1877','1877_1878','1878_1879','1879_1899','188_189','1882_7379','1883_1884','1884_1902','1887_1888','1888_1889','1889_1890','189_190','1890_7240','1891_7156','1893_1894','1894_1912','1895_1896','1896_1897','1897_6088','1898_1858','1899_1887','1899_6107','19_21','190_1614','190_191','190_192','1902_1903','1903_1904','1904_1840','1905_1804','1905_1906','1906_1907','1906_1909','1907_1908','1908_1909','1909_1713','191_819','1911_1805','1911_1913','1911_802','1912_1895','1913_1914','1914_1805','192_795','192_819','193_138','1934_1935','1934_2310','1934_2385','1935_1445','1935_1937','1937_1938','1938_1939','1939_1940','194_195','1940_1941','1941_1942','1941_4516','1942_1943','1943_1944','1944_1945','1945_1946','1946_1947','1947_1948','1947_2642','1948_1949','1949_1950','195_196','1950_1951','1951_1952','1952_1953','1953_1954','1954_1955','1955_1956','1956_1957','1956_2109','1957_1958','1958_1959','1959_1960','196_197','1960_4379','1963_4549','1964_4674','1965_1966','1966_1975','1968_1969','1969_1970','197_198','1970_1971','1971_2114','1971_2160','1972_1973','1973_1974','1974_1975','1975_1977','1975_1982','1977_1978','1978_1980','198_199','1980_1981','1981_1982','1981_4406','1982_2642','1982_4406','1982_7435','1984_1985','1985_1986','1986_1987','1987_1988','1987_1989','1988_1989','1988_2642','1989_1375','1989_1472','1989_1990','1989_1995','199_1506','199_200','1990_1992','1992_1993','1993_1994','1994_1995','1995_1996','1995_1998','1996_1997','1997_1998','1998_1999','1998_2001','1998_2002','1999_2001','2_192','2_4726','2_4962','200_201','2001_2002','2002_1358','2002_1359','2003_1358','2007_2008','2007_4571','2008_2009','2008_2071','2009_2010','201_148','201_202','2010_435','2011_2012','2012_3322','2013_2014','2014_2016','2014_4636','2015_2017','2016_2015','2016_2017','2017_2018','2017_3259','2017_7362','2018_2019','2019_2020','202_151','2020_2021','2021_2022','2022_4565','2023_2024','2024_2025','2025_6086','2026_7226','203_204','2031_2032','2032_2033','2033_2034','2034_2035','2035_2036','2036_2039','2036_2040','2039_2040','204_205','2040_2041','2040_3047','2041_2042','2041_3071','2042_2043','2043_2044','2044_2045','2045_2046','2046_4568','205_1620','205_206','205_215','2051_6086','2053_2054','2054_2055','2055_4565','2055_4732','2056_2057','2057_2058','2058_2059','2059_2060','206_207','2060_2061','2060_2062','2061_2062','2062_2063','2062_2064','2063_2064','2064_2065','2064_4727','2065_4727','2066_2067','2067_4728','2068_2069','2069_2070','207_208','2070_2084','2071_2072','2072_2073','2073_2074','2074_2075','2075_2076','2076_460','208_209','2080_2081','2081_2082','2082_2083','2083_2084','2084_768','2085_2086','2086_4455','2086_479','2087_7053','2088_2089','2089_763','2089_770','209_210','2094_1406','2095_2096','2096_2097','2097_2099','2099_2101','21_7602','21_85','210_1643','210_211','2101_2102','2102_2103','2102_2726','2103_2332','2103_7154','2105_2728','2105_6140','2109_3427','211_212','2114_2115','2115_6011','2118_2119','2119_2120','212_213','2120_2121','2121_2123','2123_2685','2124_2125','2125_2126','2126_2127','2127_2128','2128_2129','2129_4797','213_214','2133_6244','2134_2135','2134_3378','2135_2164','2136_2137','2137_2138','2137_7201','2138_2139','2139_2140','214_4432','2140_2141','2141_2142','2142_2143','2143_2145','2145_4670','2146_2147','2147_2148','2148_2149','2149_2150','215_216','2150_2151','2151_2152','2152_2153','2153_2154','2154_6245','2156_6014','2158_2159','2159_4409','216_217','2160_2161','2160_4665','2161_4665','2162_2163','2163_2168','2164_2165','2165_2166','2166_2167','2166_4545','2166_4554','2166_7229','2167_3383','2168_4536','2169_2170','217_218','217_242','2170_4672','2171_7160','2172_2159','2172_4409','2173_2174','2174_3425','2176_1982','2176_2177','2177_2178','2178_7354','218_219','2181_2182','2182_2778','2183_2184','2184_2185','2185_1423','2185_7043','2186_2187','2187_2188','2188_2189','2189_1436','219_220','2190_4434','2191_2192','2192_2193','2193_2194','2194_2195','2195_2196','2196_2197','2197_2198','2198_2200','22_23','220_1622','220_221','2200_2201','2201_4401','2201_7239','2202_2203','2203_2204','2204_4888','2205_2687','2206_4798','2207_2208','2208_2210','221_222','2210_2211','2211_2241','2213_2214','2213_4688','2214_2215','2215_2216','2215_7184','2216_4599','2217_2218','2218_2219','2219_2220','2219_3365','222_223','2220_2221','2221_2223','2223_2224','2224_3886','2225_4620','2227_7049','2229_2230','223_224','2230_2231','2231_2233','2232_2234','2233_2232','2234_2235','2235_2236','2235_2239','2236_2238','2236_2239','2238_5056','2239_2241','224_225','2241_2242','2242_2243','2242_7435','2243_2244','2244_2245','2245_2246','2246_2247','2247_2248','2248_2249','2249_2250','225_226','2250_2251','2251_1472','2255_2256','2256_2717','2257_2258','2258_2259','2259_2260','226_228','2260_2261','2261_4469','2262_2264','2264_2265','2265_2269','2269_2270','227_230','2270_2274','2274_7601','2278_2279','2279_2720','2279_6251','228_229','2280_2281','2281_2282','2282_4468','2283_616','2289_1822','229_227','23_24','230_231','231_1641','231_232','2310_2311','2311_2312','2311_5025','2312_2313','2313_2314','2314_2315','2315_2094','2317_2318','2318_2319','2319_2320','232_233','2320_2321','2321_2322','2322_6163','2323_2324','2324_2325','2325_2326','2326_5150','2327_2328','2328_2329','2329_2330','233_234','2330_2331','2331_1397','2331_1424','2332_2333','2333_2334','2334_4662','2335_2336','2336_2337','2337_2339','2338_1840','2338_1845','2339_2340','2339_2420','234_235','2340_2601','2346_4435','2349_2351','235_236','2351_2558','2351_2621','2351_4445','2352_2573','2353_2357','2353_4643','2357_2358','2357_2359','2357_2566','2358_2359','2359_2360','2359_7122','236_1190','236_237','2360_4340','2360_4348','2369_2370','237_213','2370_2371','2371_2372','2372_2373','2373_1424','2373_2377','2374_2146','2377_2378','2377_2431','2378_2183','2379_2380','2380_2382','2382_5099','2383_1479','2383_2002','2383_2003','2385_2386','2386_2387','2387_2388','2388_1290','2389_2390','2390_2391','2391_2392','2392_2393','2393_2394','2394_1090','2394_2395','2395_2396','2396_2397','2397_2398','2398_2399','2399_2400','24_25','2400_2401','2401_2402','2402_2403','2403_1096','2404_2405','2405_2406','2406_2408','2408_2409','2409_2410','2410_2411','2411_2412','2412_2438','2413_2414','2414_2415','2415_2416','2416_2417','2417_2418','2418_2419','2419_2452','242_243','2420_2421','2421_2423','2423_2424','2424_2447','2425_2426','2426_2427','2427_2428','2428_2429','2429_2430','243_244','243_254','2430_2371','2431_2432','2432_2433','2433_2327','2433_2434','2434_2435','2435_2436','2436_2437','2437_1117','2437_2438','2438_2439','2439_2440','244_245','2440_2441','2441_2442','2442_2443','2443_2444','2444_1344','2445_2457','2446_2459','2447_5133','2448_5026','245_246','2450_2490','2452_2453','2453_2454','2454_2455','2455_2456','2456_2445','2457_4392','2458_2446','2459_2460','246_248','2460_2461','2461_2462','2462_2463','2463_2431','2465_2466','2466_2467','2467_2494','2468_7319','247_1240','2471_2472','2472_2473','2473_2474','2474_2475','2475_2476','2476_2477','2477_2478','2478_1170','2479_2480','248_249','2480_2481','2481_2482','2482_2483','2483_2484','2484_2485','2485_2486','2486_2487','2487_2488','2488_2489','2489_2448','249_250','2490_2491','2491_2492','2492_2493','2493_2465','2494_2495','2495_2496','2496_2497','2497_2471','2498_2499','2499_2500','2499_7216','25_27','250_251','2500_2501','2501_7623','2508_2278','251_252','2510_7242','2515_2516','2516_2517','2517_2518','2518_4749','252_255','2522_2524','2523_2532','2524_2523','253_5033','2532_4877','2535_2536','2536_2616','2538_2606','2539_2540','254_1246','254_244','2540_2542','2540_4882','2542_3004','2547_2548','2548_2600','2548_946','255_234','2550_2516','2550_2551','2551_2553','2553_2554','2554_2425','2554_2555','2555_2556','2556_2557','2557_4436','2558_2559','2558_6170','2559_2560','2560_2561','2561_2562','2562_2564','2564_4927','2566_2567','2567_2568','2568_2569','2569_2570','2569_7550','2570_2571','2570_7222','2571_2359','2573_2574','2574_2575','2574_2577','2575_4929','2577_2578','2578_2579','2579_7395','2583_2584','2584_2585','2585_2586','2586_2587','2587_2588','2588_2589','2589_2590','2590_2591','2591_2592','2592_2357','2594_2595','2595_2596','2596_2597','2596_2611','2597_2599','2599_2600','2599_4577','2600_1155','2601_4446','2602_2613','2603_2557','2604_2605','2605_2538','2606_2607','2607_2608','2608_2609','2609_2610','261_270','261_271','2610_2554','2611_2602','2613_2614','2614_2615','2615_2535','2616_2617','2617_2557','262_270','2621_2622','2622_2623','2622_2750','2623_2624','2624_4441','2626_2669','2628_2629','2629_6095','263_271','263_277','263_4508','263_4725','263_6071','2631_2359','2632_2633','2633_2369','2634_2635','2635_2636','2636_2383','2636_5097','2637_1956','2637_2638','2637_2721','2638_2640','2638_2721','264_6059','2640_2641','2641_1948','2641_2643','2642_2643','2642_7435','2643_2644','2644_2645','2644_2709','2645_2646','2646_2647','2647_2648','2648_2649','2649_2650','265_271','265_6059','2650_2652','2652_2653','2653_2654','2654_2655','2655_2656','2656_2657','2656_2668','2657_2658','2658_2659','2659_2660','2660_2661','2661_2662','2662_2909','2663_2664','2664_2665','2665_2666','2666_2667','2667_2696','2668_2672','2669_4641','2670_2910','2672_2673','2673_2206','2674_2675','2675_2124','2676_2118','2678_2681','2679_4691','2680_2676','2680_4677','2681_2680','2682_4796','2685_2686','2686_7510','2687_2688','2688_2689','2689_2696','2696_2697','2697_2700','2697_2713','27_28','270_335','270_336','2700_2701','2701_2702','2702_2703','2703_2704','2704_2706','2706_6070','2708_5134','2709_2710','271_1278','271_340','271_404','271_4521','271_496','2710_2711','2711_2712','2712_2655','2713_2714','2714_2715','2715_2716','2716_2718','2717_2257','2718_2719','2718_7435','2719_1989','2719_7435','272_335','2720_2280','2721_7012','2722_1471','2722_4413','2725_2737','2726_2727','2727_2105','2728_2729','2729_2730','273_405','2730_6083','2733_2734','2734_5169','2735_2736','2736_2725','2737_2738','2738_2739','2739_7177','274_1505','274_2','274_7','2740_2741','2741_2742','2742_2743','2743_2745','2745_2746','2746_2747','2747_2748','2748_7062','2750_2751','2751_2752','2752_2753','2753_2754','2754_2755','2755_2756','2756_2764','2764_7178','2765_2766','2766_2767','2767_2772','2768_2769','2769_2770','277_288','277_292','277_336','2770_2771','2771_2773','2772_2768','2773_2774','2774_2777','2775_2181','2777_2775','2778_2185','278_10','278_2','278_7615','278_8','279_3','279_7','2795_756','28_29','2804_376','2809_2810','2809_408','2809_7612','281_261','281_4','281_6','281_7','2810_2811','2810_7385','2810_7387','2810_749','2810_750','2810_842','2810_843','2811_749','2811_842','2811_843','2812_2813','2813_2814','2814_852','2815_2816','2816_2817','2817_2818','2818_2819','2819_2820','2819_3012','2820_2822','2822_2823','2823_5032','2824_2825','2825_2826','2826_2827','2826_2842','2827_2828','2828_2829','2829_2830','2830_2831','2831_2832','2832_2833','2832_4349','2833_6006','2833_7375','2835_7176','2836_2837','2837_2838','2838_2839','2839_2840','284_4504','2840_2841','2840_2845','2841_2865','2842_2843','2843_2844','2844_2845','2845_2846','2846_2847','2847_4397','2848_2849','2849_2850','2850_2851','2851_2852','2852_4707','2852_4987','2853_2854','2854_2855','2855_2856','2856_2857','2857_2858','2858_2859','2859_2860','286_271','2860_2861','2861_2829','2861_2862','2862_2863','2863_2864','2864_4486','2865_2866','2866_2825','2866_2892','2866_6044','2867_1035','2868_2892','288_1171','289_496','2892_2893','2893_2894','2894_2895','2895_2896','2896_2897','2897_1062','2897_2898','2898_3562','2899_884','29_4680','2900_2901','2901_2902','2902_2903','2903_1074','2906_2907','2907_2908','2909_2906','2909_6020','2910_2911','2911_7507','2912_1358','2914_2915','2914_2989','2915_2916','2916_2917','2917_2918','2918_1300','2919_2920','292_1171','2920_2921','2921_2922','2922_2923','2923_2924','2924_2925','2925_2948','2925_7442','2927_2929','2929_2930','2930_7441','2934_2935','2935_2936','2936_2937','2937_2938','2938_7440','2940_2948','2941_2942','2942_2943','2942_7065','2943_2944','2944_2945','2945_2946','2946_2947','2947_1305','2948_2949','2949_2950','2950_2954','2954_2955','2954_7067','2955_7067','2956_2965','2957_2958','2958_4938','2961_4901','2962_2963','2963_2964','2964_4175','2965_2967','2965_2983','2966_2967','2967_2968','2968_2969','2969_2970','297_407','297_496','2970_2971','2970_4988','2971_5051','2976_2977','2977_2978','2978_2979','2979_2980','298_496','2980_2981','2981_2954','2981_2991','2983_2984','2984_2927','2984_2941','2984_7449','2985_2986','2986_2988','2988_3001','2989_2990','299_497','2990_1042','2991_2992','2992_1325','2993_2994','2994_4180','2995_3154','2996_3258','2997_4153','2998_2999','2999_3000','3_12','3_192','30_69','300_496','300_497','3000_3001','3001_3002','3002_1166','3003_2522','3004_4756','3005_4757','3006_4871','3007_6283','3008_6089','301_336','301_4495','3011_2896','3012_877','3014_3015','3015_3016','3016_3017','3017_3018','3018_3019','3019_3020','302_333','302_340','302_4495','3020_3021','3021_3022','3022_441','3023_3024','3024_3025','3025_3026','3026_3027','3027_3028','3028_3029','3029_3030','3030_3031','3031_875','3032_3033','3033_3034','3034_3036','3034_3114','3036_3037','3037_3038','3038_3039','3039_3040','3040_3041','3041_2035','3041_3042','3042_2035','3045_4150','3047_3202','3048_3049','3049_3050','3050_3051','3051_3052','3052_3053','3053_3054','3054_3055','3055_3056','3056_3057','3057_3059','3057_4830','3059_3060','3060_3061','3061_3062','3062_3063','3063_3064','3064_3065','3065_3066','3066_3067','3067_3202','3068_3069','3069_3070','307_308','3070_2041','3071_3072','3072_3073','3073_3074','3074_3075','3075_3076','3076_3077','3077_3079','3077_3084','3079_3080','308_309','3080_3081','3080_3082','3081_3084','3081_7188','3082_3084','3082_3114','3082_7660','3084_469','3085_3034','3088_3704','3089_3090','309_2383','309_7412','3090_3706','3091_3092','3092_3093','3093_3535','31_32','3112_3924','3114_3115','3114_4353','3115_3116','3116_3117','3116_6085','3117_3118','3118_3119','3119_3120','312_1444','3120_3122','3122_3123','3123_3124','3124_3125','3125_3126','3126_3127','3127_3128','3128_3130','3129_3130','3130_3131','3131_3132','3132_3133','3133_3134','3133_5046','3134_3135','3135_3136','3135_7363','3136_3138','3138_3139','3139_4124','3140_3141','3141_3142','3142_3143','3142_3224','3143_3144','3144_3145','3144_5047','3144_7639','3145_3146','3145_5128','3146_3147','3147_3148','3148_3149','3148_5127','3149_3150','315_313','315_335','315_336','315_404','315_406','315_4495','3150_3151','3151_3153','3152_2995','3153_3152','3154_3155','3155_3156','3156_3158','3158_3159','3159_3160','3160_3161','3161_3162','3162_3163','3163_3164','3164_3079','3164_3084','3164_7188','317_312','318_312','3181_451','3184_449','319_1443','319_274','319_278','319_279','319_281','319_7491','3196_3345','32_33','320_278','320_281','3202_3203','3203_3204','3204_3205','3204_4568','3204_7641','3205_3206','3206_3207','3207_3208','3208_3209','3209_3210','3210_4981','3211_3212','3212_3213','3213_3214','3214_3215','3215_3216','3216_3217','3217_3218','3218_3219','3219_3220','3219_3537','322_323','322_7697','3220_3221','3220_5046','3221_4982','3222_3223','3223_3224','3224_3225','3225_3226','3226_3227','3227_3228','3228_3229','3229_7472','3229_7644','323_332','323_6234','3230_3231','3230_3232','3231_3246','3232_3233','3233_3234','3234_4462','3235_3236','3236_3237','3237_3238','3238_3239','3239_3240','324_1555','324_1634','324_327','324_6235','3240_3048','3240_3068','3243_3244','3244_3245','3245_3212','3246_3247','3247_7056','325_1443','3258_2060','3259_7361','326_1443','327_6016','327_7113','328_1443','3280_3282','3282_3479','3283_3284','3284_3310','329_1443','33_7270','3310_3481','3317_3318','3318_3320','3320_2064','3321_2066','3321_4727','3322_3563','3328_3329','3329_3330','333_747','3330_3331','3331_3351','3332_3333','3333_3334','3334_3352','3335_3336','3336_3337','3337_3338','3338_3339','3339_3340','334_406','334_7581','3340_4324','3344_7421','3345_4727','335_1278','335_404','335_4521','3351_3332','3352_3335','3355_3356','3356_1441','336_1278','336_1279','336_7581','3365_3366','3366_3367','3367_3886','3368_4626','3368_7137','3370_7168','3370_7231','3372_3373','3373_3374','3374_2233','3376_3377','3377_3378','3378_3379','3379_4669','3383_3384','3384_3385','3385_3386','3386_3387','3387_4559','3389_3390','3390_6119','3392_3393','3393_3394','3394_3396','3396_3397','3397_3398','3398_3399','3399_6187','340_350','3400_3401','3401_3402','3402_3403','3403_3404','3404_3405','3405_3406','3406_3410','3407_3408','3408_3409','3409_4558','3410_3407','3411_3412','3412_3413','3413_3414','3414_3415','3415_3416','3416_3417','3417_3418','3418_3419','3419_2374','3419_3420','3419_4762','342_1279','342_4495','342_4521','3420_3421','3421_3422','3422_3423','3422_4671','3423_2156','3423_4409','3425_2176','3426_3434','3427_3428','3428_3429','3429_3432','3432_3426','3432_3434','3434_3435','3434_3453','3435_3453','3436_7432','3437_3439','3437_3440','3438_5022','3439_3440','3440_3441','3440_4557','3441_4560','3443_3444','3444_3445','3444_3447','3445_3446','3446_3447','3447_3448','3447_3454','3448_3449','3449_3436','345_318','345_320','3453_3437','3454_3455','3455_3456','3455_5022','3456_3457','3457_3438','346_317','3461_4532','3461_4537','3465_4673','3466_3467','3467_3469','3469_3470','3470_3471','3471_3472','3472_3473','3473_3474','3473_5015','3474_3475','3475_3476','3476_3477','3477_3478','3478_3280','3479_4086','3481_3482','3482_3483','3483_3484','3484_3485','3485_3486','3486_3487','3487_3488','3487_7415','3488_3489','3489_3492','3491_6007','3492_2835','3493_3494','3494_3495','3495_3496','3496_3497','3497_3498','3498_3499','3499_3500','35_36','350_2809','350_351','3500_3501','3501_3502','3502_3503','3503_3504','3504_3505','3505_3506','3506_3507','3507_3508','3508_3510','351_352','3510_3512','3512_3514','3514_3515','3515_3516','3516_3517','3517_3518','3518_3519','3519_3521','352_353','3521_3523','3523_3524','3524_3525','3525_3527','3527_3528','3528_3529','3529_3532','353_354','3532_3533','3533_3491','3535_3536','3536_3537','3537_3538','3538_7556','3539_3540','354_355','3540_3541','3541_3542','3542_3543','3542_3544','3543_3141','3544_3552','355_356','3552_3553','3553_3554','3554_3555','3555_3556','3556_3557','3557_7317','356_357','3562_2899','3563_444','3565_3825','3566_3567','3567_7292','3569_3571','357_390','3571_3801','3572_3701','3573_3662','3574_3641','3575_3576','3576_3577','3576_3648','3577_3578','3578_3579','3579_3580','3580_3640','3583_3584','3584_3585','3585_3587','3585_3625','3587_3588','3588_3589','3588_3590','3589_3591','3589_3605','3590_3589','3591_3592','3592_3593','3593_3594','3594_3595','3595_3596','3596_3597','3597_3598','3598_4465','3599_3600','3599_3610','36_37','3600_3601','3601_3602','3602_3604','3604_3616','3605_3606','3606_3607','3607_3608','3607_6051','3608_3609','3609_3598','3610_3611','3611_6135','3612_3613','3613_3615','3615_3616','3616_3618','3618_3619','3619_3620','3619_6087','3620_3621','3620_3622','3621_3622','3622_3623','3623_4387','3624_3587','3625_3626','3626_3627','3627_3628','3628_6052','3629_3630','3630_3631','3631_3632','3632_3633','3633_3634','3634_3635','3635_3636','3636_905','3639_3644','3640_3574','3641_3642','3642_3643','3643_3583','3644_3645','3645_3646','3646_3647','3647_3661','3648_3649','3649_3650','3650_7328','3651_3652','3652_3653','3653_3654','3653_6055','3654_5084','3655_5086','3656_3657','3656_7194','3657_3658','3658_3659','3658_7329','3659_3660','3660_3647','3661_3573','3662_1212','3663_7347','3664_3813','3665_7400','3669_3671','3669_7349','3669_7399','3669_7611','3670_3671','3671_3672','3672_3674','3674_3675','3675_3676','3675_5030','3676_3677','3676_5073','3677_3678','3678_3679','3679_3711','3679_5075','3682_3864','3686_5077','3689_3690','3689_3691','3690_5079','3690_6117','3691_3572','3691_3692','3692_3694','3694_3695','3695_3696','3695_7115','3696_3697','3697_3698','3698_3699','3699_3885','37_38','3701_3702','3702_3088','3704_3705','3705_733','3706_3707','3707_3708','3708_3709','3709_3710','371_391','3710_4925','3711_3712','3711_7189','3712_3713','3712_4916','3712_6359','3713_3714','3714_3715','3714_3719','3715_3755','3715_5175','3718_3720','3719_3718','372_373','3720_7691','3721_3722','3722_5069','3723_3724','3724_3725','3725_3726','3725_3738','3726_7039','3728_3729','3729_3730','373_374','3730_3732','3732_3733','3733_3734','3734_3735','3735_7039','3736_3738','3738_7170','374_375','3740_3741','3741_5165','3743_3780','3744_3715','3744_3745','3745_3746','3746_3749','3748_6050','3749_3750','375_2804','3750_3752','3750_6252','3751_3689','3751_7114','3752_3751','3755_3756','3756_3757','3757_3758','3758_3759','3759_3760','376_377','3760_3761','3761_3762','3762_3763','3763_3764','3764_3765','3765_3766','3766_3767','3767_3768','3768_3770','3769_3771','377_378','3770_3769','3771_3772','3772_3773','3773_3774','3773_5176','3774_7172','3775_3777','3777_3778','3778_7690','378_380','3780_3744','3781_3782','3782_3783','3783_3784','3784_3785','3785_3786','3786_3787','3787_3788','3788_3789','3789_3790','3790_3791','3791_3792','3792_3793','3793_3794','3794_3795','3795_3797','3797_3798','3798_3799','3798_6076','3799_6076','38_39','380_2087','3801_3802','3802_3805','3805_7175','3806_3807','3807_3808','3808_3809','3809_7174','381_382','3811_3812','3812_3664','3813_3814','3814_3815','3815_3816','3816_3566','3818_3819','3819_3820','382_4451','3820_3822','3822_3823','3823_3824','3824_3565','3825_3826','3826_3827','3827_3828','3828_3829','3829_3831','383_384','3831_3832','3832_3833','3833_3834','3834_3835','3835_3836','3836_3837','3837_3838','3838_3840','384_385','3840_3842','3842_3843','3843_3844','3844_3845','3845_3846','3846_3848','3848_3849','3849_3850','385_387','3850_3851','3851_3852','3852_3853','3853_3854','3854_3855','3855_3856','3856_3857','3857_3858','3858_3859','3859_3860','3860_3861','3861_3862','3862_3863','3863_3748','3864_3865','3865_4910','3865_7133','3867_7135','387_388','388_389','3881_5096','3884_3686','3885_1631','3885_3663','3886_3887','3886_3888','3887_3888','3888_3890','3888_7186','389_393','3890_3891','3891_3949','3891_7187','3892_3893','3893_3895','3895_3896','3896_3897','3897_3898','3898_3899','3899_3900','39_116','39_40','390_372','3900_3901','3901_3902','3902_3904','3904_3905','3905_3906','3905_7085','3906_3907','3907_3908','3907_3909','3908_3909','3909_5118','391_392','3910_3911','3911_3912','3912_3913','3913_3914','3914_3925','3914_7075','3917_3918','3918_3926','3918_5114','3919_3920','392_395','3920_3921','3921_3922','3922_3923','3923_3112','3923_7079','3924_3927','3925_3008','3926_7074','3927_3928','3928_3929','3929_7108','393_371','3935_3936','3936_3938','3936_7110','3938_3939','3939_3940','3940_3941','3941_3943','3943_3944','3944_3945','3945_3946','3946_4001','3947_3892','3949_3950','395_396','3950_2229','3950_3372','3952_2190','3954_3955','3955_3956','3956_3957','3957_3958','3957_3993','3958_3959','3959_3960','3959_7234','396_397','3960_3961','3960_6156','3961_3962','3962_3963','3963_3964','3964_3965','3965_3968','3966_5113','3968_3966','3968_7771','397_398','398_399','3980_6089','3981_3982','3982_3983','3982_7704','3983_3985','3985_3986','3986_3987','3986_7313','3987_3989','3989_3990','399_400','3990_3991','3991_3998','3993_3994','3994_3995','3995_3996','3996_3997','3996_3998','3997_3998','3998_3999','3999_4000','4_194','4_4962','4_7555','40_146','40_41','400_279','400_281','400_303','400_319','400_346','400_4522','400_7392','400_7564','400_7581','4000_4001','4001_3949','4001_7193','4002_5112','4003_4404','4004_4005','4005_4006','4006_4007','4007_4008','4008_4010','4009_4011','4010_4009','4011_5110','4012_4013','4013_4014','4014_4098','4015_4017','4017_4018','4018_4019','4019_4020','4020_4100','4021_4022','4022_4023','4023_4024','4024_4027','4027_7250','404_747','405_408','4051_4052','4052_4054','4054_4055','4055_4057','4056_4133','4057_4101','4058_4059','4059_4060','406_2810','406_4530','406_747','4060_4061','4061_4062','4062_4063','4063_4064','4064_4056','4065_4066','4066_4067','4067_4068','4068_4069','4069_4070','407_2498','4070_4071','4071_4072','4072_4073','4073_2583','408_409','4086_4087','4087_4088','4088_4089','4089_4090','409_410','409_411','4090_4091','4091_6097','4093_4094','4094_4096','4096_4114','4098_4015','41_42','410_411','4100_4021','4101_4102','4102_4058','411_412','411_414','4114_4115','4115_4116','4116_4117','4117_4118','4118_4119','4119_4120','412_413','4120_4121','4121_4122','4122_4123','4123_3283','4124_4125','4125_4126','4126_4127','4127_4128','4128_4129','4129_4130','413_414','4130_4131','4131_7294','4132_4134','4132_4171','4133_4065','4134_4135','4135_4136','4136_4207','414_415','414_416','4148_4149','4149_3045','415_416','4150_4151','4151_2997','4151_4156','4153_4154','4154_4416','4156_4168','416_417','4168_4170','417_418','417_485','4170_4132','4170_4153','4171_4172','4172_4173','4173_2962','4175_5091','4177_4178','4178_4179','4179_2993','418_419','4180_4181','4181_4182','4182_4151','419_420','42_43','420_2088','420_421','4201_4202','4202_4203','4203_4204','4203_5090','4204_4205','4205_4206','4206_3140','4207_4208','4208_4209','4209_4210','421_422','4210_4211','4211_4212','4211_4215','4212_4213','4213_5139','4214_4215','4215_4216','4216_4217','4216_4222','4217_4218','4218_4219','4219_4220','422_423','4220_4221','4221_4222','4222_4224','4224_4225','4225_4226','4225_7462','4226_4227','4227_4228','4228_4229','4229_4230','423_424','4230_4231','4230_4248','4231_7352','424_425','4246_4248','4248_4249','4248_4252','4249_4250','425_426','4250_4251','4251_4252','4252_4253','4252_4305','4252_6137','4253_4254','4253_4268','4253_4305','4254_6137','4256_4257','4257_4258','4258_4259','4259_7272','426_427','4260_4261','4260_7296','4261_7271','4262_4263','4263_4264','4264_7296','4265_4266','4266_4253','4266_4267','4266_4307','4267_4268','4268_4269','4268_4273','4269_4270','427_428','4270_4271','4271_4273','4273_4274','4273_4277','4274_4275','4275_4276','4276_4277','4277_4278','4277_7462','4278_4279','4279_4280','428_429','4280_4281','4281_4282','4282_4283','4283_4284','4284_4285','4284_4295','4285_4286','4286_4287','4287_4288','4288_4295','429_3032','429_6334','4295_4296','4296_4297','4296_4301','4297_4299','4299_4300','43_44','4300_4301','4301_4302','4302_4591','4302_5135','4303_4304','4304_4148','4305_4306','4306_4307','4306_4309','4307_4308','4308_4253','4308_4267','4309_4254','4314_1687','4315_4316','4316_4317','4317_7013','4319_1476','4320_1476','4321_7342','4323_1845','4324_4769','4325_3328','4330_3679','4331_4339','4333_4334','4334_4712','4336_1051','4339_7616','4339_7692','4342_4348','4344_2349','4347_2349','4347_7181','4348_4646','4348_7122','4349_3184','435_2011','435_7353','4350_5143','4353_4355','4354_3196','4355_4354','4362_2959','4362_7025','4363_1598','4364_1579','4369_6219','437_459','4371_832','4372_179','4376_4377','4377_4890','4379_1963','4379_1968','4380_1172','4380_496','4382_1185','4382_1196','4383_4699','4384_518','4384_519','4385_1201','4386_653','4387_3634','4388_4700','4392_2458','4397_2848','4399_4400','44_7603','4400_1464','4401_2202','4401_2213','4404_4405','4405_3991','4406_1984','4407_2637','4407_4320','4408_1473','4409_1972','4409_2173','4409_4550','441_442','4410_2678','4411_2239','4411_4715','4413_1476','4413_1480','4413_7453','4415_301','4415_302','4416_4201','442_443','443_444','4432_119','4433_3752','4434_2379','4435_2594','4435_2604','4435_2632','4436_5008','4438_3432','444_445','4440_2628','4441_2626','4445_2352','4446_2603','445_4779','4451_383','4455_381','4456_1284','4458_1658','4460_453','4462_3235','4464_1869','4465_944','4466_633','4468_2283','4469_2262','447_5045','4472_4386','4475_3023','4477_3321','4479_4376','448_3181','4481_4485','4485_822','4486_2865','4489_4747','449_450','4492_1861','4493_4362','4495_1278','4495_1279','4495_2809','4495_4522','4495_4720','4495_746','4495_7581','4498_1660','4499_4934','45_46','450_3181','4500_4481','4501_4935','4502_4936','4504_7028','4508_7402','4509_499','4509_7708','451_447','4513_272','4513_286','4514_4515','4515_1995','4516_1369','4518_509','4520_4825','4521_1283','4521_1934','4521_1935','4522_1934','4523_4524','4524_4322','4525_4526','4526_1700','4528_1072','453_4477','453_454','4530_748','4532_4534','4533_4844','4534_4535','4535_2169','4536_6003','4537_2176','4538_7396','4539_1606','454_4475','4542_1531','4543_1560','4545_4552','4546_1559','4548_1514','4548_980','4549_1964','4550_4551','4551_1965','4552_4554','4554_7016','4554_7229','4554_7230','4555_3418','4555_4576','4557_3441','4558_3411','4559_3389','4560_5126','4561_4629','4562_7386','4563_1218','4564_3443','4564_3444','4565_4566','4566_4567','4567_2031','4567_3238','4568_4569','4569_2023','4569_4570','4570_2056','4571_2013','4571_2014','4573_4893','4574_4575','4575_1680','4576_3418','4577_4578','4578_1100','4583_1518','4583_1576','4588_4589','4589_7364','459_2080','4590_7366','4591_4590','4592_980','4593_4594','4593_4595','4593_7516','4594_4595','4595_4596','4596_4563','4599_2217','4599_4601','4599_7184','46_47','460_2011','4601_5058','4602_4603','4603_4604','4604_4605','4605_4562','4606_7142','4607_4608','4607_7140','4608_4609','4609_4610','461_2068','4610_4611','4611_4612','4612_4613','4613_3370','4616_2227','4617_4555','4619_4561','4620_7563','4623_4619','4624_4625','4625_4628','4626_4627','4627_4624','4628_4623','4629_4630','4630_4631','4631_4632','4632_5164','4633_2234','4636_2016','4640_4347','4640_7214','4641_4642','4642_2355','4643_4644','4644_4645','4645_4440','4646_4647','4647_2346','4647_4435','4662_2335','4663_4664','4664_2160','4665_4666','4666_4667','4667_2162','4667_4668','4668_3376','4668_4677','4669_2146','4669_2165','4670_2165','4670_3420','4671_4672','4672_3465','4673_2172','4674_4663','4677_2136','4680_31','4680_90','4683_107','4688_4689','4689_4410','469_470','4690_4411','4691_2682','4691_4796','4692_1245','4693_1254','4694_1249','4695_1243','4696_4697','4697_1250','4698_4383','4699_4388','47_48','470_471','4700_1266','4705_475','4707_4708','4708_4336','4709_1037','471_472','4710_1046','4712_2853','4713_4714','4714_4960','4717_2499','472_473','4720_1443','4724_263','4725_273','4725_334','4725_4495','4725_6059','4725_6071','4726_7555','4727_4728','4727_768','4728_437','4728_461','473_4705','4731_2023','4732_3243','4742_4743','4743_4744','4743_4745','4744_4745','4745_4989','4746_1845','4747_1845','4747_1882','4749_4750','475_476','4750_3006','4750_4751','4751_4754','4754_4755','4755_3003','4756_3005','4757_4758','4758_6343','4759_4760','476_477','4760_2547','4762_4763','4763_6245','4765_4766','4766_4767','4767_4768','4768_1845','4768_4325','4769_4770','477_478','4770_4765','4779_7057','478_479','478_7053','4782_1782','4788_6273','479_480','4793_6068','4795_2686','4795_4688','4796_4690','4797_7473','4798_2674','4798_4715','4799_2205','48_49','480_481','4800_5157','4804_738','4806_5156','481_482','482_483','4825_7233','483_485','4830_7667','4843_4905','4844_4177','4847_4848','4848_4350','485_487','485_488','4854_1438','4855_1439','4856_4857','4857_307','4861_4862','4863_4864','4864_4865','4865_4577','4869_4870','487_488','4870_3007','4871_4872','4872_4873','4873_4874','4874_2532','4877_4878','4878_7180','488_489','488_491','4880_2539','4880_2606','4882_4883','4883_4884','4884_4885','4885_6343','4886_4887','4887_1150','4888_4798','489_490','4890_4369','4891_7458','4893_4894','4894_4895','4895_7217','4897_4898','4898_4574','4899_4900','49_51','490_491','4900_4746','4901_4902','4902_1672','4903_1883','4903_7219','4905_4906','4906_4907','4907_4908','4908_4909','4909_3884','4909_4921','491_492','491_493','4910_4911','4911_4912','4912_4913','4913_4914','4914_4915','4915_4957','4915_7654','4916_4917','4917_4918','4918_4919','4919_4920','492_493','4920_4910','4921_4922','4922_4923','4923_4924','4924_4958','4925_3679','4925_4926','4926_3678','4927_4928','4928_2352','4929_7460','493_494','4930_4931','4931_7459','4932_4933','4933_2590','4934_4500','4935_4502','4936_79','4937_1547','4938_1549','494_495','495_400','495_496','495_7587','495_7588','4952_768','4953_768','4957_7133','4958_3752','496_497','496_515','496_522','496_523','4960_4932','4962_264','4962_4725','4962_6059','497_515','4974_4976','4976_7126','4978_4979','4979_4986','4980_2289','4981_3211','4984_448','4985_6219','4986_7107','4987_2978','4988_2853','4989_1845','4989_4990','499_500','4990_4991','4991_4992','4992_1719','4994_4995','4995_4996','4996_1826','4997_4998','4998_4999','4999_1833','500_501','5008_4640','501_1484','5013_5014','5014_7291','5015_7290','5022_3461','5022_6002','5023_5024','5024_5161','5025_4854','5026_2450','5028_3696','5028_7115','5029_5028','5030_5031','5031_6033','5032_2824','5032_2867','5033_7130','5044_3751','5044_4433','5045_4460','5046_7639','5047_3146','5047_3224','5051_5052','5052_2975','5053_1633','5056_2239','5058_4602','5060_5061','5061_1231','5062_691','5064_7173','5069_3723','5071_5072','5072_4321','5072_7342','5073_3679','5073_5074','5074_6054','5075_5076','5076_3682','5077_3694','5077_5078','5078_3689','5079_3694','5081_5082','5082_3639','5084_6054','5086_3656','509_510','5090_4204','5091_5092','5092_7574','5096_7145','5097_2383','5099_2383','51_52','510_511','511_513','5110_4012','5111_4051','5112_4003','5113_3918','5118_3910','5119_5120','512_513','5120_5121','5121_5122','5122_3461','5123_5124','5124_5125','5125_6001','5127_5128','5128_2060','5128_2996','5129_5130','513_4513','513_7402','5130_1135','5131_5132','5132_1331','5133_2596','5134_2718','5135_4303','5136_4262','5139_4214','5140_1500','5140_620','5142_7567','5143_2837','5144_7027','5144_7702','5145_276','5147_68','5148_1401','5149_1419','515_516','5150_2327','5154_4804','5155_4806','5156_7552','5157_5154','516_4384','5160_5155','5161_5162','5162_2571','5162_4880','5164_4633','5165_3743','5165_7169','5169_2735','5171_2976','5174_6078','5175_3755','5176_3774','518_2255','519_521','5192_342','5192_350','52_261','52_262','52_263','52_265','52_277','52_4725','521_522','522_1741','522_523','523_1740','523_4382','523_525','523_669','525_526','526_527','527_528','528_529','529_530','53_54','530_531','531_709','532_533','532_649','533_534','534_535','535_536','536_537','537_538','538_539','539_540','54_55','540_541','541_542','542_543','543_544','544_545','544_546','545_546','546_547','546_910','546_914','547_548','548_549','549_550','549_677','55_56','550_551','551_552','552_553','553_554','554_555','555_557','557_558','558_559','559_560','56_58','560_562','562_563','563_564','564_565','565_566','566_567','566_707','567_568','568_569','569_570','57_58','570_571','571_572','572_573','573_574','574_575','575_576','576_577','577_579','579_580','58_4499','58_59','580_581','581_582','582_583','583_584','584_585','585_586','586_587','587_588','588_589','588_590','589_590','59_60','590_1729','590_591','591_592','592_593','593_594','594_595','595_596','596_597','597_598','598_599','599_600','6_194','60_1802','60_820','600_601','600_654','6001_4438','6002_5119','6003_5123','6004_3','6004_7','6005_1546','6005_7072','6006_3493','6007_2836','601_602','6010_3656','6011_2133','6013_6247','6014_2158','6014_6242','6015_5029','6016_6017','6017_7192','602_603','6020_6023','6021_7507','6023_6026','6024_6021','6025_6024','6026_6027','6027_6029','6028_6025','603_604','6030_6028','6033_3089','604_605','6041_2826','6042_1022','6048_5064','605_606','6050_3749','6050_7398','6051_6053','6052_3613','6053_7327','6054_4330','6055_1073','6057_2183','6059_334','6059_404','606_607','6068_1584','607_608','6070_2708','6074_748','6074_793','6075_2631','6076_6077','6077_3569','6078_7112','6079_6315','608_609','6080_1585','6080_983','6083_2733','6085_2026','6085_2053','6086_3161','6086_7419','6087_3620','6088_1898','6089_3981','609_610','6094_2102','6095_7062','6097_4093','610_611','6107_6108','6107_6109','6108_6109','6109_6110','611_612','6110_7020','6110_7029','6111_6112','6112_6113','6113_1898','6113_4323','6115_1205','6115_4593','6117_3655','6117_5079','6119_3392','612_613','6122_35','6124_4004','6125_7092','613_614','6130_1262','6132_2551','6132_4861','6135_3612','6137_4256','6137_4956','614_4415','614_615','614_675','6140_6144','6141_6142','6142_2181','6143_6011','6144_6145','6145_6146','6146_6147','6147_6149','6149_6150','615_616','6150_6143','6152_6153','6153_6243','6154_6155','6155_1987','6155_6141','6156_6165','6157_6158','6158_3964','6158_7314','6159_6160','616_617','6160_6161','6161_3991','6162_7022','6163_1095','6164_6159','6165_6157','617_618','6170_5023','6170_7183','6171_4974','6172_4493','618_619','6182_322','6185_6186','6186_6216','6187_6188','6188_6215','619_675','620_1172','620_621','620_7568','620_7569','621_289','621_291','621_293','621_4495','6215_6184','6216_3400','6219_6229','6228_4788','6228_7111','6229_6348','6229_6364','6229_7131','6230_6228','6234_332','6235_327','6238_6182','6239_104','624_625','6241_105','6242_6152','6243_6154','6244_2134','6245_2156','6245_4671','6245_6013','6247_2679','625_626','6251_5140','6251_7706','6252_301','6252_302','6253_7191','626_627','627_628','6273_136','6273_4891','6276_6362','6277_6278','6278_6279','6279_6280','628_629','6282_6335','6283_6282','6285_6286','6286_6287','6287_6288','6288_6342','629_630','6290_7120','6298_6336','63_64','630_4466','6310_6311','6311_4495','6311_7399','6312_7197','6315_6316','6316_7236','6318_6319','6319_7246','6320_4594','6324_6325','6325_6341','6326_4886','633_634','6334_7660','6335_6326','6336_7069','6339_6324','634_1005','6341_6277','6342_6298','6343_4759','6348_1182','635_636','6359_7189','636_637','6361_6367','6362_7276','6363_6361','6364_1558','6365_6276','6366_6230','6367_4985','637_638','638_639','639_640','64_65','640_641','641_642','642_643','643_644','644_645','645_1796','649_650','65_66','650_651','651_4472','653_722','654_655','655_656','656_657','657_658','658_659','659_604','66_4498','664_614','664_665','665_666','666_667','667_668','668_614','669_670','670_671','671_672','672_4382','675_1184','675_1500','675_303','675_4380','675_4415','675_620','675_621','677_678','678_679','679_680','68_30','680_681','681_682','682_683','683_684','684_685','685_686','686_688','688_689','689_690','69_7686','69_828','690_5062','691_692','692_693','692_694','692_695','693_585','694_695','695_696','696_697','697_698','698_699','699_700','7_11','7_194','700_701','7009_1604','701_702','7010_7703','7011_2171','7011_7160','7012_2201','7012_2644','7013_6162','7014_4315','7015_4617','7016_4619','7017_7018','7018_7030','702_703','7020_7029','7021_6111','7021_7163','7022_1554','7023_7014','7025_4464','7027_4538','7028_5147','7029_7038','703_704','7030_6111','7030_7021','7031_4897','7038_7011','7039_7204','704_705','7043_1424','7044_993','7047_7161','7048_7699','7049_2236','705_706','7050_7185','7053_2088','7056_4731','7057_4847','7057_5142','706_567','706_707','7062_6075','7063_1137','7064_7066','7065_7063','7066_2924','7067_2956','7067_2984','7068_2966','7069_7443','707_708','7070_7445','7073_7097','7074_3919','7075_3925','7078_1449','7079_7080','708_731','7080_7081','7081_7565','7082_7083','7083_7084','7084_7108','7085_7086','7086_7087','7087_7088','7088_7089','7089_7091','709_532','7091_3910','7092_110','7097_7098','7098_7099','7099_7100','71_72','710_711','7100_7101','7101_7505','7102_7103','7103_7104','7104_7105','7105_7106','7106_6005','7107_7102','7108_3935','711_712','7110_3938','7111_6365','7112_972','7113_127','7114_7116','7115_6310','7116_3694','7117_7118','7118_6289','7119_7127','712_713','7120_7119','7121_7183','7122_7121','7123_4347','7124_7343','7125_7344','7126_7124','7127_7143','7128_7117','713_714','7130_1237','7132_1558','7132_6230','7133_7134','7134_3867','7135_7136','7136_3874','7137_7139','7138_3370','7139_4623','714_715','7140_7141','7141_7138','7142_7015','7143_3368','7143_7168','7144_3881','7145_3884','7145_4843','7149_7277','715_716','7153_6056','7154_7153','7156_7020','7158_7048','7159_7017','7159_7388','716_717','7160_7700','7161_7162','7163_1893','7166_1847','7167_1808','7168_4616','7169_3743','717_718','7170_3740','7172_3775','7173_3811','7175_3806','7176_2836','7177_2740','7178_2765','718_719','7180_7123','7180_7331','7183_4880','7184_2225','7184_7050','7185_7186','7186_7187','7187_3947','7187_3954','7187_4001','7188_3082','719_720','7191_6238','7192_4683','7193_7049','7194_7196','7195_3653','7196_6310','7197_7195','72_5144','720_680','720_721','7201_2138','7204_3728','7204_3736','7207_7288','7208_7249','721_680','721_693','7214_4347','7216_2501','7217_7227','7218_7031','7219_7218','722_723','7222_6170','7224_1581','7227_6172','7229_3418','7229_4617','723_724','7230_4617','7231_7128','7233_4573','7234_4002','7236_7245','7237_7241','7238_7224','7239_4401','724_725','7240_1891','7241_7247','7242_7238','7243_7126','7245_6317','7246_6320','7247_7243','7248_7207','7248_7254','7249_7253','725_726','7250_7248','7250_7252','7251_4051','7252_7248','7253_7251','7254_7257','7255_7249','7256_7255','7257_7258','7258_7260','7259_7256','726_727','7260_7263','7261_7259','7262_7261','7263_7264','7264_7268','7265_7262','7266_7265','7268_7270','727_728','7270_6122','7270_7266','7271_5136','7272_4956','7276_7149','7277_6363','7278_7289','728_729','7280_7281','7281_7284','7282_7278','7283_7282','7284_7287','7286_7283','7287_7208','7288_7286','7289_7280','729_730','7290_5016','7291_3485','7292_3818','7293_1335','7294_4132','7294_4156','7296_4265','7296_4266','73_74','730_5160','731_710','7313_6164','7314_3964','7317_3224','7318_4978','7318_4980','7319_7342','7320_1486','7325_3317','7325_4984','7326_7418','7327_3629','7328_3651','7329_3659','733_6015','7330_2051','7331_2559','7333_783','7334_1838','7340_783','7342_7125','7343_4978','7343_4980','7343_7318','7344_2510','7344_7334','7347_3669','7348_3669','7348_3671','7349_1631','7350_1558','7350_6230','7352_4246','7353_4571','7353_7584','7354_1975','7358_1231','7358_1235','7358_5174','7361_7362','7362_3129','7362_3134','7363_4588','7364_7365','7365_4210','7366_7367','7367_7368','7368_3143','7370_7339','7374_1845','7375_3466','7379_4903','738_739','7380_1676','7381_4742','7382_7381','7386_4607','7388_7017','7389_4324','7389_7025','739_740','7391_493','7392_1443','7392_312','7395_6124','7396_5145','7397_5140','7397_6252','7398_6252','7398_7397','7399_7398','7399_7611','74_4371','740_741','7400_3669','7401_3665','7402_4380','7402_4717','741_742','7412_7413','7413_2383','7414_1396','7415_7416','7416_7325','7417_3471','7418_7417','7419_7420','742_743','7420_3345','7421_7422','7422_6085','7423_1566','7424_7423','7425_7424','7426_7427','7427_7428','7428_138','743_744','7430_177','7431_175','7432_3454','7433_4677','7435_2722','7435_4413','7438_1410','7439_7454','744_745','7440_2940','7441_2933','7442_7446','7443_7444','7444_2941','7445_6339','7446_7070','7449_2941','745_600','7453_1478','7453_1480','7453_2001','7454_1380','7455_7438','7458_4479','7458_7425','746_844','7460_4930','7461_4230','7461_4281','7462_7461','7469_1724','747_748','747_842','747_843','7471_1533','7472_3230','7473_2679','7475_6107','748_7390','748_749','749_494','749_750','749_751','75_4501','750_751','7505_5071','7505_7678','7507_2663','751_752','7510_2207','7510_4799','7513_807','7514_2279','7516_6079','752_7339','752_753','753_7339','753_7370','753_754','754_755','755_756','7550_2570','7551_4800','7552_924','7555_4962','7556_3539','7559_7073','7559_7107','756_757','7560_7679','7563_3368','7563_7231','7564_340','7564_4521','7565_7082','7567_7326','757_758','7571_207','7574_4177','7577_1353','7578_7582','7579_1285','758_759','758_760','7581_1283','7582_297','7582_299','7582_319','7582_340','7582_407','7584_3344','7585_7587','7586_7587','7587_303','7587_328','7587_345','7587_407','7587_496','7587_7588','7588_281','7588_325','7588_328','7588_329','7588_4717','759_760','7591_496','7592_3815','760_2007','760_761','760_762','7601_2508','7602_22','7602_85','7603_45','761_762','7611_7398','7612_2811','7616_5081','762_763','7622_297','7622_334','7623_2508','7623_3676','7623_3715','7623_3719','7623_5030','7623_6312','7623_7401','763_2007','763_4952','763_4953','763_767','7639_7640','7640_3134','7640_5047','7641_7642','7642_7643','7643_3214','7644_7645','7645_7646','7646_3238','7654_4843','7660_3114','7661_1521','7667_3091','767_768','7671_513','7672_826','7673_7674','7674_7675','7675_7676','7676_7677','7677_7559','7678_7560','7679_7680','768_2085','768_769','768_770','768_772','7680_7681','7681_7738','7683_1582','769_770','7690_3781','7691_3721','7692_6010','7698_7113','7699_7159','77_78','770_771','770_773','7700_7047','7701_7010','7702_7009','7703_5147','7704_3983','7705_6251','7706_7707','7707_512','7708_7709','7709_7710','771_772','7710_7711','7711_7705','772_417','772_773','772_775','773_774','7738_7683','774_775','774_906','775_776','775_906','776_777','777_779','777_906','7771_3966','779_780','78_79','780_781','781_782','782_783','783_784','783_785','784_785','785_786','786_791','786_792','786_793','79_80','790_7585','791_7585','792_320','792_7586','793_274','793_320','793_495','793_7586','795_796','796_56','796_57','796_797','797_798','798_799','799_800','8_12','80_82','800_801','801_802','802_1654','802_803','803_804','804_7513','807_808','808_809','809_810','81_817','810_811','811_812','812_813','813_814','814_81','817_818','818_819','819_261','819_263','819_264','819_265','819_4725','82_817','82_83','820_821','821_63','822_823','823_824','824_825','825_826','826_827','827_71','828_829','829_830','83_84','830_831','831_75','832_833','833_77','834_835','835_836','836_837','837_1506','838_839','839_840','84_51','840_841','841_49','842_1014','842_845','842_846','843_1013','843_845','844_846','845_846','846_7340','846_847','846_848','847_848','848_2795','848_758','848_759','848_849','849_850','85_203','85_86','850_851','851_852','852_853','853_854','854_855','855_2815','855_856','856_857','857_858','858_859','859_860','86_87','860_861','861_862','862_3014','87_88','875_876','876_3011','876_878','877_876','878_879','879_880','88_89','880_881','881_882','882_883','883_884','884_885','885_886','886_887','887_2900','887_903','89_27','893_894','894_1067','895_1025','896_897','897_893','90_91','903_904','904_906','905_4331','906_907','906_909','907_908','908_909','909_786','909_790','909_792','91_129','91_92','910_911','911_912','912_913','913_914','914_938','915_916','916_917','917_918','918_919','919_7551','92_93','924_925','925_927','927_938','928_588','928_929','929_930','93_94','930_931','931_587','938_940','94_102','94_126','940_941','941_942','942_943','943_3599','944_945','945_947','946_1155','947_948','948_949','949_928','950_915','956_957','957_958','958_959','959_960','960_961','961_962','962_635','969_1176','972_1625','980_981','981_1518','981_1576','983_984','984_985','985_1529','987_4937','992_7044','993_5060']

Loop through all segments and print metrics:

In [ ]:
file = open("/data_analytics/reports/segment_report_dec.csv", "w", encoding="utf8")
file.write("segment,count,mean,std,mae,rmse,r2\n")
for segment in segments:
    df_test_temp = df_test.loc[df_test.segment == segment]
    if df_test_temp.shape[0] != 0:
        count = df_test_temp.shape[0]
        mean = df_test_temp['segment_mean'].iloc[0]
        std = df_test_temp['segment_std'].iloc[0]
        X_test_temp = df_test_temp[['actualtime_arr_stop_first','segment_mean','weekday', 'segment_std','peak', 'school_hol', 'rain', 'temp', 'hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23']]
        X_test_temp = scaler.transform(X_test_temp)
        y_test_temp = df_test_temp.time_diff
        gtb_predicted_temp = (gtb_model.predict(X_test_temp))
        mae = metrics.mean_absolute_error(y_test_temp, gtb_predicted_temp)
        rmse = math.sqrt(metrics.mean_squared_error(y_test_temp, gtb_predicted_temp))
        r2 = metrics.r2_score(y_test_temp, gtb_predicted_temp)
        file.write(str(segment) + ',' + str(count) + ',' + str(mean) + ',' + str(std) + ',' + str(mae) + ',' + str(rmse) + ',' + str(r2) + '\n')
file.close()

## 19.3 Test by Time of Day (Weekdays)

In [ ]:
# make a list of all hours
hours = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,1]

In [ ]:
file = open("/data_analytics/reports/weekday_report_dec.csv", "w", encoding="utf8")
file.write("hour,count,mean,std,mae,rmse,r2\n")
for hour in hours:
    df_test_temp = df_test.loc[(df_test.hour == hour) & (df_test.weekday == 1)]
    if df_test_temp.shape[0] != 0:
        count = df_test_temp.shape[0]
        mean = df_test_temp['time_diff'].mean().round()
        std = df_test_temp['time_diff'].std().round()
        X_test_temp = df_test_temp[['actualtime_arr_stop_first','segment_mean','weekday', 'segment_std','peak', 'school_hol', 'rain', 'temp', 'hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23']]
        X_test_temp = scaler.transform(X_test_temp)
        y_test_temp = df_test_temp.time_diff
        gtb_predicted_temp = (gtb_model.predict(X_test_temp))
        mae = metrics.mean_absolute_error(y_test_temp, gtb_predicted_temp)
        rmse = math.sqrt(metrics.mean_squared_error(y_test_temp, gtb_predicted_temp))
        r2 = metrics.r2_score(y_test_temp, gtb_predicted_temp)
        file.write(str(hour) + ',' + str(count) + ',' + str(mean) + ',' + str(std) + ',' + str(mae) + ',' + str(rmse) + ',' + str(r2) + '\n')
file.close()

## 19.4 Test by Time of Day (Weekends)

In [ ]:
# make a list of all hours
hours = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,1]

In [ ]:
file = open("/data_analytics/reports/weekend_report_dec.csv", "w", encoding="utf8")
file.write("hour,count,mean,std,mae,rmse,r2\n")
for hour in hours:
    df_test_temp = df_test.loc[(df_test.hour == hour) & (df_test.weekday == 0)]
    if df_test_temp.shape[0] != 0:
        count = df_test_temp.shape[0]
        mean = df_test_temp['time_diff'].mean().round()
        std = df_test_temp['time_diff'].std().round()
        X_test_temp = df_test_temp[['actualtime_arr_stop_first','segment_mean','weekday', 'segment_std','peak', 'school_hol', 'rain', 'temp', 'hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23']]
        X_test_temp = scaler.transform(X_test_temp)
        y_test_temp = df_test_temp.time_diff
        gtb_predicted_temp = (gtb_model.predict(X_test_temp))
        mae = metrics.mean_absolute_error(y_test_temp, gtb_predicted_temp)
        rmse = math.sqrt(metrics.mean_squared_error(y_test_temp, gtb_predicted_temp))
        r2 = metrics.r2_score(y_test_temp, gtb_predicted_temp)
        file.write(str(hour) + ',' + str(count) + ',' + str(mean) + ',' + str(std) + ',' + str(mae) + ',' + str(rmse) + ',' + str(r2) + '\n')
file.close()

## 19.5 Test by Trip

In [ ]:
# sort the test day by day and tripid
df_test = df_test.sort_values(by=['day', 'tripid'])

In [ ]:
file = open("/data_analytics/reports/trip_report_dec.csv", "w", encoding="utf8")
file.write("trip_id,stop_count,actual_duration,predicted_duration\n")
last_trip = 0
for index, row in df_test.iterrows():
    if row["tripid"] != last_trip:
        if last_trip != 0:
            trip_actual = 0
            for time in actual:
                trip_actual += time
            trip_predicted = 0
            for time in predictions:
                trip_predicted += time
            file.write(str(last_trip) + ',' + str(stop_count) + ',' + str(trip_actual) + ',' + str(round(trip_predicted)) + '\n')
        predictions = []
        actual = []
        stop_count = 1
        X_test_temp = row[['actualtime_arr_stop_first','segment_mean','weekday', 'segment_std','peak', 'school_hol', 'rain', 'temp', 'hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23']]
        X_test_temp = scaler.transform([X_test_temp])
        y_test_temp = row["time_diff"]
        actual.append(y_test_temp)
        gtb_predicted_temp = (gtb_model.predict(X_test_temp))
        predictions.append(gtb_predicted_temp[0])
    else:
        X_test_temp = row[['actualtime_arr_stop_first','segment_mean','weekday', 'segment_std','peak', 'school_hol', 'rain', 'temp', 'hour_0','hour_1','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23']]
        X_test_temp = scaler.transform([X_test_temp])
        y_test_temp = row["time_diff"]
        actual.append(y_test_temp)
        gtb_predicted_temp = (gtb_model.predict(X_test_temp))
        predictions.append(gtb_predicted_temp[0])
        stop_count += 1
    last_trip = row["tripid"]
file.close()

# 20. Create JSON Files with Mean and Std

In [ ]:
# make a list of all segments
segments = ['10_12','100_1111','1005_1788','1013_1015','1014_1015','1015_1016','1015_1074','1015_2812','1016_1017','1017_1018','1018_1019','1019_1020','102_6239','1020_1076','1021_6042','1022_1023','1023_1024','1024_895','1025_1026','1026_1027','1027_1028','1028_2818','1035_1314','1036_4709','1037_1038','1038_2998','104_6241','1042_1043','1043_1044','1044_1045','1045_4710','1046_1047','1047_1048','1048_1049','1049_4333','105_6253','1051_1052','1052_1053','1053_1054','1054_1036','1055_2868','1062_1063','1063_1064','1064_1065','1065_1066','1066_1067','1067_1068','1068_1069','1069_1070','107_6125','1070_1071','1071_4528','1072_1352','1072_7577','1073_3641','1074_6074','1076_1021','1076_1077','1077_1078','1077_2479','1078_1079','1079_1080','1080_1081','1081_1082','1081_2914','1082_1083','1083_1085','1085_1086','1085_1121','1086_1087','1087_1088','1088_1089','1089_1090','1090_1091','1090_2413','1091_1092','1092_1093','1093_1094','1094_1095','1095_1096','1096_1097','1096_1101','1097_1099','1099_6132','11_14','110_112','1100_100','1101_1102','1102_1103','1103_1104','1105_1107','1107_1108','1108_1109','1109_1110','1110_1111','1111_1112','1111_2323','1111_2404','1112_1113','1113_1114','1114_1115','1115_2437','1117_1118','1118_1119','1119_1120','112_113','1120_1164','1121_1122','1122_1123','1123_1124','1124_1125','1125_1127','1127_1130','113_114','1130_1140','1130_2550','1135_7064','1137_5131','114_115','1140_1141','1140_2515','1141_1142','1142_1143','1143_1144','1144_1145','1145_4869','115_37','1150_1151','1151_1152','1152_1153','1153_1154','1154_946','1155_1157','1157_1158','1158_1159','1159_1160','116_117','1160_1161','1161_1162','1162_1163','1163_1164','1163_1337','1164_1165','1165_1166','1166_1167','1167_1168','1168_1169','1169_1170','117_118','1170_1069','1171_1172','1172_1173','1172_286','1172_4509','1172_7671','1173_1174','1174_1175','1174_4725','1175_15','1176_1177','1177_1178','1178_1179','1179_5060','1179_7112','118_119','1181_7350','1182_1550','1184_497','1185_1186','1186_1187','1187_1188','1188_216','119_44','1190_1191','1191_1192','1192_1193','1193_1194','1194_1195','1195_664','1196_1197','1197_1198','1198_1199','1198_1222','1198_4782','1199_1200','12_14','1200_4385','1201_1202','1201_4695','1202_1203','1203_6115','1205_1206','1206_1207','1207_1208','1208_1209','1209_1210','1210_1211','1210_3575','1211_3575','1212_1213','1213_1214','1214_1215','1215_1216','1216_1217','1217_4563','1218_1258','1218_1270','1219_1220','1220_1221','1221_664','1222_1223','1223_1224','1224_1225','1225_1226','1226_1227','1227_1228','1227_1253','1228_1244','1229_1231','1231_1232','1231_1235','1232_1233','1233_1234','1234_7358','1235_1251','1237_1238','1238_1239','1239_247','1240_1241','1241_1242','1242_1277','1243_4696','1244_1246','1244_254','1245_4693','1246_1229','1247_1248','1248_1273','1249_1247','1250_4698','1251_253','1251_7130','1253_969','1254_1255','1255_1256','1256_4694','1258_1259','1259_1260','126_6182','1260_1262','1260_6130','1262_4692','1265_6115','1266_1267','1267_1268','1268_1269','1269_1265','127_112','1270_1272','1272_1273','1273_1274','1274_1275','1275_1276','1276_1277','1277_1219','1278_1282','1278_1934','1279_1282','1279_1283','1279_1934','1280_6348','1280_6366','1281_1329','1282_4456','1283_4456','1283_7579','1284_1285','1285_1016','1285_1286','1285_1287','1286_1288','1287_1288','1288_1289','1289_1290','1289_1360','129_130','1290_1291','1291_1292','1291_2389','1292_1293','1293_1294','1294_1295','1295_1296','1296_1297','1297_1298','1298_1121','1298_1299','1299_1300','130_131','1300_1301','1301_1302','1302_1303','1303_1304','1303_5129','1304_1305','1305_1306','1305_1320','1306_1307','1307_1308','1308_1309','1309_1310','131_164','1310_1311','1311_1312','1312_1313','1313_1055','1314_1316','1315_1317','1316_1315','1317_1318','1318_1319','1319_1281','1320_1321','1321_1322','1322_1323','1323_1324','1324_7068','1325_1326','1326_1327','1327_1328','1328_1329','1329_1330','133_134','1330_1331','1330_2919','1331_1332','1332_1333','1333_1334','1334_1335','1334_2985','1334_7293','1335_1336','1336_1337','1337_1338','1338_1339','1339_1340','134_4890','1340_1341','1341_1342','1342_1343','1343_1344','1344_1345','1345_1347','1347_1348','1347_2634','1348_1349','1349_1350','1350_1352','1352_1353','1353_1354','1353_1355','1354_7578','1355_1357','1357_1358','1357_1359','1358_298','1358_319','1358_326','1358_340','1358_342','1358_4494','1358_5192','1358_7564','1358_7665','1359_274','1359_279','1359_319','1359_320','1359_4724','136_137','1360_1362','1362_1363','1363_1365','1365_1365','1365_1366','1366_1367','1367_1369','1367_7455','1369_1370','137_193','1370_1371','1371_1372','1371_1454','1372_1373','1373_1374','1374_1375','1374_1946','1375_1376','1376_1377','1377_1378','1378_1379','1378_4514','1379_1380','138_133','138_167','1380_1381','1381_1382','1382_1383','1383_1384','1384_1385','1385_1348','1386_1387','1387_1388','1388_1389','1389_3355','1396_2317','1397_1398','1398_1399','1399_5148','14_15','14_53','1401_1402','1402_1403','1403_1404','1404_1405','1405_4856','1406_1407','1407_1409','1409_2095','141_142','1410_1411','1411_1412','1412_1413','1413_1414','1414_1415','1415_1417','1417_1418','1418_5149','1419_6094','142_143','1423_1424','1424_1425','1424_2186','1424_2190','1425_1426','1426_1427','1426_1463','1427_1428','1428_1429','1429_1430','143_115','1430_1432','1432_1433','1433_1434','1434_1435','1435_7439','1436_3952','1438_4855','1439_1440','1440_1386','1441_1442','1442_7414','1443_1444','1443_1616','1443_1647','1444_1445','1445_1447','1445_1647','1445_4319','1445_4407','1445_7078','1447_2637','1449_1450','1450_1451','1451_1452','1451_2191','1452_2640','1453_1456','1454_1455','1455_1453','1456_1457','1457_1458','1458_1459','1459_1460','146_147','1460_1462','1462_5149','1463_4399','1464_1469','1465_1466','1466_1467','1467_1468','1468_1377','1469_1465','147_184','1471_1472','1472_4408','1473_1474','1474_1476','1476_7453','1478_1479','1478_315','1479_315','1479_7622','148_149','1480_1358','1480_2912','1481_7320','1482_1494','1484_1481','1485_4518','1486_1487','1487_1488','1488_1489','1489_1490','149_150','1491_1492','1492_1493','1493_1482','1494_1495','1495_1496','1496_1515','1499_1485','15_17','15_834','150_27','1500_302','1505_4509','1506_1507','1507_1508','1508_1509','1509_1510','151_153','1510_1511','1511_1512','1511_1592','1512_1538','1514_980','1515_1499','1517_1678','1518_1519','1519_1520','1520_7661','1521_4546','1523_1524','1524_1525','1525_1526','1526_6080','1528_1710','1529_4542','153_154','1531_1532','1532_1533','1533_1534','1534_1535','1535_1536','1536_1537','1537_186','1538_4543','1538_4548','1539_1540','154_155','1540_7426','1546_7072','1546_7560','1547_1548','1548_1280','1549_1523','155_156','1550_1551','1551_1552','1552_1553','1552_332','1553_7023','1554_1555','1555_1556','1556_1557','1557_1181','1558_1549','1558_2957','1558_4542','1559_1560','1559_987','156_157','156_7430','1560_1280','1566_1567','1567_173','1567_4372','157_158','1573_1574','1574_1575','1575_4583','1576_1577','1577_1578','1578_4364','1579_1580','1579_7673','158_159','1580_7237','1581_1582','1582_4793','1584_1585','1584_983','1585_1586','1586_1587','1586_1600','1587_1591','1587_4539','159_1539','1591_4542','1592_1593','1593_1594','1594_1595','1595_1596','1596_4363','1597_1599','1598_1597','1599_1573','16_73','1600_7701','1604_1573','1606_1607','1607_1608','1608_1609','1609_1610','1610_7471','1614_1615','1615_1478','1616_1617','1617_1618','1618_1619','1619_195','1620_220','1622_1623','1623_1624','1624_1625','1625_1626','1626_1627','1627_1628','1628_1629','1629_1630','1629_6234','1630_3670','1630_7347','1630_7348','1631_1632','1632_5053','1633_1634','1634_1635','1635_1636','1636_1637','1637_1638','1638_1639','1638_992','1639_1640','164_165','1640_231','1641_1642','1642_213','1643_7571','1647_1648','1647_1649','1648_1649','1649_1911','165_166','1654_1655','1655_1656','1656_1657','1657_4458','1658_1659','1659_1660','166_7458','1660_1661','1661_1662','1661_4523','1662_1664','1662_1666','1664_1665','1665_1666','1666_1667','1666_1807','1666_1808','1667_1668','1668_1669','1669_1670','167_168','1670_1671','1671_1672','1672_1673','1673_1674','1674_1675','1675_1849','1675_7380','1676_1517','1678_1679','1679_4520','168_169','1680_1681','1681_1682','1682_1683','1683_1684','1684_1685','1685_1686','1686_1687','1687_1688','1688_1689','1689_1690','1689_4899','169_141','1690_1691','1691_1692','1692_1693','1693_1694','1694_1695','1695_1696','1696_1697','1696_1700','1697_1698','1698_1699','1699_1700','17_18','1700_1701','1700_1906','1701_1702','1702_16','1702_1703','1703_1704','1703_1905','1704_1705','1705_1706','1706_1707','1707_1708','1708_1709','1709_1528','1710_1711','1711_1712','1712_1713','1713_1714','1713_1715','1714_1715','1715_1478','1715_7453','1719_1828','1724_1725','1725_1726','1726_1727','1727_1728','1728_1729','1729_1730','1729_1731','173_174','1730_1731','1731_1732','1731_1738','1732_1733','1733_1734','1734_1735','1735_1736','1736_1737','1737_1738','1738_1739','1738_614','1739_614','174_175','1740_1741','1741_1742','1741_1747','1742_1743','1743_1744','1744_1745','1745_1746','1746_1747','1747_1748','1747_1749','1748_1749','1749_1751','1749_544','175_176','1750_1769','1751_1752','1752_1753','1753_1754','1754_1759','1755_1756','1756_1757','1757_1794','1758_1750','1759_1758','176_181','1760_1761','1761_1762','1762_1763','1763_1764','1764_1765','1765_1767','1769_1770','177_178','1770_1771','1771_1760','1772_1773','1773_1774','1774_1775','1775_1776','1776_1777','1777_1778','1778_1755','178_1539','1782_1783','1783_1784','1784_1785','1785_1786','1786_624','1788_1789','1789_7199','179_7431','1791_1792','1792_1793','1793_962','1794_7469','1795_1806','1796_1797','1797_1798','1798_1799','1799_1800','18_19','1800_1801','1801_1277','1802_1795','1803_77','1804_1803','1805_1662','1805_1806','1806_1660','1807_7167','1808_1813','1808_1816','1808_2961','1808_4744','1808_7389','1808_7475','181_182','1813_1814','1814_1816','1814_1852','1816_1817','1816_6171','1817_1818','1818_1819','1819_1822','182_183','1820_2468','1822_4994','1826_1827','1827_7382','1828_1829','1829_4997','183_184','1833_1834','1834_1820','1834_1835','1835_1836','1836_1837','1837_2338','1838_2338','184_185','1840_1841','1841_1842','1841_4314','1842_7374','1844_6172','1845_1696','1845_7166','1847_1696','1849_1851','185_186','1850_1867','1851_1813','1852_1853','1853_1854','1854_1844','1858_1859','1859_1860','186_187','186_838','1860_4492','1861_1862','1862_1863','1863_1864','1864_1865','1865_1866','1866_1850','1867_4489','1869_1870','187_188','1870_1871','1871_1872','1872_1873','1873_1874','1874_1875','1875_1876','1876_1877','1877_1878','1878_1879','1879_1899','188_189','1882_7379','1883_1884','1884_1902','1887_1888','1888_1889','1889_1890','189_190','1890_7240','1891_7156','1893_1894','1894_1912','1895_1896','1896_1897','1897_6088','1898_1858','1899_1887','1899_6107','19_21','190_1614','190_191','190_192','1902_1903','1903_1904','1904_1840','1905_1804','1905_1906','1906_1907','1906_1909','1907_1908','1908_1909','1909_1713','191_819','1911_1805','1911_1913','1911_802','1912_1895','1913_1914','1914_1805','192_795','192_819','193_138','1934_1935','1934_2310','1934_2385','1935_1445','1935_1937','1937_1938','1938_1939','1939_1940','194_195','1940_1941','1941_1942','1941_4516','1942_1943','1943_1944','1944_1945','1945_1946','1946_1947','1947_1948','1947_2642','1948_1949','1949_1950','195_196','1950_1951','1951_1952','1952_1953','1953_1954','1954_1955','1955_1956','1956_1957','1956_2109','1957_1958','1958_1959','1959_1960','196_197','1960_4379','1963_4549','1964_4674','1965_1966','1966_1975','1968_1969','1969_1970','197_198','1970_1971','1971_2114','1971_2160','1972_1973','1973_1974','1974_1975','1975_1977','1975_1982','1977_1978','1978_1980','198_199','1980_1981','1981_1982','1981_4406','1982_2642','1982_4406','1982_7435','1984_1985','1985_1986','1986_1987','1987_1988','1987_1989','1988_1989','1988_2642','1989_1375','1989_1472','1989_1990','1989_1995','199_1506','199_200','1990_1992','1992_1993','1993_1994','1994_1995','1995_1996','1995_1998','1996_1997','1997_1998','1998_1999','1998_2001','1998_2002','1999_2001','2_192','2_4726','2_4962','200_201','2001_2002','2002_1358','2002_1359','2003_1358','2007_2008','2007_4571','2008_2009','2008_2071','2009_2010','201_148','201_202','2010_435','2011_2012','2012_3322','2013_2014','2014_2016','2014_4636','2015_2017','2016_2015','2016_2017','2017_2018','2017_3259','2017_7362','2018_2019','2019_2020','202_151','2020_2021','2021_2022','2022_4565','2023_2024','2024_2025','2025_6086','2026_7226','203_204','2031_2032','2032_2033','2033_2034','2034_2035','2035_2036','2036_2039','2036_2040','2039_2040','204_205','2040_2041','2040_3047','2041_2042','2041_3071','2042_2043','2043_2044','2044_2045','2045_2046','2046_4568','205_1620','205_206','205_215','2051_6086','2053_2054','2054_2055','2055_4565','2055_4732','2056_2057','2057_2058','2058_2059','2059_2060','206_207','2060_2061','2060_2062','2061_2062','2062_2063','2062_2064','2063_2064','2064_2065','2064_4727','2065_4727','2066_2067','2067_4728','2068_2069','2069_2070','207_208','2070_2084','2071_2072','2072_2073','2073_2074','2074_2075','2075_2076','2076_460','208_209','2080_2081','2081_2082','2082_2083','2083_2084','2084_768','2085_2086','2086_4455','2086_479','2087_7053','2088_2089','2089_763','2089_770','209_210','2094_1406','2095_2096','2096_2097','2097_2099','2099_2101','21_7602','21_85','210_1643','210_211','2101_2102','2102_2103','2102_2726','2103_2332','2103_7154','2105_2728','2105_6140','2109_3427','211_212','2114_2115','2115_6011','2118_2119','2119_2120','212_213','2120_2121','2121_2123','2123_2685','2124_2125','2125_2126','2126_2127','2127_2128','2128_2129','2129_4797','213_214','2133_6244','2134_2135','2134_3378','2135_2164','2136_2137','2137_2138','2137_7201','2138_2139','2139_2140','214_4432','2140_2141','2141_2142','2142_2143','2143_2145','2145_4670','2146_2147','2147_2148','2148_2149','2149_2150','215_216','2150_2151','2151_2152','2152_2153','2153_2154','2154_6245','2156_6014','2158_2159','2159_4409','216_217','2160_2161','2160_4665','2161_4665','2162_2163','2163_2168','2164_2165','2165_2166','2166_2167','2166_4545','2166_4554','2166_7229','2167_3383','2168_4536','2169_2170','217_218','217_242','2170_4672','2171_7160','2172_2159','2172_4409','2173_2174','2174_3425','2176_1982','2176_2177','2177_2178','2178_7354','218_219','2181_2182','2182_2778','2183_2184','2184_2185','2185_1423','2185_7043','2186_2187','2187_2188','2188_2189','2189_1436','219_220','2190_4434','2191_2192','2192_2193','2193_2194','2194_2195','2195_2196','2196_2197','2197_2198','2198_2200','22_23','220_1622','220_221','2200_2201','2201_4401','2201_7239','2202_2203','2203_2204','2204_4888','2205_2687','2206_4798','2207_2208','2208_2210','221_222','2210_2211','2211_2241','2213_2214','2213_4688','2214_2215','2215_2216','2215_7184','2216_4599','2217_2218','2218_2219','2219_2220','2219_3365','222_223','2220_2221','2221_2223','2223_2224','2224_3886','2225_4620','2227_7049','2229_2230','223_224','2230_2231','2231_2233','2232_2234','2233_2232','2234_2235','2235_2236','2235_2239','2236_2238','2236_2239','2238_5056','2239_2241','224_225','2241_2242','2242_2243','2242_7435','2243_2244','2244_2245','2245_2246','2246_2247','2247_2248','2248_2249','2249_2250','225_226','2250_2251','2251_1472','2255_2256','2256_2717','2257_2258','2258_2259','2259_2260','226_228','2260_2261','2261_4469','2262_2264','2264_2265','2265_2269','2269_2270','227_230','2270_2274','2274_7601','2278_2279','2279_2720','2279_6251','228_229','2280_2281','2281_2282','2282_4468','2283_616','2289_1822','229_227','23_24','230_231','231_1641','231_232','2310_2311','2311_2312','2311_5025','2312_2313','2313_2314','2314_2315','2315_2094','2317_2318','2318_2319','2319_2320','232_233','2320_2321','2321_2322','2322_6163','2323_2324','2324_2325','2325_2326','2326_5150','2327_2328','2328_2329','2329_2330','233_234','2330_2331','2331_1397','2331_1424','2332_2333','2333_2334','2334_4662','2335_2336','2336_2337','2337_2339','2338_1840','2338_1845','2339_2340','2339_2420','234_235','2340_2601','2346_4435','2349_2351','235_236','2351_2558','2351_2621','2351_4445','2352_2573','2353_2357','2353_4643','2357_2358','2357_2359','2357_2566','2358_2359','2359_2360','2359_7122','236_1190','236_237','2360_4340','2360_4348','2369_2370','237_213','2370_2371','2371_2372','2372_2373','2373_1424','2373_2377','2374_2146','2377_2378','2377_2431','2378_2183','2379_2380','2380_2382','2382_5099','2383_1479','2383_2002','2383_2003','2385_2386','2386_2387','2387_2388','2388_1290','2389_2390','2390_2391','2391_2392','2392_2393','2393_2394','2394_1090','2394_2395','2395_2396','2396_2397','2397_2398','2398_2399','2399_2400','24_25','2400_2401','2401_2402','2402_2403','2403_1096','2404_2405','2405_2406','2406_2408','2408_2409','2409_2410','2410_2411','2411_2412','2412_2438','2413_2414','2414_2415','2415_2416','2416_2417','2417_2418','2418_2419','2419_2452','242_243','2420_2421','2421_2423','2423_2424','2424_2447','2425_2426','2426_2427','2427_2428','2428_2429','2429_2430','243_244','243_254','2430_2371','2431_2432','2432_2433','2433_2327','2433_2434','2434_2435','2435_2436','2436_2437','2437_1117','2437_2438','2438_2439','2439_2440','244_245','2440_2441','2441_2442','2442_2443','2443_2444','2444_1344','2445_2457','2446_2459','2447_5133','2448_5026','245_246','2450_2490','2452_2453','2453_2454','2454_2455','2455_2456','2456_2445','2457_4392','2458_2446','2459_2460','246_248','2460_2461','2461_2462','2462_2463','2463_2431','2465_2466','2466_2467','2467_2494','2468_7319','247_1240','2471_2472','2472_2473','2473_2474','2474_2475','2475_2476','2476_2477','2477_2478','2478_1170','2479_2480','248_249','2480_2481','2481_2482','2482_2483','2483_2484','2484_2485','2485_2486','2486_2487','2487_2488','2488_2489','2489_2448','249_250','2490_2491','2491_2492','2492_2493','2493_2465','2494_2495','2495_2496','2496_2497','2497_2471','2498_2499','2499_2500','2499_7216','25_27','250_251','2500_2501','2501_7623','2508_2278','251_252','2510_7242','2515_2516','2516_2517','2517_2518','2518_4749','252_255','2522_2524','2523_2532','2524_2523','253_5033','2532_4877','2535_2536','2536_2616','2538_2606','2539_2540','254_1246','254_244','2540_2542','2540_4882','2542_3004','2547_2548','2548_2600','2548_946','255_234','2550_2516','2550_2551','2551_2553','2553_2554','2554_2425','2554_2555','2555_2556','2556_2557','2557_4436','2558_2559','2558_6170','2559_2560','2560_2561','2561_2562','2562_2564','2564_4927','2566_2567','2567_2568','2568_2569','2569_2570','2569_7550','2570_2571','2570_7222','2571_2359','2573_2574','2574_2575','2574_2577','2575_4929','2577_2578','2578_2579','2579_7395','2583_2584','2584_2585','2585_2586','2586_2587','2587_2588','2588_2589','2589_2590','2590_2591','2591_2592','2592_2357','2594_2595','2595_2596','2596_2597','2596_2611','2597_2599','2599_2600','2599_4577','2600_1155','2601_4446','2602_2613','2603_2557','2604_2605','2605_2538','2606_2607','2607_2608','2608_2609','2609_2610','261_270','261_271','2610_2554','2611_2602','2613_2614','2614_2615','2615_2535','2616_2617','2617_2557','262_270','2621_2622','2622_2623','2622_2750','2623_2624','2624_4441','2626_2669','2628_2629','2629_6095','263_271','263_277','263_4508','263_4725','263_6071','2631_2359','2632_2633','2633_2369','2634_2635','2635_2636','2636_2383','2636_5097','2637_1956','2637_2638','2637_2721','2638_2640','2638_2721','264_6059','2640_2641','2641_1948','2641_2643','2642_2643','2642_7435','2643_2644','2644_2645','2644_2709','2645_2646','2646_2647','2647_2648','2648_2649','2649_2650','265_271','265_6059','2650_2652','2652_2653','2653_2654','2654_2655','2655_2656','2656_2657','2656_2668','2657_2658','2658_2659','2659_2660','2660_2661','2661_2662','2662_2909','2663_2664','2664_2665','2665_2666','2666_2667','2667_2696','2668_2672','2669_4641','2670_2910','2672_2673','2673_2206','2674_2675','2675_2124','2676_2118','2678_2681','2679_4691','2680_2676','2680_4677','2681_2680','2682_4796','2685_2686','2686_7510','2687_2688','2688_2689','2689_2696','2696_2697','2697_2700','2697_2713','27_28','270_335','270_336','2700_2701','2701_2702','2702_2703','2703_2704','2704_2706','2706_6070','2708_5134','2709_2710','271_1278','271_340','271_404','271_4521','271_496','2710_2711','2711_2712','2712_2655','2713_2714','2714_2715','2715_2716','2716_2718','2717_2257','2718_2719','2718_7435','2719_1989','2719_7435','272_335','2720_2280','2721_7012','2722_1471','2722_4413','2725_2737','2726_2727','2727_2105','2728_2729','2729_2730','273_405','2730_6083','2733_2734','2734_5169','2735_2736','2736_2725','2737_2738','2738_2739','2739_7177','274_1505','274_2','274_7','2740_2741','2741_2742','2742_2743','2743_2745','2745_2746','2746_2747','2747_2748','2748_7062','2750_2751','2751_2752','2752_2753','2753_2754','2754_2755','2755_2756','2756_2764','2764_7178','2765_2766','2766_2767','2767_2772','2768_2769','2769_2770','277_288','277_292','277_336','2770_2771','2771_2773','2772_2768','2773_2774','2774_2777','2775_2181','2777_2775','2778_2185','278_10','278_2','278_7615','278_8','279_3','279_7','2795_756','28_29','2804_376','2809_2810','2809_408','2809_7612','281_261','281_4','281_6','281_7','2810_2811','2810_7385','2810_7387','2810_749','2810_750','2810_842','2810_843','2811_749','2811_842','2811_843','2812_2813','2813_2814','2814_852','2815_2816','2816_2817','2817_2818','2818_2819','2819_2820','2819_3012','2820_2822','2822_2823','2823_5032','2824_2825','2825_2826','2826_2827','2826_2842','2827_2828','2828_2829','2829_2830','2830_2831','2831_2832','2832_2833','2832_4349','2833_6006','2833_7375','2835_7176','2836_2837','2837_2838','2838_2839','2839_2840','284_4504','2840_2841','2840_2845','2841_2865','2842_2843','2843_2844','2844_2845','2845_2846','2846_2847','2847_4397','2848_2849','2849_2850','2850_2851','2851_2852','2852_4707','2852_4987','2853_2854','2854_2855','2855_2856','2856_2857','2857_2858','2858_2859','2859_2860','286_271','2860_2861','2861_2829','2861_2862','2862_2863','2863_2864','2864_4486','2865_2866','2866_2825','2866_2892','2866_6044','2867_1035','2868_2892','288_1171','289_496','2892_2893','2893_2894','2894_2895','2895_2896','2896_2897','2897_1062','2897_2898','2898_3562','2899_884','29_4680','2900_2901','2901_2902','2902_2903','2903_1074','2906_2907','2907_2908','2909_2906','2909_6020','2910_2911','2911_7507','2912_1358','2914_2915','2914_2989','2915_2916','2916_2917','2917_2918','2918_1300','2919_2920','292_1171','2920_2921','2921_2922','2922_2923','2923_2924','2924_2925','2925_2948','2925_7442','2927_2929','2929_2930','2930_7441','2934_2935','2935_2936','2936_2937','2937_2938','2938_7440','2940_2948','2941_2942','2942_2943','2942_7065','2943_2944','2944_2945','2945_2946','2946_2947','2947_1305','2948_2949','2949_2950','2950_2954','2954_2955','2954_7067','2955_7067','2956_2965','2957_2958','2958_4938','2961_4901','2962_2963','2963_2964','2964_4175','2965_2967','2965_2983','2966_2967','2967_2968','2968_2969','2969_2970','297_407','297_496','2970_2971','2970_4988','2971_5051','2976_2977','2977_2978','2978_2979','2979_2980','298_496','2980_2981','2981_2954','2981_2991','2983_2984','2984_2927','2984_2941','2984_7449','2985_2986','2986_2988','2988_3001','2989_2990','299_497','2990_1042','2991_2992','2992_1325','2993_2994','2994_4180','2995_3154','2996_3258','2997_4153','2998_2999','2999_3000','3_12','3_192','30_69','300_496','300_497','3000_3001','3001_3002','3002_1166','3003_2522','3004_4756','3005_4757','3006_4871','3007_6283','3008_6089','301_336','301_4495','3011_2896','3012_877','3014_3015','3015_3016','3016_3017','3017_3018','3018_3019','3019_3020','302_333','302_340','302_4495','3020_3021','3021_3022','3022_441','3023_3024','3024_3025','3025_3026','3026_3027','3027_3028','3028_3029','3029_3030','3030_3031','3031_875','3032_3033','3033_3034','3034_3036','3034_3114','3036_3037','3037_3038','3038_3039','3039_3040','3040_3041','3041_2035','3041_3042','3042_2035','3045_4150','3047_3202','3048_3049','3049_3050','3050_3051','3051_3052','3052_3053','3053_3054','3054_3055','3055_3056','3056_3057','3057_3059','3057_4830','3059_3060','3060_3061','3061_3062','3062_3063','3063_3064','3064_3065','3065_3066','3066_3067','3067_3202','3068_3069','3069_3070','307_308','3070_2041','3071_3072','3072_3073','3073_3074','3074_3075','3075_3076','3076_3077','3077_3079','3077_3084','3079_3080','308_309','3080_3081','3080_3082','3081_3084','3081_7188','3082_3084','3082_3114','3082_7660','3084_469','3085_3034','3088_3704','3089_3090','309_2383','309_7412','3090_3706','3091_3092','3092_3093','3093_3535','31_32','3112_3924','3114_3115','3114_4353','3115_3116','3116_3117','3116_6085','3117_3118','3118_3119','3119_3120','312_1444','3120_3122','3122_3123','3123_3124','3124_3125','3125_3126','3126_3127','3127_3128','3128_3130','3129_3130','3130_3131','3131_3132','3132_3133','3133_3134','3133_5046','3134_3135','3135_3136','3135_7363','3136_3138','3138_3139','3139_4124','3140_3141','3141_3142','3142_3143','3142_3224','3143_3144','3144_3145','3144_5047','3144_7639','3145_3146','3145_5128','3146_3147','3147_3148','3148_3149','3148_5127','3149_3150','315_313','315_335','315_336','315_404','315_406','315_4495','3150_3151','3151_3153','3152_2995','3153_3152','3154_3155','3155_3156','3156_3158','3158_3159','3159_3160','3160_3161','3161_3162','3162_3163','3163_3164','3164_3079','3164_3084','3164_7188','317_312','318_312','3181_451','3184_449','319_1443','319_274','319_278','319_279','319_281','319_7491','3196_3345','32_33','320_278','320_281','3202_3203','3203_3204','3204_3205','3204_4568','3204_7641','3205_3206','3206_3207','3207_3208','3208_3209','3209_3210','3210_4981','3211_3212','3212_3213','3213_3214','3214_3215','3215_3216','3216_3217','3217_3218','3218_3219','3219_3220','3219_3537','322_323','322_7697','3220_3221','3220_5046','3221_4982','3222_3223','3223_3224','3224_3225','3225_3226','3226_3227','3227_3228','3228_3229','3229_7472','3229_7644','323_332','323_6234','3230_3231','3230_3232','3231_3246','3232_3233','3233_3234','3234_4462','3235_3236','3236_3237','3237_3238','3238_3239','3239_3240','324_1555','324_1634','324_327','324_6235','3240_3048','3240_3068','3243_3244','3244_3245','3245_3212','3246_3247','3247_7056','325_1443','3258_2060','3259_7361','326_1443','327_6016','327_7113','328_1443','3280_3282','3282_3479','3283_3284','3284_3310','329_1443','33_7270','3310_3481','3317_3318','3318_3320','3320_2064','3321_2066','3321_4727','3322_3563','3328_3329','3329_3330','333_747','3330_3331','3331_3351','3332_3333','3333_3334','3334_3352','3335_3336','3336_3337','3337_3338','3338_3339','3339_3340','334_406','334_7581','3340_4324','3344_7421','3345_4727','335_1278','335_404','335_4521','3351_3332','3352_3335','3355_3356','3356_1441','336_1278','336_1279','336_7581','3365_3366','3366_3367','3367_3886','3368_4626','3368_7137','3370_7168','3370_7231','3372_3373','3373_3374','3374_2233','3376_3377','3377_3378','3378_3379','3379_4669','3383_3384','3384_3385','3385_3386','3386_3387','3387_4559','3389_3390','3390_6119','3392_3393','3393_3394','3394_3396','3396_3397','3397_3398','3398_3399','3399_6187','340_350','3400_3401','3401_3402','3402_3403','3403_3404','3404_3405','3405_3406','3406_3410','3407_3408','3408_3409','3409_4558','3410_3407','3411_3412','3412_3413','3413_3414','3414_3415','3415_3416','3416_3417','3417_3418','3418_3419','3419_2374','3419_3420','3419_4762','342_1279','342_4495','342_4521','3420_3421','3421_3422','3422_3423','3422_4671','3423_2156','3423_4409','3425_2176','3426_3434','3427_3428','3428_3429','3429_3432','3432_3426','3432_3434','3434_3435','3434_3453','3435_3453','3436_7432','3437_3439','3437_3440','3438_5022','3439_3440','3440_3441','3440_4557','3441_4560','3443_3444','3444_3445','3444_3447','3445_3446','3446_3447','3447_3448','3447_3454','3448_3449','3449_3436','345_318','345_320','3453_3437','3454_3455','3455_3456','3455_5022','3456_3457','3457_3438','346_317','3461_4532','3461_4537','3465_4673','3466_3467','3467_3469','3469_3470','3470_3471','3471_3472','3472_3473','3473_3474','3473_5015','3474_3475','3475_3476','3476_3477','3477_3478','3478_3280','3479_4086','3481_3482','3482_3483','3483_3484','3484_3485','3485_3486','3486_3487','3487_3488','3487_7415','3488_3489','3489_3492','3491_6007','3492_2835','3493_3494','3494_3495','3495_3496','3496_3497','3497_3498','3498_3499','3499_3500','35_36','350_2809','350_351','3500_3501','3501_3502','3502_3503','3503_3504','3504_3505','3505_3506','3506_3507','3507_3508','3508_3510','351_352','3510_3512','3512_3514','3514_3515','3515_3516','3516_3517','3517_3518','3518_3519','3519_3521','352_353','3521_3523','3523_3524','3524_3525','3525_3527','3527_3528','3528_3529','3529_3532','353_354','3532_3533','3533_3491','3535_3536','3536_3537','3537_3538','3538_7556','3539_3540','354_355','3540_3541','3541_3542','3542_3543','3542_3544','3543_3141','3544_3552','355_356','3552_3553','3553_3554','3554_3555','3555_3556','3556_3557','3557_7317','356_357','3562_2899','3563_444','3565_3825','3566_3567','3567_7292','3569_3571','357_390','3571_3801','3572_3701','3573_3662','3574_3641','3575_3576','3576_3577','3576_3648','3577_3578','3578_3579','3579_3580','3580_3640','3583_3584','3584_3585','3585_3587','3585_3625','3587_3588','3588_3589','3588_3590','3589_3591','3589_3605','3590_3589','3591_3592','3592_3593','3593_3594','3594_3595','3595_3596','3596_3597','3597_3598','3598_4465','3599_3600','3599_3610','36_37','3600_3601','3601_3602','3602_3604','3604_3616','3605_3606','3606_3607','3607_3608','3607_6051','3608_3609','3609_3598','3610_3611','3611_6135','3612_3613','3613_3615','3615_3616','3616_3618','3618_3619','3619_3620','3619_6087','3620_3621','3620_3622','3621_3622','3622_3623','3623_4387','3624_3587','3625_3626','3626_3627','3627_3628','3628_6052','3629_3630','3630_3631','3631_3632','3632_3633','3633_3634','3634_3635','3635_3636','3636_905','3639_3644','3640_3574','3641_3642','3642_3643','3643_3583','3644_3645','3645_3646','3646_3647','3647_3661','3648_3649','3649_3650','3650_7328','3651_3652','3652_3653','3653_3654','3653_6055','3654_5084','3655_5086','3656_3657','3656_7194','3657_3658','3658_3659','3658_7329','3659_3660','3660_3647','3661_3573','3662_1212','3663_7347','3664_3813','3665_7400','3669_3671','3669_7349','3669_7399','3669_7611','3670_3671','3671_3672','3672_3674','3674_3675','3675_3676','3675_5030','3676_3677','3676_5073','3677_3678','3678_3679','3679_3711','3679_5075','3682_3864','3686_5077','3689_3690','3689_3691','3690_5079','3690_6117','3691_3572','3691_3692','3692_3694','3694_3695','3695_3696','3695_7115','3696_3697','3697_3698','3698_3699','3699_3885','37_38','3701_3702','3702_3088','3704_3705','3705_733','3706_3707','3707_3708','3708_3709','3709_3710','371_391','3710_4925','3711_3712','3711_7189','3712_3713','3712_4916','3712_6359','3713_3714','3714_3715','3714_3719','3715_3755','3715_5175','3718_3720','3719_3718','372_373','3720_7691','3721_3722','3722_5069','3723_3724','3724_3725','3725_3726','3725_3738','3726_7039','3728_3729','3729_3730','373_374','3730_3732','3732_3733','3733_3734','3734_3735','3735_7039','3736_3738','3738_7170','374_375','3740_3741','3741_5165','3743_3780','3744_3715','3744_3745','3745_3746','3746_3749','3748_6050','3749_3750','375_2804','3750_3752','3750_6252','3751_3689','3751_7114','3752_3751','3755_3756','3756_3757','3757_3758','3758_3759','3759_3760','376_377','3760_3761','3761_3762','3762_3763','3763_3764','3764_3765','3765_3766','3766_3767','3767_3768','3768_3770','3769_3771','377_378','3770_3769','3771_3772','3772_3773','3773_3774','3773_5176','3774_7172','3775_3777','3777_3778','3778_7690','378_380','3780_3744','3781_3782','3782_3783','3783_3784','3784_3785','3785_3786','3786_3787','3787_3788','3788_3789','3789_3790','3790_3791','3791_3792','3792_3793','3793_3794','3794_3795','3795_3797','3797_3798','3798_3799','3798_6076','3799_6076','38_39','380_2087','3801_3802','3802_3805','3805_7175','3806_3807','3807_3808','3808_3809','3809_7174','381_382','3811_3812','3812_3664','3813_3814','3814_3815','3815_3816','3816_3566','3818_3819','3819_3820','382_4451','3820_3822','3822_3823','3823_3824','3824_3565','3825_3826','3826_3827','3827_3828','3828_3829','3829_3831','383_384','3831_3832','3832_3833','3833_3834','3834_3835','3835_3836','3836_3837','3837_3838','3838_3840','384_385','3840_3842','3842_3843','3843_3844','3844_3845','3845_3846','3846_3848','3848_3849','3849_3850','385_387','3850_3851','3851_3852','3852_3853','3853_3854','3854_3855','3855_3856','3856_3857','3857_3858','3858_3859','3859_3860','3860_3861','3861_3862','3862_3863','3863_3748','3864_3865','3865_4910','3865_7133','3867_7135','387_388','388_389','3881_5096','3884_3686','3885_1631','3885_3663','3886_3887','3886_3888','3887_3888','3888_3890','3888_7186','389_393','3890_3891','3891_3949','3891_7187','3892_3893','3893_3895','3895_3896','3896_3897','3897_3898','3898_3899','3899_3900','39_116','39_40','390_372','3900_3901','3901_3902','3902_3904','3904_3905','3905_3906','3905_7085','3906_3907','3907_3908','3907_3909','3908_3909','3909_5118','391_392','3910_3911','3911_3912','3912_3913','3913_3914','3914_3925','3914_7075','3917_3918','3918_3926','3918_5114','3919_3920','392_395','3920_3921','3921_3922','3922_3923','3923_3112','3923_7079','3924_3927','3925_3008','3926_7074','3927_3928','3928_3929','3929_7108','393_371','3935_3936','3936_3938','3936_7110','3938_3939','3939_3940','3940_3941','3941_3943','3943_3944','3944_3945','3945_3946','3946_4001','3947_3892','3949_3950','395_396','3950_2229','3950_3372','3952_2190','3954_3955','3955_3956','3956_3957','3957_3958','3957_3993','3958_3959','3959_3960','3959_7234','396_397','3960_3961','3960_6156','3961_3962','3962_3963','3963_3964','3964_3965','3965_3968','3966_5113','3968_3966','3968_7771','397_398','398_399','3980_6089','3981_3982','3982_3983','3982_7704','3983_3985','3985_3986','3986_3987','3986_7313','3987_3989','3989_3990','399_400','3990_3991','3991_3998','3993_3994','3994_3995','3995_3996','3996_3997','3996_3998','3997_3998','3998_3999','3999_4000','4_194','4_4962','4_7555','40_146','40_41','400_279','400_281','400_303','400_319','400_346','400_4522','400_7392','400_7564','400_7581','4000_4001','4001_3949','4001_7193','4002_5112','4003_4404','4004_4005','4005_4006','4006_4007','4007_4008','4008_4010','4009_4011','4010_4009','4011_5110','4012_4013','4013_4014','4014_4098','4015_4017','4017_4018','4018_4019','4019_4020','4020_4100','4021_4022','4022_4023','4023_4024','4024_4027','4027_7250','404_747','405_408','4051_4052','4052_4054','4054_4055','4055_4057','4056_4133','4057_4101','4058_4059','4059_4060','406_2810','406_4530','406_747','4060_4061','4061_4062','4062_4063','4063_4064','4064_4056','4065_4066','4066_4067','4067_4068','4068_4069','4069_4070','407_2498','4070_4071','4071_4072','4072_4073','4073_2583','408_409','4086_4087','4087_4088','4088_4089','4089_4090','409_410','409_411','4090_4091','4091_6097','4093_4094','4094_4096','4096_4114','4098_4015','41_42','410_411','4100_4021','4101_4102','4102_4058','411_412','411_414','4114_4115','4115_4116','4116_4117','4117_4118','4118_4119','4119_4120','412_413','4120_4121','4121_4122','4122_4123','4123_3283','4124_4125','4125_4126','4126_4127','4127_4128','4128_4129','4129_4130','413_414','4130_4131','4131_7294','4132_4134','4132_4171','4133_4065','4134_4135','4135_4136','4136_4207','414_415','414_416','4148_4149','4149_3045','415_416','4150_4151','4151_2997','4151_4156','4153_4154','4154_4416','4156_4168','416_417','4168_4170','417_418','417_485','4170_4132','4170_4153','4171_4172','4172_4173','4173_2962','4175_5091','4177_4178','4178_4179','4179_2993','418_419','4180_4181','4181_4182','4182_4151','419_420','42_43','420_2088','420_421','4201_4202','4202_4203','4203_4204','4203_5090','4204_4205','4205_4206','4206_3140','4207_4208','4208_4209','4209_4210','421_422','4210_4211','4211_4212','4211_4215','4212_4213','4213_5139','4214_4215','4215_4216','4216_4217','4216_4222','4217_4218','4218_4219','4219_4220','422_423','4220_4221','4221_4222','4222_4224','4224_4225','4225_4226','4225_7462','4226_4227','4227_4228','4228_4229','4229_4230','423_424','4230_4231','4230_4248','4231_7352','424_425','4246_4248','4248_4249','4248_4252','4249_4250','425_426','4250_4251','4251_4252','4252_4253','4252_4305','4252_6137','4253_4254','4253_4268','4253_4305','4254_6137','4256_4257','4257_4258','4258_4259','4259_7272','426_427','4260_4261','4260_7296','4261_7271','4262_4263','4263_4264','4264_7296','4265_4266','4266_4253','4266_4267','4266_4307','4267_4268','4268_4269','4268_4273','4269_4270','427_428','4270_4271','4271_4273','4273_4274','4273_4277','4274_4275','4275_4276','4276_4277','4277_4278','4277_7462','4278_4279','4279_4280','428_429','4280_4281','4281_4282','4282_4283','4283_4284','4284_4285','4284_4295','4285_4286','4286_4287','4287_4288','4288_4295','429_3032','429_6334','4295_4296','4296_4297','4296_4301','4297_4299','4299_4300','43_44','4300_4301','4301_4302','4302_4591','4302_5135','4303_4304','4304_4148','4305_4306','4306_4307','4306_4309','4307_4308','4308_4253','4308_4267','4309_4254','4314_1687','4315_4316','4316_4317','4317_7013','4319_1476','4320_1476','4321_7342','4323_1845','4324_4769','4325_3328','4330_3679','4331_4339','4333_4334','4334_4712','4336_1051','4339_7616','4339_7692','4342_4348','4344_2349','4347_2349','4347_7181','4348_4646','4348_7122','4349_3184','435_2011','435_7353','4350_5143','4353_4355','4354_3196','4355_4354','4362_2959','4362_7025','4363_1598','4364_1579','4369_6219','437_459','4371_832','4372_179','4376_4377','4377_4890','4379_1963','4379_1968','4380_1172','4380_496','4382_1185','4382_1196','4383_4699','4384_518','4384_519','4385_1201','4386_653','4387_3634','4388_4700','4392_2458','4397_2848','4399_4400','44_7603','4400_1464','4401_2202','4401_2213','4404_4405','4405_3991','4406_1984','4407_2637','4407_4320','4408_1473','4409_1972','4409_2173','4409_4550','441_442','4410_2678','4411_2239','4411_4715','4413_1476','4413_1480','4413_7453','4415_301','4415_302','4416_4201','442_443','443_444','4432_119','4433_3752','4434_2379','4435_2594','4435_2604','4435_2632','4436_5008','4438_3432','444_445','4440_2628','4441_2626','4445_2352','4446_2603','445_4779','4451_383','4455_381','4456_1284','4458_1658','4460_453','4462_3235','4464_1869','4465_944','4466_633','4468_2283','4469_2262','447_5045','4472_4386','4475_3023','4477_3321','4479_4376','448_3181','4481_4485','4485_822','4486_2865','4489_4747','449_450','4492_1861','4493_4362','4495_1278','4495_1279','4495_2809','4495_4522','4495_4720','4495_746','4495_7581','4498_1660','4499_4934','45_46','450_3181','4500_4481','4501_4935','4502_4936','4504_7028','4508_7402','4509_499','4509_7708','451_447','4513_272','4513_286','4514_4515','4515_1995','4516_1369','4518_509','4520_4825','4521_1283','4521_1934','4521_1935','4522_1934','4523_4524','4524_4322','4525_4526','4526_1700','4528_1072','453_4477','453_454','4530_748','4532_4534','4533_4844','4534_4535','4535_2169','4536_6003','4537_2176','4538_7396','4539_1606','454_4475','4542_1531','4543_1560','4545_4552','4546_1559','4548_1514','4548_980','4549_1964','4550_4551','4551_1965','4552_4554','4554_7016','4554_7229','4554_7230','4555_3418','4555_4576','4557_3441','4558_3411','4559_3389','4560_5126','4561_4629','4562_7386','4563_1218','4564_3443','4564_3444','4565_4566','4566_4567','4567_2031','4567_3238','4568_4569','4569_2023','4569_4570','4570_2056','4571_2013','4571_2014','4573_4893','4574_4575','4575_1680','4576_3418','4577_4578','4578_1100','4583_1518','4583_1576','4588_4589','4589_7364','459_2080','4590_7366','4591_4590','4592_980','4593_4594','4593_4595','4593_7516','4594_4595','4595_4596','4596_4563','4599_2217','4599_4601','4599_7184','46_47','460_2011','4601_5058','4602_4603','4603_4604','4604_4605','4605_4562','4606_7142','4607_4608','4607_7140','4608_4609','4609_4610','461_2068','4610_4611','4611_4612','4612_4613','4613_3370','4616_2227','4617_4555','4619_4561','4620_7563','4623_4619','4624_4625','4625_4628','4626_4627','4627_4624','4628_4623','4629_4630','4630_4631','4631_4632','4632_5164','4633_2234','4636_2016','4640_4347','4640_7214','4641_4642','4642_2355','4643_4644','4644_4645','4645_4440','4646_4647','4647_2346','4647_4435','4662_2335','4663_4664','4664_2160','4665_4666','4666_4667','4667_2162','4667_4668','4668_3376','4668_4677','4669_2146','4669_2165','4670_2165','4670_3420','4671_4672','4672_3465','4673_2172','4674_4663','4677_2136','4680_31','4680_90','4683_107','4688_4689','4689_4410','469_470','4690_4411','4691_2682','4691_4796','4692_1245','4693_1254','4694_1249','4695_1243','4696_4697','4697_1250','4698_4383','4699_4388','47_48','470_471','4700_1266','4705_475','4707_4708','4708_4336','4709_1037','471_472','4710_1046','4712_2853','4713_4714','4714_4960','4717_2499','472_473','4720_1443','4724_263','4725_273','4725_334','4725_4495','4725_6059','4725_6071','4726_7555','4727_4728','4727_768','4728_437','4728_461','473_4705','4731_2023','4732_3243','4742_4743','4743_4744','4743_4745','4744_4745','4745_4989','4746_1845','4747_1845','4747_1882','4749_4750','475_476','4750_3006','4750_4751','4751_4754','4754_4755','4755_3003','4756_3005','4757_4758','4758_6343','4759_4760','476_477','4760_2547','4762_4763','4763_6245','4765_4766','4766_4767','4767_4768','4768_1845','4768_4325','4769_4770','477_478','4770_4765','4779_7057','478_479','478_7053','4782_1782','4788_6273','479_480','4793_6068','4795_2686','4795_4688','4796_4690','4797_7473','4798_2674','4798_4715','4799_2205','48_49','480_481','4800_5157','4804_738','4806_5156','481_482','482_483','4825_7233','483_485','4830_7667','4843_4905','4844_4177','4847_4848','4848_4350','485_487','485_488','4854_1438','4855_1439','4856_4857','4857_307','4861_4862','4863_4864','4864_4865','4865_4577','4869_4870','487_488','4870_3007','4871_4872','4872_4873','4873_4874','4874_2532','4877_4878','4878_7180','488_489','488_491','4880_2539','4880_2606','4882_4883','4883_4884','4884_4885','4885_6343','4886_4887','4887_1150','4888_4798','489_490','4890_4369','4891_7458','4893_4894','4894_4895','4895_7217','4897_4898','4898_4574','4899_4900','49_51','490_491','4900_4746','4901_4902','4902_1672','4903_1883','4903_7219','4905_4906','4906_4907','4907_4908','4908_4909','4909_3884','4909_4921','491_492','491_493','4910_4911','4911_4912','4912_4913','4913_4914','4914_4915','4915_4957','4915_7654','4916_4917','4917_4918','4918_4919','4919_4920','492_493','4920_4910','4921_4922','4922_4923','4923_4924','4924_4958','4925_3679','4925_4926','4926_3678','4927_4928','4928_2352','4929_7460','493_494','4930_4931','4931_7459','4932_4933','4933_2590','4934_4500','4935_4502','4936_79','4937_1547','4938_1549','494_495','495_400','495_496','495_7587','495_7588','4952_768','4953_768','4957_7133','4958_3752','496_497','496_515','496_522','496_523','4960_4932','4962_264','4962_4725','4962_6059','497_515','4974_4976','4976_7126','4978_4979','4979_4986','4980_2289','4981_3211','4984_448','4985_6219','4986_7107','4987_2978','4988_2853','4989_1845','4989_4990','499_500','4990_4991','4991_4992','4992_1719','4994_4995','4995_4996','4996_1826','4997_4998','4998_4999','4999_1833','500_501','5008_4640','501_1484','5013_5014','5014_7291','5015_7290','5022_3461','5022_6002','5023_5024','5024_5161','5025_4854','5026_2450','5028_3696','5028_7115','5029_5028','5030_5031','5031_6033','5032_2824','5032_2867','5033_7130','5044_3751','5044_4433','5045_4460','5046_7639','5047_3146','5047_3224','5051_5052','5052_2975','5053_1633','5056_2239','5058_4602','5060_5061','5061_1231','5062_691','5064_7173','5069_3723','5071_5072','5072_4321','5072_7342','5073_3679','5073_5074','5074_6054','5075_5076','5076_3682','5077_3694','5077_5078','5078_3689','5079_3694','5081_5082','5082_3639','5084_6054','5086_3656','509_510','5090_4204','5091_5092','5092_7574','5096_7145','5097_2383','5099_2383','51_52','510_511','511_513','5110_4012','5111_4051','5112_4003','5113_3918','5118_3910','5119_5120','512_513','5120_5121','5121_5122','5122_3461','5123_5124','5124_5125','5125_6001','5127_5128','5128_2060','5128_2996','5129_5130','513_4513','513_7402','5130_1135','5131_5132','5132_1331','5133_2596','5134_2718','5135_4303','5136_4262','5139_4214','5140_1500','5140_620','5142_7567','5143_2837','5144_7027','5144_7702','5145_276','5147_68','5148_1401','5149_1419','515_516','5150_2327','5154_4804','5155_4806','5156_7552','5157_5154','516_4384','5160_5155','5161_5162','5162_2571','5162_4880','5164_4633','5165_3743','5165_7169','5169_2735','5171_2976','5174_6078','5175_3755','5176_3774','518_2255','519_521','5192_342','5192_350','52_261','52_262','52_263','52_265','52_277','52_4725','521_522','522_1741','522_523','523_1740','523_4382','523_525','523_669','525_526','526_527','527_528','528_529','529_530','53_54','530_531','531_709','532_533','532_649','533_534','534_535','535_536','536_537','537_538','538_539','539_540','54_55','540_541','541_542','542_543','543_544','544_545','544_546','545_546','546_547','546_910','546_914','547_548','548_549','549_550','549_677','55_56','550_551','551_552','552_553','553_554','554_555','555_557','557_558','558_559','559_560','56_58','560_562','562_563','563_564','564_565','565_566','566_567','566_707','567_568','568_569','569_570','57_58','570_571','571_572','572_573','573_574','574_575','575_576','576_577','577_579','579_580','58_4499','58_59','580_581','581_582','582_583','583_584','584_585','585_586','586_587','587_588','588_589','588_590','589_590','59_60','590_1729','590_591','591_592','592_593','593_594','594_595','595_596','596_597','597_598','598_599','599_600','6_194','60_1802','60_820','600_601','600_654','6001_4438','6002_5119','6003_5123','6004_3','6004_7','6005_1546','6005_7072','6006_3493','6007_2836','601_602','6010_3656','6011_2133','6013_6247','6014_2158','6014_6242','6015_5029','6016_6017','6017_7192','602_603','6020_6023','6021_7507','6023_6026','6024_6021','6025_6024','6026_6027','6027_6029','6028_6025','603_604','6030_6028','6033_3089','604_605','6041_2826','6042_1022','6048_5064','605_606','6050_3749','6050_7398','6051_6053','6052_3613','6053_7327','6054_4330','6055_1073','6057_2183','6059_334','6059_404','606_607','6068_1584','607_608','6070_2708','6074_748','6074_793','6075_2631','6076_6077','6077_3569','6078_7112','6079_6315','608_609','6080_1585','6080_983','6083_2733','6085_2026','6085_2053','6086_3161','6086_7419','6087_3620','6088_1898','6089_3981','609_610','6094_2102','6095_7062','6097_4093','610_611','6107_6108','6107_6109','6108_6109','6109_6110','611_612','6110_7020','6110_7029','6111_6112','6112_6113','6113_1898','6113_4323','6115_1205','6115_4593','6117_3655','6117_5079','6119_3392','612_613','6122_35','6124_4004','6125_7092','613_614','6130_1262','6132_2551','6132_4861','6135_3612','6137_4256','6137_4956','614_4415','614_615','614_675','6140_6144','6141_6142','6142_2181','6143_6011','6144_6145','6145_6146','6146_6147','6147_6149','6149_6150','615_616','6150_6143','6152_6153','6153_6243','6154_6155','6155_1987','6155_6141','6156_6165','6157_6158','6158_3964','6158_7314','6159_6160','616_617','6160_6161','6161_3991','6162_7022','6163_1095','6164_6159','6165_6157','617_618','6170_5023','6170_7183','6171_4974','6172_4493','618_619','6182_322','6185_6186','6186_6216','6187_6188','6188_6215','619_675','620_1172','620_621','620_7568','620_7569','621_289','621_291','621_293','621_4495','6215_6184','6216_3400','6219_6229','6228_4788','6228_7111','6229_6348','6229_6364','6229_7131','6230_6228','6234_332','6235_327','6238_6182','6239_104','624_625','6241_105','6242_6152','6243_6154','6244_2134','6245_2156','6245_4671','6245_6013','6247_2679','625_626','6251_5140','6251_7706','6252_301','6252_302','6253_7191','626_627','627_628','6273_136','6273_4891','6276_6362','6277_6278','6278_6279','6279_6280','628_629','6282_6335','6283_6282','6285_6286','6286_6287','6287_6288','6288_6342','629_630','6290_7120','6298_6336','63_64','630_4466','6310_6311','6311_4495','6311_7399','6312_7197','6315_6316','6316_7236','6318_6319','6319_7246','6320_4594','6324_6325','6325_6341','6326_4886','633_634','6334_7660','6335_6326','6336_7069','6339_6324','634_1005','6341_6277','6342_6298','6343_4759','6348_1182','635_636','6359_7189','636_637','6361_6367','6362_7276','6363_6361','6364_1558','6365_6276','6366_6230','6367_4985','637_638','638_639','639_640','64_65','640_641','641_642','642_643','643_644','644_645','645_1796','649_650','65_66','650_651','651_4472','653_722','654_655','655_656','656_657','657_658','658_659','659_604','66_4498','664_614','664_665','665_666','666_667','667_668','668_614','669_670','670_671','671_672','672_4382','675_1184','675_1500','675_303','675_4380','675_4415','675_620','675_621','677_678','678_679','679_680','68_30','680_681','681_682','682_683','683_684','684_685','685_686','686_688','688_689','689_690','69_7686','69_828','690_5062','691_692','692_693','692_694','692_695','693_585','694_695','695_696','696_697','697_698','698_699','699_700','7_11','7_194','700_701','7009_1604','701_702','7010_7703','7011_2171','7011_7160','7012_2201','7012_2644','7013_6162','7014_4315','7015_4617','7016_4619','7017_7018','7018_7030','702_703','7020_7029','7021_6111','7021_7163','7022_1554','7023_7014','7025_4464','7027_4538','7028_5147','7029_7038','703_704','7030_6111','7030_7021','7031_4897','7038_7011','7039_7204','704_705','7043_1424','7044_993','7047_7161','7048_7699','7049_2236','705_706','7050_7185','7053_2088','7056_4731','7057_4847','7057_5142','706_567','706_707','7062_6075','7063_1137','7064_7066','7065_7063','7066_2924','7067_2956','7067_2984','7068_2966','7069_7443','707_708','7070_7445','7073_7097','7074_3919','7075_3925','7078_1449','7079_7080','708_731','7080_7081','7081_7565','7082_7083','7083_7084','7084_7108','7085_7086','7086_7087','7087_7088','7088_7089','7089_7091','709_532','7091_3910','7092_110','7097_7098','7098_7099','7099_7100','71_72','710_711','7100_7101','7101_7505','7102_7103','7103_7104','7104_7105','7105_7106','7106_6005','7107_7102','7108_3935','711_712','7110_3938','7111_6365','7112_972','7113_127','7114_7116','7115_6310','7116_3694','7117_7118','7118_6289','7119_7127','712_713','7120_7119','7121_7183','7122_7121','7123_4347','7124_7343','7125_7344','7126_7124','7127_7143','7128_7117','713_714','7130_1237','7132_1558','7132_6230','7133_7134','7134_3867','7135_7136','7136_3874','7137_7139','7138_3370','7139_4623','714_715','7140_7141','7141_7138','7142_7015','7143_3368','7143_7168','7144_3881','7145_3884','7145_4843','7149_7277','715_716','7153_6056','7154_7153','7156_7020','7158_7048','7159_7017','7159_7388','716_717','7160_7700','7161_7162','7163_1893','7166_1847','7167_1808','7168_4616','7169_3743','717_718','7170_3740','7172_3775','7173_3811','7175_3806','7176_2836','7177_2740','7178_2765','718_719','7180_7123','7180_7331','7183_4880','7184_2225','7184_7050','7185_7186','7186_7187','7187_3947','7187_3954','7187_4001','7188_3082','719_720','7191_6238','7192_4683','7193_7049','7194_7196','7195_3653','7196_6310','7197_7195','72_5144','720_680','720_721','7201_2138','7204_3728','7204_3736','7207_7288','7208_7249','721_680','721_693','7214_4347','7216_2501','7217_7227','7218_7031','7219_7218','722_723','7222_6170','7224_1581','7227_6172','7229_3418','7229_4617','723_724','7230_4617','7231_7128','7233_4573','7234_4002','7236_7245','7237_7241','7238_7224','7239_4401','724_725','7240_1891','7241_7247','7242_7238','7243_7126','7245_6317','7246_6320','7247_7243','7248_7207','7248_7254','7249_7253','725_726','7250_7248','7250_7252','7251_4051','7252_7248','7253_7251','7254_7257','7255_7249','7256_7255','7257_7258','7258_7260','7259_7256','726_727','7260_7263','7261_7259','7262_7261','7263_7264','7264_7268','7265_7262','7266_7265','7268_7270','727_728','7270_6122','7270_7266','7271_5136','7272_4956','7276_7149','7277_6363','7278_7289','728_729','7280_7281','7281_7284','7282_7278','7283_7282','7284_7287','7286_7283','7287_7208','7288_7286','7289_7280','729_730','7290_5016','7291_3485','7292_3818','7293_1335','7294_4132','7294_4156','7296_4265','7296_4266','73_74','730_5160','731_710','7313_6164','7314_3964','7317_3224','7318_4978','7318_4980','7319_7342','7320_1486','7325_3317','7325_4984','7326_7418','7327_3629','7328_3651','7329_3659','733_6015','7330_2051','7331_2559','7333_783','7334_1838','7340_783','7342_7125','7343_4978','7343_4980','7343_7318','7344_2510','7344_7334','7347_3669','7348_3669','7348_3671','7349_1631','7350_1558','7350_6230','7352_4246','7353_4571','7353_7584','7354_1975','7358_1231','7358_1235','7358_5174','7361_7362','7362_3129','7362_3134','7363_4588','7364_7365','7365_4210','7366_7367','7367_7368','7368_3143','7370_7339','7374_1845','7375_3466','7379_4903','738_739','7380_1676','7381_4742','7382_7381','7386_4607','7388_7017','7389_4324','7389_7025','739_740','7391_493','7392_1443','7392_312','7395_6124','7396_5145','7397_5140','7397_6252','7398_6252','7398_7397','7399_7398','7399_7611','74_4371','740_741','7400_3669','7401_3665','7402_4380','7402_4717','741_742','7412_7413','7413_2383','7414_1396','7415_7416','7416_7325','7417_3471','7418_7417','7419_7420','742_743','7420_3345','7421_7422','7422_6085','7423_1566','7424_7423','7425_7424','7426_7427','7427_7428','7428_138','743_744','7430_177','7431_175','7432_3454','7433_4677','7435_2722','7435_4413','7438_1410','7439_7454','744_745','7440_2940','7441_2933','7442_7446','7443_7444','7444_2941','7445_6339','7446_7070','7449_2941','745_600','7453_1478','7453_1480','7453_2001','7454_1380','7455_7438','7458_4479','7458_7425','746_844','7460_4930','7461_4230','7461_4281','7462_7461','7469_1724','747_748','747_842','747_843','7471_1533','7472_3230','7473_2679','7475_6107','748_7390','748_749','749_494','749_750','749_751','75_4501','750_751','7505_5071','7505_7678','7507_2663','751_752','7510_2207','7510_4799','7513_807','7514_2279','7516_6079','752_7339','752_753','753_7339','753_7370','753_754','754_755','755_756','7550_2570','7551_4800','7552_924','7555_4962','7556_3539','7559_7073','7559_7107','756_757','7560_7679','7563_3368','7563_7231','7564_340','7564_4521','7565_7082','7567_7326','757_758','7571_207','7574_4177','7577_1353','7578_7582','7579_1285','758_759','758_760','7581_1283','7582_297','7582_299','7582_319','7582_340','7582_407','7584_3344','7585_7587','7586_7587','7587_303','7587_328','7587_345','7587_407','7587_496','7587_7588','7588_281','7588_325','7588_328','7588_329','7588_4717','759_760','7591_496','7592_3815','760_2007','760_761','760_762','7601_2508','7602_22','7602_85','7603_45','761_762','7611_7398','7612_2811','7616_5081','762_763','7622_297','7622_334','7623_2508','7623_3676','7623_3715','7623_3719','7623_5030','7623_6312','7623_7401','763_2007','763_4952','763_4953','763_767','7639_7640','7640_3134','7640_5047','7641_7642','7642_7643','7643_3214','7644_7645','7645_7646','7646_3238','7654_4843','7660_3114','7661_1521','7667_3091','767_768','7671_513','7672_826','7673_7674','7674_7675','7675_7676','7676_7677','7677_7559','7678_7560','7679_7680','768_2085','768_769','768_770','768_772','7680_7681','7681_7738','7683_1582','769_770','7690_3781','7691_3721','7692_6010','7698_7113','7699_7159','77_78','770_771','770_773','7700_7047','7701_7010','7702_7009','7703_5147','7704_3983','7705_6251','7706_7707','7707_512','7708_7709','7709_7710','771_772','7710_7711','7711_7705','772_417','772_773','772_775','773_774','7738_7683','774_775','774_906','775_776','775_906','776_777','777_779','777_906','7771_3966','779_780','78_79','780_781','781_782','782_783','783_784','783_785','784_785','785_786','786_791','786_792','786_793','79_80','790_7585','791_7585','792_320','792_7586','793_274','793_320','793_495','793_7586','795_796','796_56','796_57','796_797','797_798','798_799','799_800','8_12','80_82','800_801','801_802','802_1654','802_803','803_804','804_7513','807_808','808_809','809_810','81_817','810_811','811_812','812_813','813_814','814_81','817_818','818_819','819_261','819_263','819_264','819_265','819_4725','82_817','82_83','820_821','821_63','822_823','823_824','824_825','825_826','826_827','827_71','828_829','829_830','83_84','830_831','831_75','832_833','833_77','834_835','835_836','836_837','837_1506','838_839','839_840','84_51','840_841','841_49','842_1014','842_845','842_846','843_1013','843_845','844_846','845_846','846_7340','846_847','846_848','847_848','848_2795','848_758','848_759','848_849','849_850','85_203','85_86','850_851','851_852','852_853','853_854','854_855','855_2815','855_856','856_857','857_858','858_859','859_860','86_87','860_861','861_862','862_3014','87_88','875_876','876_3011','876_878','877_876','878_879','879_880','88_89','880_881','881_882','882_883','883_884','884_885','885_886','886_887','887_2900','887_903','89_27','893_894','894_1067','895_1025','896_897','897_893','90_91','903_904','904_906','905_4331','906_907','906_909','907_908','908_909','909_786','909_790','909_792','91_129','91_92','910_911','911_912','912_913','913_914','914_938','915_916','916_917','917_918','918_919','919_7551','92_93','924_925','925_927','927_938','928_588','928_929','929_930','93_94','930_931','931_587','938_940','94_102','94_126','940_941','941_942','942_943','943_3599','944_945','945_947','946_1155','947_948','948_949','949_928','950_915','956_957','957_958','958_959','959_960','960_961','961_962','962_635','969_1176','972_1625','980_981','981_1518','981_1576','983_984','984_985','985_1529','987_4937','992_7044','993_5060']

Loop through all segments and add relevant information to the JSON file:

In [ ]:
file = open("/data_analytics/JSON/dec_segments.json", "w", encoding="utf8")
file.write("{\n")
for i in range(len(segments)):
    segment = segments[i]
    df_temp = df.loc[df.segment == segment]
    if df_temp.shape[0] != 0:
        mean = df_temp['segment_mean'].iloc[0]
        std = df_temp['segment_std'].iloc[0]
        if i != (len(segments)-1):
            file.write('"' + str(segment) + '": {"mean": ' + str(mean) + ', "std": ' + str(std) + '},\n')
        else:
            file.write('"' + str(segment) + '": {"mean": ' + str(mean) + ', "std": ' + str(std) + '}\n')
file.write("}")
file.close()